# Do logarithmic proximity measures outperform plain ones in graph clustering?

In [9]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('../..')

In [10]:
import numpy as np
from numpy import array, float64
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import adjusted_rand_score
from joblib import Parallel, delayed
from collections import defaultdict

In [11]:
from pygraphs.graphs.generator import StochasticBlockModel
from pygraphs.graphs.dataset import Datasets
from pygraphs.measure import ALL_kernels
from pygraphs.cluster import KernelKMeansSklearn
from pygraphs.scenario import ParallelByGraphs, measures_right_order, d3_right_order
from pygraphs.util import load_or_calc_and_save, configure_logging
from pygraphs.scorer import copeland

In [12]:
import logging
configure_logging()
logger = logging.getLogger()

## 6. Cluster analysis on several classical datasets

For each dataset and each measure family, we sorted 55 values of the family parameter in the descending order of the
corresponding ARI. ARI against the rank of the family parameter value is shown in Fig. 9.

In [13]:
results = {'cora': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 1.  ]),
   array([ 0.00039824,  0.00576553,  0.02263988,  0.02263988,  0.02263988,
           0.02263988,  0.02263988,  0.02263988,  0.02263988,  0.02220336,
           0.02327132,  0.02327132,  0.00094126,  0.02057234, -0.00189327,
          -0.00189327,  0.02533785,  0.04128937,  0.04128937,  0.02566891,
           0.02566891,  0.02259993,  0.02587255,  0.0275816 ,  0.02777803,
           0.02234255,  0.01110426,  0.01868969,  0.02669662,  0.01120598,
           0.01045815,  0.01739105,  0.01972438,  0.01422882,  0.00033004,
           0.00812332,  0.00071765,  0.02373063,  0.00036427,  0.01401326,
           0.04357828,  0.07225023,  0.01978767,  0.03876626,  0.03073889,
           0.03027161,  0.02416613,  0.04297379,  0.07740777])),
  'Walk H': (array([], dtype=float64), array([], dtype=float64)),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 3.98242511e-04,  1.41388671e-04, -2.78987152e-05, -4.52377348e-04,
           1.86602317e-02, -1.25472461e-04, -7.47688457e-05,  5.40865872e-04,
           1.54064575e-04, -3.25106162e-03,  1.06580250e-03, -4.87736448e-04,
           2.15442636e-04, -2.31851849e-03,  5.32447099e-03,  4.11001174e-02,
          -1.01623862e-03,  5.87696640e-05, -4.41396606e-03,  4.71559668e-04,
          -1.67589011e-04,  9.43288248e-04,  8.24738699e-04,  3.41079551e-04,
           4.82664530e-04,  3.76349627e-03, -2.15517869e-03,  2.06113029e-02,
          -1.42474519e-03, -5.44991181e-04,  9.33281258e-04,  2.15729782e-04,
           1.18394720e-03,  1.52533830e-03,  1.50290164e-03, -3.35339290e-03,
          -2.59535443e-04,  8.23104324e-03, -2.21751703e-03, -3.23007875e-03,
           8.75952885e-04, -1.51607406e-02, -1.70707726e-03,  7.00788726e-04,
           1.14810621e-01, -1.36952604e-04,  1.04771777e-03,  5.08054238e-04,
          -4.11234535e-04, -8.38820532e-04])),
  'logFor H': (array([], dtype=float64), array([], dtype=float64)),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00039824,  0.02263988,  0.00073277, -0.00739692,  0.0275816 ,
           0.00675525,  0.01014141,  0.01972868,  0.02945182,  0.02985891,
           0.0338612 ,  0.02162643,  0.01493372,  0.0206303 ,  0.01930634,
           0.01895305,  0.02426996,  0.02046978,  0.01732853,  0.0454776 ,
           0.01799596,  0.01799596,  0.04469691,  0.07128905,  0.07075456,
           0.07075456,  0.07083916,  0.07075456,  0.07050002,  0.06846448,
           0.0658303 ,  0.06212824,  0.07892732,  0.07727074,  0.07702596,
           0.07702974,  0.0777302 ,  0.07740777,  0.07740777,  0.07706906,
           0.07706906,  0.07740777,  0.07740777,  0.07740777,  0.07740777,
           0.07605812,  0.07605812,  0.07605812,  0.07605812,  0.07605812])),
  'logComm H': (array([], dtype=float64), array([], dtype=float64)),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 3.98242511e-04, -4.03008038e-04,  2.11743264e-03,  3.06865675e-03,
           7.45664435e-03,  1.08441507e-03,  3.28803549e-06, -3.74082872e-05,
           4.73953981e-03,  5.54134223e-04, -1.38557699e-03,  9.09735519e-05,
          -3.75936217e-04,  3.73322036e-04, -1.01606614e-03,  1.32157288e-05,
          -9.74203733e-04,  3.15968074e-03,  1.25386070e-03, -1.71635876e-03,
          -9.86394643e-04,  9.57454949e-04,  1.38993686e-03, -8.52150260e-04,
          -7.59906552e-05, -1.20354150e-03, -1.94432265e-03, -6.68316466e-03,
           4.06468482e-04, -2.24646332e-03, -1.50009740e-03,  5.43483229e-03,
          -8.95487501e-04,  1.12927292e-03, -1.01911677e-03, -7.42893781e-04,
          -1.27937145e-03, -1.83358741e-03, -2.46208433e-03, -7.98128226e-04,
          -3.98468757e-04, -6.27572952e-04,  4.12613825e-04, -1.06959582e-03,
          -1.66199805e-03, -1.24098049e-03,  1.77282004e-03, -5.47565066e-04,
          -2.07131867e-03, -2.94012433e-03])),
  'logHeat H': (array([], dtype=float64), array([], dtype=float64)),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.        ,  0.00257124,  0.00229316,  0.11714702,  0.03655967,
          -0.00041434,  0.11429589,  0.15647444,  0.16461885,  0.16101856,
           0.16990602,  0.14645909,  0.1035872 ,  0.16905185,  0.16553034,
           0.26364019,  0.19057975,  0.15424521,  0.30413787,  0.15469055,
           0.30891181,  0.30914015,  0.28497409,  0.31443688,  0.28525184,
           0.14693564,  0.29827793,  0.2003839 ,  0.30910834,  0.31033323,
           0.3110207 ,  0.31048223,  0.21626544,  0.31373592,  0.2168971 ,
           0.21927344,  0.32102843,  0.33098973,  0.33045981,  0.32442587,
           0.28964919,  0.32858394,  0.35067554,  0.33568558,  0.3353446 ,
           0.33643041,  0.34791251,  0.40148547,  0.33583343,  0.37836677])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([ 0.00000000e+00,  7.77293437e-04, -8.41001480e-04, -1.24217452e-03,
           1.15477142e-04,  8.25655210e-05,  1.00479940e-04, -1.14115887e-03,
          -1.14115887e-03, -1.14115887e-03, -1.14115887e-03, -1.06555693e-03,
          -1.40454526e-03, -1.49486375e-03, -1.39983728e-03, -1.40308559e-03,
          -1.39983728e-03, -1.82531174e-03, -1.39983728e-03, -1.82274898e-03,
          -1.82274898e-03, -1.82274898e-03, -1.82274898e-03, -1.82274898e-03,
          -1.82274898e-03, -1.82274898e-03, -1.82274898e-03, -1.82740514e-03,
          -1.82274898e-03, -2.23306344e-03, -2.43604576e-03, -2.43409861e-03,
          -2.42461640e-03, -2.42461640e-03, -2.42461640e-03, -1.60671665e-03,
          -2.67637939e-03, -2.67637939e-03, -2.67637939e-03, -2.67637939e-03,
          -2.08734199e-03, -1.62402221e-03, -1.62402221e-03, -2.68640284e-03,
          -2.70475750e-03, -2.63970898e-03, -2.62967655e-03, -2.61317881e-03,
          -2.65463764e-03, -3.06829312e-03, -5.47192998e-03])),
  'RSP K': (array([], dtype=float64), array([], dtype=float64)),
  'FE K': (array([], dtype=float64), array([], dtype=float64)),
  'SP-CT H': (array([], dtype=float64), array([], dtype=float64)),
  'NormalizedHeat R': (array([], dtype=float64), array([], dtype=float64)),
  'PersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'ModifiedPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'HeatPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64))},
 'football': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-0.00132797,  0.75239016,  0.75239016,  0.75239016,  0.75239016,
           0.75239016,  0.74442711,  0.74442711,  0.74442711,  0.74442711,
           0.74442711,  0.75239016,  0.67418939,  0.71492627,  0.74172097,
           0.74172097,  0.69797154,  0.77356709,  0.77356709,  0.77356709,
           0.77356709,  0.77356709,  0.77356709,  0.73391525,  0.73391525,
           0.73391525,  0.84132357,  0.77152179,  0.7911229 ,  0.80958758,
           0.80958758,  0.80751079,  0.82663201,  0.82663201,  0.88915225,
           0.90234207,  0.90234207,  0.90234207,  0.90234207,  0.84006864,
           0.84006864,  0.89665001,  0.85603465,  0.85603465,  0.89665001,
           0.89665001,  0.90633416,  0.89665001,  0.90633416,  0.90251231,
           0.17451294])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.88934316, 0.88934316, 0.88392028, 0.88392028, 0.88392028,
          0.88392028, 0.88392028, 0.88392028, 0.88392028, 0.88392028,
          0.88392028, 0.88392028, 0.88392028, 0.88392028, 0.81859537,
          0.81859537, 0.81859537, 0.81859537, 0.81859537, 0.8927024 ,
          0.8927024 , 0.8927024 , 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88544205, 0.88544205, 0.8927024 , 0.89665001,
          0.8927024 , 0.87393977, 0.88544205, 0.89494799, 0.89494799,
          0.87393977, 0.88783563, 0.8927024 , 0.8927024 , 0.8927024 ,
          0.88544205, 0.8927024 , 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88544205, 0.89854076, 0.90441989, 0.19238868])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00132797,  0.44549291,  0.55871674,  0.55871674,  0.51953736,
           0.53621611,  0.61432595,  0.74171324,  0.70819866,  0.67316157,
           0.73465021,  0.70362773,  0.81457385,  0.81457385,  0.82614072,
           0.82614072,  0.82758487,  0.82754024,  0.82932002,  0.83972734,
           0.88844666,  0.80397554,  0.88844666,  0.88844666,  0.86569421,
           0.88524475,  0.86569421,  0.86324506,  0.86324506,  0.88451848,
           0.88451848,  0.88451848,  0.88451848,  0.88451848,  0.88934316,
           0.88934316,  0.88934316,  0.88934316,  0.88934316,  0.88934316,
           0.88934316,  0.8117922 ,  0.8117922 ,  0.80637474,  0.80637474,
           0.82481881,  0.89213832,  0.89665001,  0.86569421,  0.86569421])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.88392028, 0.88392028, 0.89494799, 0.81859537, 0.81859537,
          0.81859537, 0.88934316, 0.88934316, 0.88783563, 0.88783563,
          0.86436524, 0.8927024 , 0.86436524, 0.82068048, 0.86281916,
          0.86281916, 0.89665001, 0.89665001, 0.89665001, 0.89665001,
          0.89665001, 0.89665001, 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.8882223 , 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88544205, 0.88544205, 0.88544205, 0.89665001,
          0.89665001, 0.88544205, 0.89665001, 0.89665001, 0.89665001,
          0.88934316, 0.8927024 , 0.8927024 , 0.8927024 , 0.89665001,
          0.8927024 , 0.8927024 , 0.89665001, 0.89665001])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00132797,  0.75239016,  0.75239016,  0.75239016,  0.77356709,
           0.84827677,  0.86981057,  0.82663201,  0.82872078,  0.89157424,
           0.89157424,  0.89665001,  0.86262343,  0.89665001,  0.88934316,
           0.90633416,  0.89665001,  0.90633416,  0.90633416,  0.90633416,
           0.90633416,  0.90633416,  0.90633416,  0.90921814,  0.90921814,
           0.90921814,  0.90921814,  0.90921814,  0.90921814,  0.90937709,
           0.90937709,  0.90937709,  0.90346527,  0.90346527,  0.90346527,
           0.85156076,  0.81222753,  0.64912723,  0.52891806,  0.41407614,
           0.29667606,  0.24028432,  0.1828624 ,  0.20286422,  0.17451294,
           0.17451294,  0.17451294,  0.17451294,  0.17451294,  0.17451294])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.88934316, 0.81739282, 0.86089391, 0.88934316, 0.88392028,
          0.8927024 , 0.81474978, 0.88392028, 0.88544205, 0.88544205,
          0.88544205, 0.79763094, 0.81507327, 0.88544205, 0.81507327,
          0.8927024 , 0.88544205, 0.88544205, 0.85016546, 0.88544205,
          0.88114635, 0.88114635, 0.88114635, 0.88544205, 0.88544205,
          0.88544205, 0.8880293 , 0.87992433, 0.89063899, 0.86041395,
          0.89063899, 0.90475255, 0.89552577, 0.87413167, 0.79954568,
          0.69230139, 0.61285365, 0.45138964, 0.43243585, 0.33815789,
          0.24468345, 0.19513816, 0.18926773, 0.19238868, 0.19238868,
          0.19238868, 0.19238868, 0.19238868, 0.19238868])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00132797,  0.44549291,  0.43533753,  0.53081663,  0.53017482,
           0.53321501,  0.67477764,  0.65645544,  0.82614072,  0.83375092,
           0.90589882,  0.85329978,  0.88934316,  0.89512954,  0.89665001,
           0.89665001,  0.89665001,  0.82068048,  0.88934316,  0.89665001,
           0.89665001,  0.89665001,  0.89665001,  0.89665001,  0.89665001,
           0.89665001,  0.8927024 ,  0.8927024 ,  0.8927024 ,  0.88392028,
           0.89008381,  0.87888857,  0.85987315,  0.84098391,  0.85016546,
           0.84098391,  0.83746354,  0.81084298,  0.82146362,  0.77982388,
           0.70057903,  0.67911636,  0.60765087,  0.55416413,  0.47265694,
           0.42443665,  0.34258086,  0.29853815,  0.29711573,  0.27707296])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.88934316, 0.81739282, 0.8053639 , 0.88934316, 0.88934316,
          0.8927024 , 0.81474978, 0.88392028, 0.87585397, 0.89494799,
          0.83714288, 0.88783563, 0.88783563, 0.81474978, 0.794512  ,
          0.88392028, 0.88392028, 0.8927024 , 0.88392028, 0.88392028,
          0.88544205, 0.88544205, 0.82099417, 0.88544205, 0.88544205,
          0.86254811, 0.80312954, 0.81683124, 0.86254811, 0.79343643,
          0.8196106 , 0.79343643, 0.85016546, 0.85016546, 0.83746354,
          0.8258431 , 0.8258431 , 0.82146362, 0.78399531, 0.75132835,
          0.73777279, 0.64757353, 0.5416253 , 0.47265694, 0.42443665,
          0.34258086, 0.29853815, 0.29711573, 0.1650723 ])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.86569421, 0.86569421, 0.86569421, 0.86569421,
          0.86569421, 0.86569421, 0.82634971, 0.80883979, 0.83229553,
          0.83229553, 0.88934316, 0.89552577, 0.89665001, 0.8927024 ,
          0.8927024 , 0.89665001, 0.89665001, 0.89665001, 0.89665001,
          0.89665001, 0.89665001, 0.89665001, 0.8927024 , 0.89665001,
          0.89665001, 0.89665001, 0.89665001, 0.8927024 , 0.8927024 ,
          0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 , 0.88544205,
          0.88544205, 0.88544205, 0.88544205, 0.88544205, 0.88392028,
          0.88544205, 0.88544205, 0.88544205, 0.79952358, 0.76914353,
          0.8611837 , 0.82427893, 0.80073733, 0.71002954, 0.72143893])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 ,
          0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 ,
          0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 , 0.9004381 ,
          0.9004381 , 0.88544205, 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88544205, 0.88544205, 0.8927024 , 0.8927024 ,
          0.8927024 , 0.88544205, 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88544205, 0.88544205, 0.88544205, 0.87200254,
          0.84843709, 0.77112122, 0.83020319, 0.82365676, 0.82673294,
          0.77332082, 0.80450693, 0.78651545, 0.78399531, 0.71891356,
          0.77758527, 0.73073368, 0.78834119, 0.74296456, 0.61129517])),
  'RSP K': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.89854076, 0.89854076, 0.89854076, 0.89854076, 0.89854076,
          0.89854076, 0.89854076, 0.89854076, 0.89854076, 0.89854076,
          0.89854076, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.88934316, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.88934316, 0.88934316, 0.86817601, 0.80122752, 0.80122752,
          0.80122752, 0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 ,
          0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 ,
          0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 , 0.88392028,
          0.8863672 , 0.89665001, 0.84028612, 0.89665001, 0.89665001,
          0.89665001, 0.8248507 , 0.88544205, 0.88934316])),
  'FE K': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.88934316, 0.88934316, 0.88934316, 0.82634971, 0.82634971,
          0.90234207, 0.90234207, 0.90234207, 0.90234207, 0.90234207,
          0.90234207, 0.90234207, 0.90234207, 0.90234207, 0.89836389,
          0.90234207, 0.90234207, 0.82068048, 0.87107156, 0.87107156,
          0.87107156, 0.87107156, 0.87107156, 0.87107156, 0.88934316,
          0.88934316, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.89665001, 0.89665001, 0.89665001, 0.89665001, 0.89665001,
          0.83050602, 0.81859537, 0.88934316, 0.81859537, 0.86281916,
          0.8882223 , 0.8882223 , 0.88934316, 0.89665001, 0.88544205,
          0.89665001, 0.89665001, 0.88934316, 0.89665001])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.86569421, 0.89552577, 0.89176454, 0.88544205, 0.89176454,
          0.8062306 , 0.8927024 , 0.89854076, 0.88934316, 0.88934316,
          0.88934316, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.88934316, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.88934316, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.88934316, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.88934316, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.88934316, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.88934316, 0.88934316, 0.88934316, 0.88934316, 0.88934316,
          0.89854076, 0.89854076, 0.89854076, 0.89854076, 0.89854076,
          0.89854076])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96]),
   array([0.8882223 , 0.89665001, 0.89665001, 0.86564501, 0.88544205,
          0.88544205, 0.88544205, 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88544205, 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88544205, 0.81739282, 0.81739282, 0.81859537,
          0.81859537, 0.81859537, 0.81859537, 0.88392028, 0.88392028,
          0.88392028, 0.88392028, 0.88392028, 0.88392028, 0.88392028,
          0.82068048, 0.88544205, 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88934316, 0.88392028, 0.88392028, 0.88392028,
          0.88544205, 0.82099417, 0.88392028, 0.82099417, 0.88544205,
          0.86281916, 0.87222072, 0.83796936])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.89552577, 0.81557193, 0.86281916, 0.86281916, 0.89665001,
          0.81739282, 0.81739282, 0.89723722, 0.89723722, 0.8927024 ,
          0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 ,
          0.8927024 , 0.88392028, 0.88392028, 0.88392028, 0.8927024 ,
          0.8927024 , 0.88544205, 0.88934316, 0.88934316, 0.89836389,
          0.88783563, 0.88544205, 0.88934316, 0.88934316, 0.89665001,
          0.8927024 , 0.88934316, 0.8927024 ])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.89552577, 0.81557193, 0.86281916, 0.86281916, 0.89665001,
          0.81739282, 0.81739282, 0.89723722, 0.89723722, 0.8927024 ,
          0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 ,
          0.8927024 , 0.88392028, 0.88392028, 0.88392028, 0.8927024 ,
          0.8927024 , 0.88544205, 0.88934316, 0.88934316, 0.89836389,
          0.88783563, 0.88544205, 0.88934316, 0.88934316, 0.89665001,
          0.8927024 , 0.88934316, 0.8927024 ])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.8882223 , 0.89665001, 0.89665001, 0.89665001, 0.88544205,
          0.88544205, 0.88544205, 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88544205, 0.88544205, 0.88544205, 0.88544205,
          0.88544205, 0.88544205, 0.81947976, 0.81947976, 0.81947976,
          0.81947976, 0.81947976, 0.89889265, 0.89889265, 0.89889265,
          0.88392028, 0.88392028, 0.88392028, 0.88392028, 0.88392028,
          0.88392028, 0.8927024 , 0.8927024 , 0.8927024 , 0.8927024 ,
          0.8927024 , 0.89665001, 0.88392028, 0.88392028, 0.88392028,
          0.82754024, 0.82754024, 0.88392028, 0.81298125]))},
 'news_2cl_1': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.        , 0.13486726, 0.13486726, 0.13486726, 0.02139042,
          0.11727066, 0.09756313, 0.08845769, 0.08260112, 0.08560006,
          0.07981965, 0.01879438, 0.01756272, 0.01760509, 0.01504643,
          0.0126886 , 0.0126886 , 0.0126886 , 0.01053157, 0.00952835,
          0.00767248, 0.00681982, 0.00681982, 0.00681982, 0.00681982,
          0.00681982, 0.00681982, 0.04967378, 0.03341158, 0.0316049 ,
          0.0316049 , 0.0316049 , 0.02814224, 0.02814224, 0.02814224,
          0.02648626, 0.02648626, 0.02648626, 0.02648626, 0.02488051,
          0.02488051, 0.01760509, 0.01630066, 0.02208148, 0.01768189,
          0.02528785, 0.02528785, 0.0176365 , 0.01486653, 0.01620471,
          0.01620471])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00765618,  0.00675229, -0.00195161, -0.00203718,  0.01102453,
           0.00037574,  0.00684959,  0.00593149,  0.00781795,  0.00683213,
           0.00683213,  0.00780075,  0.00780075,  0.00499465,  0.00501115,
           0.00501115,  0.00419275,  0.00419275,  0.00419275,  0.00501115,
           0.00080178,  0.00202151,  0.00198791,  0.00198791,  0.00198791,
           0.00136973,  0.00136973,  0.00136973, -0.00023311, -0.00023311,
          -0.00065109, -0.00065109, -0.00023311, -0.00023311, -0.00023311,
           0.00019067,  0.00122923,  0.00252105,  0.00252105,  0.00252105,
           0.00252105,  0.00252105,  0.00483567,  0.00567175,  0.00567175,
           0.00565676,  0.01576571,  0.01714286,  0.03368033])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00000000e+00,  0.00000000e+00,  5.01278134e-05,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  5.01278134e-05,  5.01278134e-05,  5.01278134e-05,
           0.00000000e+00,  0.00000000e+00, -4.98771743e-05, -4.98771743e-05,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-1.76698015e-03,  7.81794548e-03,  4.79917307e-04,  8.55262258e-01,
           8.55262258e-01,  8.55262258e-01,  8.55262258e-01,  8.64560587e-01,
           8.64560587e-01,  8.64560553e-01,  8.64560553e-01,  8.64560553e-01,
           8.73908990e-01,  8.73908990e-01,  8.73908990e-01,  8.73908990e-01,
           8.92756279e-01,  8.92756279e-01,  8.92756279e-01,  8.92756279e-01,
           8.92756279e-01,  8.92756279e-01,  8.92756279e-01,  8.92756279e-01,
           8.92756279e-01,  8.92756279e-01,  8.92756279e-01,  8.92756279e-01,
           8.92756279e-01,  8.92756279e-01,  9.02255123e-01,  9.11803963e-01,
           9.11803963e-01,  9.11803963e-01,  9.11803963e-01,  9.11803963e-01,
           9.11803963e-01,  9.11803963e-01,  9.11803963e-01,  9.21403015e-01,
           9.21403015e-01,  9.11803963e-01,  9.11803963e-01,  9.11803963e-01,
           9.11803963e-01,  9.11803963e-01,  9.11803963e-01,  9.21403035e-01,
          -4.98771743e-05])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.13486726, 0.02293535, 0.08260112, 0.07981965,
          0.01630066, 0.0126886 , 0.00952835, 0.00681982, 0.03341158,
          0.02814224, 0.02984845, 0.02488051, 0.02488051, 0.01760509,
          0.01760509, 0.01630066, 0.01630066, 0.01630066, 0.01630066,
          0.01630066, 0.01630066, 0.01630066, 0.01630066, 0.01630066,
          0.01631588, 0.01631588, 0.01631588, 0.01624769, 0.01620471,
          0.01620471, 0.01620471, 0.01486653, 0.01486653, 0.01357855,
          0.01620471, 0.01620471, 0.01620471, 0.01620471, 0.01620471,
          0.01620471, 0.01620471, 0.01620471, 0.01620471, 0.01620471,
          0.01620471, 0.01620471, 0.01622595, 0.01622595, 0.01622595])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00105051, -0.00215447,  0.00043674,  0.00593149,  0.00781795,
           0.00780075,  0.00780075,  0.0058798 ,  0.00501115,  0.00419275,
           0.00419275,  0.00337504,  0.00414397,  0.00080178,  0.00198791,
           0.00025094, -0.00023311, -0.00023311, -0.00023311, -0.00020045,
           0.00258244,  0.00129321,  0.00012662,  0.00183898,  0.00251027,
           0.00399386,  0.00481617,  0.00397584,  0.00319476,  0.00396758,
           0.01316217,  0.02157581,  0.02157581,  0.02157581,  0.03368033,
           0.03560949,  0.03560949,  0.03560949,  0.03561095,  0.03561095,
           0.03561095,  0.03561095,  0.03561095,  0.03561095,  0.03561095,
           0.03561095,  0.03561095,  0.03561095,  0.03561095])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00000000e+00,  0.00000000e+00,  5.01278134e-05,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00, -4.98771743e-05, -4.98771743e-05, -4.98771743e-05,
          -4.98771743e-05,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-1.36851378e-03, -2.20490592e-03, -2.80225989e-04,  4.36741103e-04,
           4.79917307e-04,  1.50678799e-05,  2.95313481e-03,  8.55262258e-01,
           8.55262258e-01,  8.64560587e-01,  8.55262258e-01,  8.64560587e-01,
           8.64560587e-01,  8.64560553e-01,  8.73908990e-01,  8.73908990e-01,
           8.73908990e-01,  8.73908990e-01,  8.92756279e-01,  8.92756279e-01,
           9.02255050e-01,  9.02255050e-01,  9.02255050e-01,  9.02255050e-01,
           9.21403035e-01,  9.21403035e-01,  9.21403035e-01,  9.21403035e-01,
           9.02255123e-01,  9.02255123e-01,  9.02255123e-01,  8.83307802e-01,
          -4.96277636e-05, -4.96277636e-05, -4.96277636e-05, -4.96277636e-05,
          -4.96277636e-05, -4.96277636e-05,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -4.98771743e-05,
          -4.98771743e-05, -4.98771743e-05, -4.98771743e-05, -4.98771743e-05,
           0.00000000e+00])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00000000e+00,  0.00000000e+00, -4.98771743e-05, -4.98771743e-05,
          -4.98771743e-05, -4.98771743e-05,  5.01278134e-05,  5.01278134e-05,
          -4.96277636e-05, -4.96277636e-05, -4.96277636e-05, -4.96277636e-05,
          -4.96277636e-05, -4.96277636e-05, -9.92579715e-05, -9.92579715e-05,
           9.02255123e-01,  9.11804007e-01,  9.21403035e-01,  9.21403035e-01,
           9.21403035e-01,  9.21403035e-01,  8.73909371e-01,  9.21403035e-01,
           9.11803963e-01,  9.11803963e-01,  8.73909054e-01,  8.73909054e-01,
           8.73909054e-01,  8.83307567e-01,  8.92756225e-01,  9.02255025e-01,
           9.02255025e-01,  9.02255025e-01,  8.83307567e-01,  8.83307567e-01,
           9.11803963e-01,  9.11803963e-01,  9.11803963e-01,  9.11803963e-01,
           9.11803963e-01,  9.21403015e-01,  9.21403015e-01,  9.21403015e-01,
           9.21403015e-01,  9.21403015e-01,  9.21403015e-01,  8.92756549e-01,
           8.92756549e-01,  8.64561404e-01])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00000000e+00, -4.98771743e-05, -4.98771743e-05,  5.01278134e-05,
          -4.96277636e-05,  4.31985924e-02,  4.53067358e-02,  3.91330524e-02,
           3.91330524e-02,  4.11406986e-02,  8.92756225e-01,  8.92756225e-01,
           8.92756225e-01,  8.92756225e-01,  8.92756225e-01,  8.92756225e-01,
           8.92756225e-01,  9.02255050e-01,  9.02255050e-01,  9.02255050e-01,
           9.02255050e-01,  9.02255050e-01,  9.02255050e-01,  9.02255050e-01,
           9.31052203e-01,  9.31052203e-01,  9.31052203e-01,  9.21403015e-01,
           9.21403015e-01,  9.21403015e-01,  9.21403015e-01,  9.21403015e-01,
           9.21403015e-01,  9.21403015e-01,  9.21403015e-01,  9.21403015e-01,
           9.21403015e-01,  9.21403015e-01,  9.21403015e-01,  9.21403015e-01,
           9.21403015e-01,  9.21403015e-01,  9.21403015e-01,  9.11803963e-01,
           9.11803963e-01,  9.02255025e-01,  9.02255025e-01,  8.83307567e-01,
           8.73909054e-01,  8.64560689e-01])),
  'RSP K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.74759698, 0.74759698, 0.74759698, 0.74759698, 0.74759698,
          0.73894993, 0.73894993, 0.74759698, 0.74759698, 0.74759698,
          0.74759698, 0.74759698, 0.74759698, 0.74759698, 0.74759698,
          0.74759698, 0.73895104, 0.73895104, 0.73895104, 0.73895104,
          0.73895104, 0.73895104, 0.73895104, 0.74759698, 0.74759698,
          0.74759698, 0.74759698, 0.74759698, 0.74759698, 0.74759812,
          0.75629416, 0.75629311, 0.75629311, 0.76503944, 0.76503944,
          0.75629311, 0.75629311, 0.74759698, 0.75629311, 0.75629311,
          0.77383595, 0.80052658, 0.80052658, 0.82766848, 0.81857057,
          0.82766848, 0.88330765])),
  'FE K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.74759698, 0.74759698, 0.74759698, 0.74759698, 0.75629311,
          0.75629311, 0.76504038, 0.76504038, 0.76504038, 0.7915802 ,
          0.80052718, 0.80052718, 0.80052718, 0.80052718, 0.7915802 ,
          0.7915802 , 0.7915802 , 0.7915802 , 0.7915802 , 0.7915802 ,
          0.7915802 , 0.7915802 , 0.7915802 , 0.7915802 , 0.80052718,
          0.83681685, 0.84601469, 0.84601469, 0.83681685, 0.83681685,
          0.83681685, 0.84601504, 0.84601504, 0.83681685, 0.84601469,
          0.84601469, 0.84601469, 0.84601469, 0.8645611 , 0.8645611 ,
          0.8833078 , 0.8833078 , 0.89275639, 0.89275639, 0.90225503,
          0.91180396, 0.92140302])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.        , 0.76503861, 0.76503861, 0.76503861, 0.76503861,
          0.76503861, 0.76503861, 0.76503861, 0.76503861, 0.76503861,
          0.76503861, 0.76503861, 0.76503861, 0.76503861, 0.76503861,
          0.76503861, 0.76503861, 0.76503861, 0.76503861, 0.76503861,
          0.76503861, 0.74759698, 0.74759698, 0.74759698, 0.74759698,
          0.74759698, 0.74759698, 0.74759698, 0.74759698, 0.74759698,
          0.74759698, 0.74759698, 0.74759698, 0.74759698, 0.74759698,
          0.74759698, 0.74759698, 0.74759698, 0.74759698, 0.74759698,
          0.74759698, 0.74759698, 0.74759698, 0.74759698, 0.74759698,
          0.74759698, 0.74759698, 0.74759698, 0.74759698, 0.74759698,
          0.74759698])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94]),
   array([0.00248095, 0.00323182, 0.00864482, 0.00764446, 0.00573994,
          0.00868964, 0.76503649, 0.77383339, 0.77383322, 0.80052568,
          0.81857057, 0.81857057, 0.82766826, 0.82766826, 0.83681611,
          0.85526233, 0.85526233, 0.87390899, 0.86456059, 0.86456059,
          0.86456059, 0.87390899, 0.87390899, 0.88330757, 0.88330757,
          0.89275628, 0.90225505, 0.90225505, 0.90225505, 0.89275628,
          0.90225505, 0.90225505, 0.90225505, 0.91180396, 0.92140302,
          0.91180396, 0.91180396, 0.91180396, 0.89275628, 0.89275628,
          0.89275628, 0.89275639, 0.90225525, 0.86456086, 0.00865528,
          0.00401388, 0.00114567])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.12383178, 0.73035123, 0.74759242, 0.76503625, 0.77383322,
          0.79157848, 0.79157848, 0.80952338, 0.81857039, 0.81857039,
          0.82766826, 0.83681611, 0.84601423, 0.85526233, 0.85526248,
          0.87390899, 0.87390905, 0.88330757, 0.88330757, 0.89275623,
          0.89275623, 0.90225505, 0.90225503, 0.90225503, 0.91180396,
          0.91180396, 0.91180396, 0.91180396, 0.92140303, 0.92140303,
          0.92140303, 0.92140303, 0.89275628])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.12383178, 0.73035123, 0.74759242, 0.76503625, 0.77383322,
          0.79157848, 0.79157848, 0.80952338, 0.81857039, 0.81857039,
          0.82766826, 0.83681611, 0.84601423, 0.85526233, 0.85526248,
          0.87390899, 0.87390905, 0.88330757, 0.88330757, 0.89275623,
          0.89275623, 0.90225505, 0.90225503, 0.90225503, 0.91180396,
          0.91180396, 0.91180396, 0.91180396, 0.92140303, 0.92140303,
          0.92140303, 0.92140303, 0.89275628])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.04384594, 0.12383178, 0.72180388, 0.73035123, 0.74759242,
          0.73894573, 0.76503625, 0.73894573, 0.76503613, 0.77383317,
          0.79157848, 0.79157848, 0.79157848, 0.80952338, 0.81857039,
          0.81857039, 0.81857039, 0.82766826, 0.82766826, 0.83681611,
          0.83681611, 0.84601411, 0.85526226, 0.85526226, 0.85526226,
          0.86456069, 0.86456069, 0.90225505, 0.90225505, 0.90225505,
          0.90225505, 0.91180396, 0.91180396, 0.91180396, 0.91180396,
          0.91180396, 0.91180396, 0.91180396, 0.91180396, 0.92140303,
          0.92140303, 0.92140303, 0.92140303, 0.92140303]))},
 'news_2cl_2': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-5.02525125e-05,  7.44193162e-02,  7.44193162e-02,  7.72375053e-02,
           2.21365253e-01,  2.31021733e-01,  2.31079186e-01,  2.40960729e-01,
           2.31194648e-01,  2.16876056e-01,  2.07623990e-01,  1.94069501e-01,
           1.85309247e-01,  1.81064094e-01,  1.68428194e-01,  1.64317858e-01,
           1.56249628e-01,  1.48384648e-01,  1.44528374e-01,  1.40722911e-01,
           1.26009137e-01,  1.15507261e-01,  1.08760018e-01,  8.97374128e-02,
           8.38028810e-02,  7.52819808e-02,  7.25432468e-02,  6.98552928e-02,
           6.72181171e-02,  5.47938477e-02,  4.79485023e-02,  3.37525391e-02,
           3.01532840e-02,  2.84297668e-02,  2.67569865e-02,  2.67569865e-02,
           8.97374128e-02,  8.67447527e-02,  8.67447527e-02,  8.38028810e-02,
           8.38028810e-02, -2.15978237e-03,  3.31174790e-04,  4.00234016e-03,
           1.65383933e-02,  1.02564104e-02,  5.35769961e-03,  6.18524516e-03,
           6.16292203e-03,  4.42043850e-03,  1.01585721e-03])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.01732625, 0.02678124, 0.03033189, 0.00586008, 0.00259409,
          0.00190443, 0.01357662, 0.00259409, 0.00887182, 0.00696638,
          0.00794401, 0.00794401, 0.00792573, 0.00602068, 0.00602068,
          0.00789067, 0.00691223, 0.00596724, 0.00596724, 0.00596724,
          0.00596724, 0.00598455, 0.00694783, 0.0122963 , 0.01111456,
          0.01113053, 0.00998357, 0.00998357, 0.00998357, 0.00996783,
          0.01359355, 0.0162087 , 0.01487575, 0.01359355, 0.01359355,
          0.01234492, 0.01357662, 0.01357662, 0.01357662, 0.01232822,
          0.01231201, 0.01113053, 0.01113053, 0.00999982, 0.00999982,
          0.00999982, 0.00999982, 0.00999982, 0.0088423 ])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-5.02525125e-05, -5.02525125e-05, -5.02525125e-05, -5.02525125e-05,
          -9.98783565e-05, -9.98783565e-05, -9.98783565e-05, -5.02525125e-05,
          -5.02525125e-05, -5.02525125e-05, -5.02525125e-05, -5.02525125e-05,
          -5.02525125e-05, -5.02525125e-05, -5.02525125e-05, -5.02525125e-05,
          -9.98783565e-05, -9.98783565e-05, -9.98783565e-05, -9.98783565e-05,
          -9.98783565e-05, -9.98783565e-05, -9.98783565e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  1.52156266e-04,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05, -5.02525125e-05, -5.02525125e-05,
          -9.92491444e-05, -9.92491444e-05,  1.01585721e-03, -5.02525125e-05,
          -5.02525125e-05, -5.02525125e-05])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.03033189, 0.01489291, 0.01619231, 0.0110841 , 0.01116398,
          0.00520305, 0.00518297, 0.00435787, 0.00358352, 0.00437822,
          0.00360414, 0.00220824, 0.00220824, 0.00154407, 0.00154407,
          0.67008643, 0.67008643, 0.68669397, 0.68669397, 0.67836471,
          0.67836471, 0.67836471, 0.67836471, 0.67836471, 0.67836471,
          0.67836471, 0.67008693, 0.67836495, 0.67836495, 0.67008659,
          0.67008659, 0.67008693, 0.67008693, 0.67008643, 0.67836471,
          0.68669365, 0.67008693, 0.67836462, 0.67836462, 0.67836462,
          0.68669349, 0.68669349, 0.68669349, 0.68669349, 0.67836462,
          0.67836471, 0.67836471, 0.66185916, 0.66185916])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-5.02525125e-05,  7.44193162e-02,  2.21378029e-01,  2.31094518e-01,
           2.31194648e-01,  1.98536849e-01,  1.81064094e-01,  1.60258336e-01,
           1.44528374e-01,  1.33264410e-01,  1.08760018e-01,  8.38028810e-02,
           8.38028810e-02,  7.52819808e-02,  6.46317179e-02,  2.12305205e-01,
           1.89719213e-01,  1.94157345e-01,  1.81064094e-01,  2.21715953e-01,
           1.72589345e-01,  1.72589345e-01,  1.40722911e-01,  6.72181171e-02,
           6.46317179e-02,  7.22799321e-03,  7.17938880e-03,  7.13278423e-03,
           7.11023206e-03,  6.18524516e-03,  6.18524516e-03,  6.16292203e-03,
           6.16292203e-03,  6.16292203e-03,  6.16292203e-03,  4.42043850e-03,
           4.42043850e-03,  4.42043850e-03,  4.42043850e-03,  4.42043850e-03,
           4.42043850e-03,  4.42043850e-03,  4.42043850e-03,  4.42043850e-03,
           4.42043850e-03,  4.42043850e-03,  4.42043850e-03,  4.42043850e-03,
           4.46466051e-03,  4.46466051e-03])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.02676944, 0.00875471, 0.00259409, 0.01619231, 0.00995258,
          0.00698542, 0.00694783, 0.00692978, 0.00790795, 0.00893687,
          0.00689518, 0.00689518, 0.00596724, 0.00792573, 0.00895439,
          0.01116398, 0.01001656, 0.01113053, 0.01232822, 0.01356019,
          0.01236212, 0.01359355, 0.01359355, 0.01234492, 0.01234492,
          0.01232822, 0.01231201, 0.01231201, 0.01231201, 0.00999982,
          0.00999982, 0.00999982, 0.00999982, 0.00999982, 0.00680342,
          0.00680342, 0.00779749, 0.00779749, 0.0088423 , 0.0088423 ,
          0.0088423 , 0.0088423 , 0.0088423 , 0.0088423 , 0.0088423 ,
          0.0088423 , 0.0088423 , 0.0088423 , 0.0088423 ])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-5.02525125e-05, -5.02525125e-05, -5.02525125e-05, -5.02525125e-05,
          -9.98783565e-05, -9.98783565e-05, -5.02525125e-05, -5.02525125e-05,
          -5.02525125e-05, -5.02525125e-05, -5.02525125e-05, -9.98783565e-05,
          -9.98783565e-05, -9.98783565e-05, -5.02525125e-05, -5.02525125e-05,
          -5.02525125e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
          -5.02525125e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05,  5.07601137e-05,  5.07601137e-05,
           5.07601137e-05,  5.07601137e-05])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([2.67694420e-02, 2.85380611e-02, 7.15201006e-04, 1.23449213e-02,
          1.75763052e-02, 1.01275057e-02, 6.14109958e-03, 4.39907988e-03,
          3.60414457e-03, 3.58352225e-03, 3.58352225e-03, 2.88081694e-03,
          2.88081694e-03, 2.88081694e-03, 1.60833095e-03, 1.60833095e-03,
          1.03751177e-03, 1.60833095e-03, 1.06020187e-03, 3.01669835e-03,
          1.18119836e-03, 1.18119836e-03, 1.25983035e-03, 6.86693494e-01,
          6.70086926e-01, 6.70086926e-01, 6.78364949e-01, 6.78364949e-01,
          6.78364949e-01, 6.78364949e-01, 6.86693652e-01, 6.95073032e-01,
          6.95073032e-01, 6.86693494e-01, 6.70086426e-01, 6.70086426e-01,
          6.70086426e-01, 6.78364705e-01, 6.61858818e-01, 6.78365355e-01,
          1.77631592e-03, 1.37075648e-03, 1.41880720e-03, 1.41880720e-03,
          3.04192199e-04, 5.07601137e-05, 5.07601137e-05, 5.07601137e-05,
          5.07601137e-05])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00000000e+00, -5.02525125e-05,  1.01585721e-03, -9.92491444e-05,
           1.52156266e-04,  1.52156266e-04,  3.38363488e-01,  5.13836177e-03,
           1.41880720e-03, -4.98740112e-05,  1.32320435e-03,  2.09300280e-03,
           1.54489110e-03,  4.51823436e-03,  3.25052956e-03,  4.51823436e-03,
           7.37729980e-01,  7.46417198e-01,  7.29102734e-01,  6.95074880e-01,
           7.29097262e-01,  7.29097262e-01,  7.37729980e-01,  6.86693494e-01,
           7.29097262e-01,  7.20515628e-01,  6.53682059e-01,  6.86693494e-01,
           6.86693494e-01,  6.95073032e-01,  6.95073032e-01,  6.95073032e-01,
           6.95073032e-01,  6.95073032e-01,  6.95073032e-01,  6.95073032e-01,
           6.95073032e-01,  6.86693494e-01,  6.78364705e-01,  7.03503087e-01,
           6.95072955e-01,  6.86693494e-01,  6.86693494e-01,  6.86693494e-01,
           6.86693494e-01,  6.86693494e-01,  6.86693494e-01,  6.95073032e-01,
           6.78364949e-01,  6.86695867e-01])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.67836462, 0.00141881, 0.67008643, 0.67008643,
          0.67008643, 0.67008643, 0.67008643, 0.69507365, 0.67008643,
          0.67008643, 0.67008643, 0.68669349, 0.67836462, 0.67836462,
          0.67008643, 0.67008643, 0.67836471, 0.67836471, 0.68669349,
          0.67836462, 0.67836462, 0.67836462, 0.67836462, 0.67836462,
          0.67836462, 0.67836462, 0.67836462, 0.67836462, 0.67836462,
          0.68669365, 0.68669365, 0.68669365, 0.68669365, 0.68669365,
          0.68669365, 0.70350309, 0.70350309, 0.70350309, 0.68669365,
          0.68669365, 0.68669349, 0.68669349, 0.68669444, 0.67836536,
          0.67836536, 0.67836536, 0.69507365, 0.68669508, 0.68669508])),
  'RSP K': (array([0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24,
          0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46,
          0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68,
          0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 ,
          0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([ 6.29456279e-01,  6.29456279e-01,  6.21482286e-01,  6.21482286e-01,
           6.21482286e-01,  6.21482286e-01,  6.21482286e-01,  6.21482286e-01,
           6.21482286e-01,  6.37482613e-01,  6.37482613e-01,  6.37482613e-01,
           6.37482613e-01,  6.29458150e-01,  6.29458150e-01,  6.37482613e-01,
           6.37482613e-01,  6.37482613e-01,  6.37482613e-01,  6.37482613e-01,
           6.29457121e-01,  6.29457121e-01,  6.21481522e-01,  6.21481522e-01,
           6.21481522e-01,  6.21481522e-01,  6.21481522e-01,  6.21481522e-01,
           6.21481522e-01,  6.21481522e-01,  6.37480050e-01,  6.37480050e-01,
           6.37480050e-01,  6.45555627e-01,  6.37480050e-01,  6.61858818e-01,
           6.53681884e-01,  6.61858903e-01,  6.61859160e-01,  6.61859160e-01,
           6.61859160e-01,  6.78364949e-01,  6.78364949e-01,  6.86693652e-01,
           6.86693494e-01,  7.03503985e-01,  7.11984466e-01,  7.03503087e-01,
          -5.02525125e-05])),
  'FE K': (array([0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24,
          0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46,
          0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68,
          0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 ,
          0.92, 0.94, 0.96, 0.98]),
   array([0.62945712, 0.62945712, 0.62945628, 0.62945628, 0.62148152,
          0.62148152, 0.62148152, 0.62148152, 0.62148152, 0.62148152,
          0.62148152, 0.62148152, 0.62148152, 0.62148152, 0.61355811,
          0.61355811, 0.61355811, 0.6455559 , 0.65368206, 0.6618589 ,
          0.67008659, 0.67008659, 0.67008659, 0.67836495, 0.67836495,
          0.67836495, 0.67836495, 0.67836495, 0.68669365, 0.68669365,
          0.69507303, 0.69507303, 0.69507303, 0.69507303, 0.69507303,
          0.69507303, 0.70350309, 0.70350324, 0.70350354, 0.71198447,
          0.72051605, 0.72051605, 0.70350324, 0.70350324, 0.70350324,
          0.7205152 , 0.72051535, 0.72051535])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-5.02525125e-05,  6.37480965e-01,  6.37481698e-01,  6.37481698e-01,
           6.37481698e-01,  6.37481698e-01,  6.37481698e-01,  6.37481698e-01,
           6.29456279e-01,  6.37481698e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01,  6.29456279e-01,
           6.29456279e-01,  6.29456279e-01,  6.29456279e-01])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96]),
   array([ 0.03233041,  0.03616397,  0.03815691,  0.0422951 ,  0.04662458,
           0.04887164,  0.05591746,  0.0608588 ,  0.06341075,  0.06601348,
           0.10777233,  0.0740594 ,  0.0979527 ,  0.11111818,  0.16357213,
           0.20711499,  0.21174829,  0.21643232,  0.22116707,  0.2164339 ,
           0.23078698,  0.17612286,  0.19353573,  0.64555697,  0.65368293,
           0.64555697,  0.66185959,  0.66185959,  0.66185959,  0.67008743,
           0.67836536,  0.67836536,  0.67836536,  0.67836536,  0.67836536,
           0.67008659,  0.67008659,  0.67008659,  0.67008659,  0.65368241,
           0.65368241,  0.67008659,  0.66185882,  0.01757631,  0.00778372,
           0.00195785, -0.00218037, -0.00218037])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.03234458, 0.03234458, 0.03234458, 0.03234458, 0.03234458,
          0.0402137 , 0.38674703, 0.43860665, 0.48662218, 0.55200257,
          0.56708794, 0.57470672, 0.58237621, 0.59786907, 0.63748261,
          0.64555777, 0.66185959, 0.67008743, 0.67008743, 0.67008743,
          0.67836536, 0.67836536, 0.67836536, 0.67836536, 0.66185959,
          0.66185959, 0.66185959, 0.66185959, 0.67008693, 0.67008693,
          0.67008743, 0.68669349, 0.68669349])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.03234458, 0.03234458, 0.03234458, 0.03234458, 0.03234458,
          0.0402137 , 0.38674703, 0.43860665, 0.48662218, 0.55200257,
          0.56708794, 0.57470672, 0.58237621, 0.59786907, 0.63748261,
          0.64555777, 0.66185959, 0.67008743, 0.67008743, 0.67008743,
          0.67836536, 0.67836536, 0.67836536, 0.67836536, 0.66185959,
          0.66185959, 0.66185959, 0.66185959, 0.67008693, 0.67008693,
          0.67008743, 0.68669349, 0.68669349])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.02871464, 0.03234458, 0.03234458, 0.03234458, 0.03234458,
          0.03234458, 0.03234458, 0.03234458, 0.0361776 , 0.03817027,
          0.04230791, 0.37429575, 0.39305584, 0.39940796, 0.44531285,
          0.47264858, 0.50079829, 0.56708794, 0.57470672, 0.57470672,
          0.57470672, 0.58237621, 0.59786907, 0.64555697, 0.63748261,
          0.64555777, 0.66185959, 0.67008743, 0.67008743, 0.67008743,
          0.67008743, 0.67836536, 0.67836536, 0.67836536, 0.67836536,
          0.67836495, 0.66185959, 0.65368293, 0.65368293, 0.65368293,
          0.65368293, 0.66185959, 0.67836471, 0.67836462]))},
 'news_2cl_3': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-2.50632848e-05,  4.26502887e-01,  4.26502887e-01,  4.26502887e-01,
           4.19957174e-01,  4.01332657e-02,  5.73324501e-02,  6.49450242e-02,
           7.30386621e-02,  6.77340773e-02,  7.31476412e-02,  7.59585289e-02,
           8.17038599e-02,  8.17038599e-02,  8.76515952e-02,  8.47286217e-02,
           7.61714289e-02,  7.90376378e-02,  7.07856104e-02,  7.08194712e-02,
           6.55714141e-02,  6.05254448e-02,  5.80782384e-02,  5.33353724e-02,
           5.10397087e-02,  5.10397087e-02,  4.87945553e-02,  4.65999100e-02,
           4.44557706e-02,  4.23621351e-02,  4.23621351e-02,  4.23621351e-02,
           4.23621351e-02,  4.23621351e-02,  4.23621351e-02,  4.44557706e-02,
           2.18236852e-01,  2.22958719e-01,  4.00756775e-01,  3.75700576e-01,
           3.28014947e-01,  3.39606156e-01,  3.05355279e-01,  3.05355279e-01,
           2.99823760e-01,  2.67696450e-01,  2.67696450e-01,  2.47271556e-01,
           2.42295920e-01,  2.42295920e-01,  2.42295920e-01])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.43305144, 0.45315214, 0.56801297, 0.61441098, 0.61441152,
          0.59874359, 0.59098572, 0.59098572, 0.5909864 , 0.5909864 ,
          0.5909864 , 0.61441317, 0.60655488, 0.60655488, 0.61441429,
          0.63028431, 0.6223241 , 0.6223241 , 0.6223241 , 0.59874841,
          0.54549928, 0.53074055, 0.52343688, 0.52343688, 0.52343899,
          0.50182429, 0.49472022, 0.48766843, 0.48066508, 0.45995753,
          0.42646437, 0.42646437, 0.40697318, 0.40697318, 0.36325232,
          0.16075064, 0.19034054, 0.16893818, 0.17734462, 0.1192232 ,
          0.11573572, 0.11921531, 0.12992107, 0.14492185, 0.14492185,
          0.1607268 , 0.20378618, 0.20378487, 0.21300305, 0.25698211])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-2.50632848e-05,  2.51892309e-05, -2.50632848e-05,  2.51892309e-05,
           2.51892309e-05, -2.50632848e-05, -2.50632848e-05,  2.51892309e-05,
           2.51892309e-05, -2.50632848e-05, -2.50632848e-05,  2.51892309e-05,
           2.51892309e-05, -2.50632848e-05, -2.50632848e-05, -2.50632848e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05, -2.50632848e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.76448574, 0.79108618, 0.80005303, 0.8090703 , 0.81813843,
          0.81813843, 0.81813843, 0.81813843, 0.82725648, 0.82725648,
          0.82725648, 0.83642534, 0.83642534, 0.83642534, 0.84564417,
          0.85491341, 0.83642441, 0.83642441, 0.83642441, 0.83642441,
          0.83642441, 0.83642441, 0.83642427, 0.83642427, 0.83642427,
          0.83642427, 0.83642427, 0.84564331, 0.85491277, 0.85491277,
          0.85491277, 0.85491277, 0.85491277, 0.85491277, 0.85491277,
          0.85491277, 0.85491277, 0.84564335, 0.84564335, 0.84564335,
          0.82725572, 0.82725572, 0.82725572, 0.82725572, 0.82725572,
          0.82725572, 0.82725572, 0.84564335, 0.84564399])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-2.50632848e-05,  4.26502887e-01,  3.81390364e-02,  7.04156754e-02,
           8.17038599e-02,  7.33879596e-02,  7.90705029e-02,  6.05254448e-02,
           4.65999100e-02,  4.44557706e-02,  4.23621351e-02,  3.44925863e-02,
           3.44925863e-02,  3.44925863e-02,  3.51453283e-01,  3.69562915e-01,
           2.99854686e-01,  3.22281768e-01,  2.99854686e-01,  3.05385610e-01,
           2.99854686e-01,  2.94342802e-01,  2.78203302e-01,  2.72924595e-01,
           2.57391851e-01,  2.57391851e-01,  2.47271556e-01,  2.47271556e-01,
           2.47271556e-01,  2.42295920e-01,  2.42295920e-01,  2.42295920e-01,
           2.42295920e-01,  2.42295920e-01,  2.42295920e-01,  2.42295920e-01,
           2.42295920e-01,  2.42295920e-01,  2.42295920e-01,  2.42295920e-01,
           2.42295920e-01,  2.42295920e-01,  2.42295920e-01,  2.42295920e-01,
           2.42295920e-01,  2.42295920e-01,  2.42295920e-01,  2.22899007e-01,
           2.22899007e-01,  2.22899007e-01])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.45315214, 0.6065519 , 0.59874359, 0.62231981, 0.63828971,
          0.60655488, 0.6223241 , 0.63028431, 0.63028431, 0.6223241 ,
          0.56046123, 0.56046123, 0.53074251, 0.52343899, 0.48766843,
          0.47371214, 0.43306239, 0.42646437, 0.36325232, 0.16073507,
          0.18162532, 0.12992734, 0.1227305 , 0.11920085, 0.12629715,
          0.14492185, 0.14492185, 0.14880336, 0.19476864, 0.19476864,
          0.19925151, 0.19925151, 0.20378487, 0.20836982, 0.21300523,
          0.22242452, 0.22720988, 0.23204573, 0.23204573, 0.23204573,
          0.23204573, 0.23693206, 0.25698211, 0.25698211, 0.25698211,
          0.25698211, 0.25698211, 0.25698211, 0.25698211])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-2.50632848e-05,  2.51892309e-05, -2.50632848e-05,  2.51892309e-05,
          -2.50632848e-05,  2.51892309e-05, -2.50632848e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05, -2.50632848e-05,  2.51892309e-05,
           2.26714357e-04, -2.50632848e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05,  2.51892309e-05,  2.51892309e-05,
           2.51892309e-05,  2.51892309e-05])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([6.06554334e-01, 7.82168612e-01, 7.91086183e-01, 7.91086183e-01,
          8.09070300e-01, 8.09070300e-01, 8.09070300e-01, 8.09070300e-01,
          8.09070300e-01, 8.18137995e-01, 8.27256480e-01, 8.27256480e-01,
          8.27256480e-01, 8.09069577e-01, 8.09069577e-01, 8.18137398e-01,
          8.18137398e-01, 8.27255629e-01, 8.18137237e-01, 8.36424227e-01,
          8.36424227e-01, 8.36424516e-01, 8.36424516e-01, 8.36424516e-01,
          8.36424516e-01, 8.36424516e-01, 8.36424516e-01, 8.36424516e-01,
          8.36424516e-01, 8.27255957e-01, 8.27255957e-01, 8.27255957e-01,
          8.27255957e-01, 8.27255957e-01, 8.27255957e-01, 8.27255957e-01,
          8.18138202e-01, 8.18138684e-01, 2.51892309e-05, 2.51892309e-05,
          2.51892309e-05, 2.51892309e-05, 2.51892309e-05, 2.51892309e-05,
          2.51892309e-05, 2.51892309e-05, 2.51892309e-05, 2.51892309e-05,
          2.51892309e-05])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.00000000e+00, 2.51892309e-05, 2.51892309e-05, 2.51892309e-05,
          2.51892309e-05, 2.51892309e-05, 7.91088030e-01, 4.03057696e-04,
          7.91086500e-01, 8.54912865e-01, 8.09072494e-01, 8.54912865e-01,
          8.73602829e-01, 8.27256698e-01, 8.36425136e-01, 8.36424516e-01,
          8.64232647e-01, 8.54912773e-01, 8.36424785e-01, 8.27255957e-01,
          8.18137513e-01, 8.27255957e-01, 8.18137513e-01, 8.18137513e-01,
          8.18137513e-01, 8.36424330e-01, 8.27255717e-01, 8.27255717e-01,
          8.27255717e-01, 8.27255717e-01, 8.27255717e-01, 8.27255717e-01,
          8.27255717e-01, 8.27255717e-01, 8.27255717e-01, 8.27255717e-01,
          8.45643350e-01, 8.45643350e-01, 8.45643350e-01, 8.45643350e-01,
          8.45643350e-01, 8.45643350e-01, 8.45643350e-01, 8.54912773e-01,
          8.54912773e-01, 8.54912773e-01, 8.54912755e-01, 8.54912938e-01,
          8.73603148e-01, 8.09072904e-01])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.83642433, 0.83642433, 0.83642433, 0.83642433,
          0.83642433, 0.83642433, 0.83642433, 0.83642433, 0.83642433,
          0.83642433, 0.83642433, 0.83642433, 0.83642433, 0.83642433,
          0.83642433, 0.83642433, 0.83642433, 0.83642433, 0.83642433,
          0.83642433, 0.83642433, 0.83642433, 0.83642433, 0.83642433,
          0.83642433, 0.83642433, 0.83642433, 0.83642433, 0.83642433,
          0.83642433, 0.83642433, 0.83642433, 0.83642433, 0.83642433,
          0.83642433, 0.83642433, 0.83642433, 0.83642433, 0.83642433,
          0.83642433, 0.83642433, 0.83642433, 0.83642433, 0.83642433,
          0.83642433, 0.83642433, 0.83642433, 0.83642433, 0.83642433])),
  'RSP K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.8924944 , 0.8924944 , 0.8924944 , 0.8924944 , 0.8924944 ,
          0.8924944 , 0.8924944 , 0.8924944 , 0.8924944 , 0.8924944 ,
          0.8924944 , 0.8924944 , 0.8924944 , 0.8924944 , 0.8924944 ,
          0.8924944 , 0.90201573, 0.90201573, 0.90201573, 0.90201573,
          0.90201573, 0.90201573, 0.90201573, 0.90201573, 0.90201573,
          0.90201573, 0.90201573, 0.8924944 , 0.8924944 , 0.88302341,
          0.88302341, 0.88302341, 0.88302341, 0.88302341, 0.87360281,
          0.86423261, 0.86423261, 0.87360286, 0.88302342, 0.88302342,
          0.89249442, 0.90201581, 0.89249452, 0.89249452, 0.86423308,
          0.88302363, 0.87360297])),
  'FE K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.90201573, 0.90201573, 0.90201573, 0.90201573, 0.90201573,
          0.90201573, 0.90201573, 0.90201573, 0.90201573, 0.90201573,
          0.90201573, 0.90201573, 0.90201573, 0.90201573, 0.90201573,
          0.90201573, 0.89249438, 0.89249438, 0.90201574, 0.90201574,
          0.90201574, 0.90201574, 0.90201574, 0.90201574, 0.90201574,
          0.90201574, 0.89249452, 0.88302363, 0.88302363, 0.88302363,
          0.88302363, 0.88302363, 0.88302363, 0.89249452, 0.88302363,
          0.88302363, 0.88302363, 0.88302363, 0.88302363, 0.88302363,
          0.88302363, 0.88302363, 0.87360286, 0.8830235 , 0.87360297,
          0.87360297, 0.87360286])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([2.51892309e-05, 9.02015732e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01, 8.92494395e-01,
          8.92494395e-01, 8.92494395e-01, 8.92494395e-01])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96]),
   array([0.59875271, 0.71264719, 0.72116119, 0.76448645, 0.77330147,
          0.78216765, 0.78216765, 0.78216765, 0.79108444, 0.80906924,
          0.80005172, 0.80005172, 0.80005172, 0.80005172, 0.81813717,
          0.81813717, 0.81813717, 0.81813717, 0.81813717, 0.81813717,
          0.81813717, 0.81813717, 0.81813717, 0.80906922, 0.80906922,
          0.80906922, 0.80906922, 0.81813717, 0.81813717, 0.81813717,
          0.81813717, 0.81813717, 0.83642421, 0.83642421, 0.83642423,
          0.85491286, 0.84564349, 0.84564349, 0.85491277, 0.85491277,
          0.85491277, 0.84564384, 0.85491341, 0.81814029, 0.75572655,
          0.07798075, 0.03696246, 0.03696246])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.77330221, 0.80005235, 0.80906972, 0.80005202, 0.80005202,
          0.81813731, 0.81813781, 0.82725628, 0.82725628, 0.82725628,
          0.82725628, 0.82725628, 0.82725628, 0.82725628, 0.82725628,
          0.82725628, 0.82725628, 0.81813781, 0.81813781, 0.81813781,
          0.82725596, 0.82725596, 0.81813781, 0.82725596, 0.82725596,
          0.82725596, 0.83642452, 0.83642452, 0.83642452, 0.83642433,
          0.83642433, 0.83642433, 0.82725572])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.77330221, 0.80005235, 0.80906972, 0.80005202, 0.80005202,
          0.81813731, 0.81813781, 0.82725628, 0.82725628, 0.82725628,
          0.82725628, 0.82725628, 0.82725628, 0.82725628, 0.82725628,
          0.82725628, 0.82725628, 0.81813781, 0.81813781, 0.81813781,
          0.82725596, 0.82725596, 0.81813781, 0.82725596, 0.82725596,
          0.82725596, 0.83642452, 0.83642452, 0.83642452, 0.83642433,
          0.83642433, 0.83642433, 0.82725572])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.64636216, 0.77330221, 0.79108539, 0.80005235, 0.80005235,
          0.79108539, 0.80005235, 0.80906946, 0.80005202, 0.81813731,
          0.81813731, 0.81813781, 0.82725628, 0.82725628, 0.82725628,
          0.82725628, 0.82725628, 0.82725628, 0.82725628, 0.82725628,
          0.82725628, 0.82725628, 0.82725628, 0.82725628, 0.82725628,
          0.82725628, 0.82725628, 0.82725628, 0.82725628, 0.81813781,
          0.82725596, 0.82725596, 0.81813781, 0.82725596, 0.82725596,
          0.82725596, 0.82725596, 0.83642452, 0.83642452, 0.84564349,
          0.84564349, 0.83642433, 0.83642433, 0.83642433]))},
 'news_3cl_1': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([1.11670389e-05, 1.03991081e-01, 1.00849577e-01, 1.00840762e-01,
          1.04717780e-01, 1.03302087e-01, 1.09087826e-01, 1.15157980e-01,
          1.22205457e-01, 1.47614288e-01, 1.61064863e-01, 1.54678108e-01,
          1.58961085e-01, 1.65482164e-01, 1.63932724e-01, 1.54065640e-01,
          1.58700097e-01, 1.58815435e-01, 1.52178849e-01, 1.54786949e-01,
          1.58655601e-01, 1.57314251e-01, 1.54988980e-01, 1.80941172e-01,
          1.81566235e-01, 1.84253288e-01, 1.96966716e-01, 1.96966716e-01,
          2.05290383e-01, 2.12354697e-01, 2.23211189e-01, 2.26899502e-01,
          2.30622612e-01, 1.43432017e-01, 1.53268200e-01, 1.51497061e-01,
          2.65116578e-01, 2.83618622e-01, 2.81170363e-01, 2.82039470e-01,
          2.76972846e-01, 2.53482697e-01, 1.66575413e-01, 1.61128802e-01,
          1.78657618e-01, 1.26884909e-01, 1.50380865e-01, 1.42240440e-01,
          1.14826556e-01, 1.25743727e-01, 1.17943834e-01])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.18188687, 0.20392994, 0.22871163, 0.25845046, 0.2725007 ,
          0.27946748, 0.30668752, 0.33488595, 0.34284187, 0.33421899,
          0.3474812 , 0.34542053, 0.3468075 , 0.34772643, 0.35583583,
          0.35583583, 0.35290086, 0.33178021, 0.32963381, 0.32963381,
          0.32963381, 0.32567111, 0.32830314, 0.30972736, 0.33412216,
          0.33448553, 0.31617558, 0.32788353, 0.320338  , 0.31917138,
          0.13142231, 0.16137969, 0.11596717, 0.10855288, 0.10599748,
          0.10856911, 0.10070927, 0.09808469, 0.1012089 , 0.1008678 ,
          0.09902737, 0.09190401, 0.09190401, 0.08882293, 0.08752502,
          0.08190455, 0.08806065, 0.09060592, 0.09570839, 0.10620519])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 1.11670389e-05, -5.55560185e-06,  1.11670389e-05,  1.11670389e-05,
          -5.55560185e-06,  1.79570437e-04, -5.55560185e-06, -5.55560185e-06,
          -5.55560185e-06, -5.55560185e-06,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05, -5.55560185e-06,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.12416725e-05,
           1.12416725e-05,  1.12416725e-05,  1.12416725e-05, -5.55560185e-06,
          -5.55560185e-06,  1.11670389e-05,  1.11670389e-05, -5.55560185e-06,
          -5.55560185e-06, -5.55560185e-06,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05, -5.55560185e-06])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.27556537, 0.3619366 , 0.38602656, 0.3944862 , 0.44663719,
          0.45462471, 0.50133417, 0.49968846, 0.73393058, 0.7424444 ,
          0.75127322, 0.76404456, 0.76837859, 0.77699538, 0.81257113,
          0.82199106, 0.79832328, 0.82199106, 0.82199106, 0.82672132,
          0.82672132, 0.82647803, 0.82647803, 0.82672132, 0.82699117,
          0.83628136, 0.83144176, 0.83144176, 0.82630799, 0.83108455,
          0.84987667, 0.83623592, 0.84086497, 0.84082287, 0.84538587,
          0.84541371, 0.84082287, 0.84082287, 0.83171992, 0.82656137,
          0.82647562, 0.83182969, 0.82699265, 0.82253117, 0.81305552,
          0.81305552, 0.81819243, 0.79849918, 0.47319791])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([1.11670389e-05, 1.00597333e-01, 1.15157980e-01, 1.57548624e-01,
          1.65455366e-01, 5.02144927e-02, 1.58587164e-01, 1.76206640e-01,
          2.34723810e-01, 2.80822500e-01, 2.51309527e-01, 2.43577601e-01,
          2.35987230e-01, 2.47425846e-01, 2.52380044e-01, 1.65205041e-01,
          1.68984873e-01, 1.70407318e-01, 1.49231131e-01, 1.42299002e-01,
          1.46212063e-01, 1.37695839e-01, 1.33345391e-01, 1.22654761e-01,
          1.22654761e-01, 1.21461700e-01, 1.24072075e-01, 1.11794714e-01,
          1.22410635e-01, 1.22410635e-01, 1.22410635e-01, 1.22410635e-01,
          1.17943834e-01, 1.17943834e-01, 1.17943834e-01, 1.17943834e-01,
          1.17943834e-01, 1.17943834e-01, 1.17943834e-01, 1.17943834e-01,
          1.17943834e-01, 1.17943834e-01, 1.17943834e-01, 1.17943834e-01,
          1.17943834e-01, 1.17943834e-01, 1.28409914e-01, 1.28409914e-01,
          1.28409914e-01, 1.28409914e-01])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.23310123, 0.26768119, 0.33601238, 0.34542053, 0.3468075 ,
          0.35583583, 0.33639632, 0.34616468, 0.34613305, 0.3248426 ,
          0.33994862, 0.33317188, 0.25969176, 0.22368543, 0.15808179,
          0.11945192, 0.11225313, 0.10897282, 0.09662828, 0.09565111,
          0.08792188, 0.08868448, 0.08460489, 0.08905621, 0.09023695,
          0.07964884, 0.08166653, 0.08465669, 0.08528985, 0.09063309,
          0.09305275, 0.09663715, 0.09663715, 0.09955963, 0.10034892,
          0.10174237, 0.10209373, 0.10050826, 0.10692092, 0.10692092,
          0.10620519, 0.10620519, 0.10620519, 0.10620519, 0.10620519,
          0.10620519, 0.10620519, 0.10620519, 0.10620519])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 1.11670389e-05, -5.55560185e-06,  1.11670389e-05, -5.55560185e-06,
          -5.55560185e-06, -5.55560185e-06,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  3.04203947e-04,  1.11670389e-05, -5.55560185e-06,
          -5.55560185e-06,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
          -4.43444745e-06,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           5.05040510e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05,  1.11670389e-05,  1.11670389e-05,  1.11670389e-05,
           1.11670389e-05, -2.22596301e-05, -2.22596301e-05, -2.22596301e-05,
          -2.22596301e-05, -2.22596301e-05])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 2.24618757e-01,  2.77639978e-01,  3.47454583e-01,  3.80597204e-01,
           3.86026562e-01,  3.89087939e-01,  3.92594291e-01,  3.94084499e-01,
           4.40388160e-01,  4.59059690e-01,  4.97206131e-01,  5.19184725e-01,
           5.26276847e-01,  7.60500922e-01,  7.64653761e-01,  7.60336462e-01,
           7.77707673e-01,  7.95053221e-01,  8.27014138e-01,  8.41236957e-01,
           8.17801655e-01,  8.22421954e-01,  8.17694736e-01,  8.27143032e-01,
           8.17969444e-01,  8.13379559e-01,  8.13459450e-01,  8.13459450e-01,
           8.18155727e-01,  8.22902157e-01,  8.18377832e-01,  4.56032898e-01,
           4.51965030e-01,  4.54219673e-01,  4.49733245e-01,  4.48353107e-01,
           1.96872991e-04,  1.96872991e-04,  1.96872991e-04,  1.96872991e-04,
           1.96872991e-04,  1.96872991e-04,  1.96872991e-04,  1.12416725e-05,
          -2.22596301e-05, -2.22596301e-05, -2.22596301e-05, -2.22596301e-05,
          -2.22596301e-05])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00000000e+00,  1.11670389e-05, -5.55560185e-06,  2.25210046e-05,
           4.63075913e-01,  1.12416725e-05,  4.60173048e-01,  7.08112070e-04,
           8.96358393e-05,  6.15523378e-04,  3.91568238e-01,  4.23642682e-01,
           7.99714249e-01,  4.41390148e-01,  5.90400191e-02,  5.90400191e-02,
           8.00593535e-01,  8.13959074e-01,  4.83817011e-01,  8.41297178e-01,
           8.41255348e-01,  4.51599819e-01,  7.89393490e-01,  7.98602138e-01,
           8.27359589e-01,  8.13620770e-01,  8.07792795e-01,  8.07792795e-01,
           8.28042447e-01,  8.27286525e-01,  8.08393590e-01,  7.94734926e-01,
           8.03855074e-01,  8.03855074e-01,  8.22685458e-01,  8.08475366e-01,
           8.12579241e-01,  8.17719797e-01,  8.21860552e-01,  8.17719797e-01,
           8.22127886e-01,  8.22127886e-01,  8.26762252e-01,  8.31207290e-01,
           8.45315712e-01,  8.26220998e-01,  8.26220998e-01,  8.21652493e-01,
           8.06608389e-01,  7.56502956e-01])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.46602819, 0.44060513, 0.4406352 , 0.4406352 ,
          0.81787684, 0.81787684, 0.82257533, 0.82257533, 0.82257533,
          0.82257533, 0.82257533, 0.83163382, 0.83163382, 0.83163382,
          0.83163382, 0.83163382, 0.83163382, 0.83163382, 0.83163382,
          0.83163382, 0.83163382, 0.83163382, 0.83170435, 0.83164238,
          0.83164238, 0.83626952, 0.83626952, 0.83626952, 0.83626952,
          0.82710343, 0.83167312, 0.82698957, 0.83179896, 0.82698957,
          0.83167928, 0.83169485, 0.83167928, 0.83167928, 0.83167928,
          0.83177581, 0.83186722, 0.83653353, 0.83202773, 0.84571961,
          0.85047643, 0.8458904 , 0.83678756, 0.8368135 , 0.8368135 ])),
  'RSP K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.80131495, 0.80131495, 0.80131495, 0.80131495, 0.80131495,
          0.80131495, 0.80131495, 0.80131495, 0.80131495, 0.80131495,
          0.80131495, 0.80142691, 0.80142691, 0.80142691, 0.80142691,
          0.80142691, 0.80142691, 0.80142691, 0.80142691, 0.76562726,
          0.76530669, 0.76530669, 0.76530669, 0.76530669, 0.76530669,
          0.76530669, 0.76530669, 0.76530669, 0.76551694, 0.76129891,
          0.76129891, 0.76129891, 0.76206162, 0.75783044, 0.75755563,
          0.75311897, 0.75313639, 0.75704452, 0.75285544, 0.75286106,
          0.76127418, 0.77876011, 0.78306985, 0.78270938, 0.81803967,
          0.83189422, 0.83653353])),
  'FE K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.80131495, 0.80569359, 0.80569359, 0.80569359, 0.80569359,
          0.80122972, 0.80122972, 0.79657689, 0.79632151, 0.79632151,
          0.79197232, 0.79197232, 0.78758613, 0.78758613, 0.78758613,
          0.79192158, 0.79192158, 0.79192158, 0.79192158, 0.79192158,
          0.79189766, 0.79189766, 0.8006746 , 0.8006746 , 0.80534277,
          0.80534277, 0.8006746 , 0.8097819 , 0.8097819 , 0.80486066,
          0.80932625, 0.80932625, 0.80470013, 0.81367242, 0.81815691,
          0.83197757, 0.83656855, 0.84111465, 0.84568675, 0.84568675,
          0.85034357, 0.845741  , 0.84116481, 0.83658728, 0.83194438,
          0.83653353, 0.83164047])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([1.11670389e-05, 8.10435315e-01, 8.05693593e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01, 8.01314953e-01,
          8.01314953e-01, 8.01314953e-01, 8.01314953e-01])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96]),
   array([0.07411989, 0.13749513, 0.19895079, 0.21145742, 0.21471406,
          0.25945106, 0.27614952, 0.28585942, 0.29966248, 0.30569633,
          0.30750878, 0.33579551, 0.35796884, 0.42407465, 0.43250919,
          0.45057812, 0.47233796, 0.74763987, 0.73853671, 0.70436264,
          0.71706854, 0.72551791, 0.78691992, 0.75154974, 0.7823271 ,
          0.80101404, 0.80101404, 0.80101404, 0.81362387, 0.79186271,
          0.79170298, 0.79605249, 0.80889355, 0.80889355, 0.80874225,
          0.81336548, 0.81336548, 0.81336548, 0.83165938, 0.83166213,
          0.83622876, 0.83137839, 0.82714495, 0.36001898, 0.32074494,
          0.13159203, 0.04166531, 0.04058096])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.12343503, 0.31555155, 0.34585969, 0.67082209, 0.67937778,
          0.68723472, 0.69590967, 0.70902905, 0.7221126 , 0.7394191 ,
          0.74323852, 0.75209808, 0.76510105, 0.76510105, 0.76953677,
          0.77850649, 0.80067473, 0.79668567, 0.7963603 , 0.80087364,
          0.80951518, 0.80457537, 0.8090555 , 0.80457537, 0.8228535 ,
          0.81373825, 0.81814901, 0.82262546, 0.82713995, 0.82713995,
          0.83169627, 0.83169627, 0.81284818])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.12343503, 0.31555155, 0.34585969, 0.67082209, 0.67937778,
          0.68723472, 0.69590967, 0.70902905, 0.7221126 , 0.7394191 ,
          0.74323852, 0.75209808, 0.76510105, 0.76510105, 0.76953677,
          0.77850649, 0.80067473, 0.79668567, 0.7963603 , 0.80087364,
          0.80951518, 0.80457537, 0.8090555 , 0.80457537, 0.8228535 ,
          0.81373825, 0.81814901, 0.82262546, 0.82713995, 0.82713995,
          0.83169627, 0.83169627, 0.81284818])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.07589138, 0.12343503, 0.24591883, 0.31555155, 0.35854943,
          0.36523714, 0.40580245, 0.67479075, 0.67937778, 0.68363412,
          0.68723472, 0.69232011, 0.70043573, 0.70065099, 0.7178541 ,
          0.72185096, 0.7394191 , 0.74323852, 0.74772998, 0.75209808,
          0.76093656, 0.76490277, 0.76953677, 0.76551429, 0.77850649,
          0.77876832, 0.78318632, 0.7963603 , 0.79195657, 0.79601956,
          0.80034385, 0.80469435, 0.80012172, 0.80012172, 0.80469435,
          0.80469435, 0.80448477, 0.80906947, 0.82272395, 0.82262546,
          0.82262546, 0.82262546, 0.8317175 , 0.8317175 ]))},
 'news_3cl_2': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([5.62856901e-05, 1.99234040e-01, 2.06469696e-01, 2.12519060e-01,
          2.07336197e-01, 1.99967358e-01, 1.82097976e-01, 1.56746138e-01,
          1.27224113e-01, 1.17198769e-01, 1.19327621e-01, 7.46024785e-02,
          6.67026985e-02, 6.06362944e-02, 5.57695776e-02, 4.80266472e-02,
          3.95402880e-02, 3.94309217e-02, 3.76202473e-02, 3.63686189e-02,
          3.65192042e-02, 3.44384130e-02, 3.25014132e-02, 3.08714799e-02,
          2.70807529e-02, 2.75778069e-02, 2.75778069e-02, 2.58567882e-02,
          2.58567882e-02, 2.58567882e-02, 2.44040258e-02, 2.34474740e-01,
          2.38267662e-01, 2.31173871e-01, 2.38720161e-01, 2.39201106e-01,
          2.74880312e-01, 2.81185862e-01, 2.70862364e-01, 2.72306703e-01,
          2.31859015e-01, 1.16990262e-01, 2.04362719e-01, 1.47488771e-01,
          1.66915576e-01, 1.43914609e-01, 8.61197692e-02, 7.88206506e-02,
          7.89528349e-02, 7.82053091e-02, 7.82053091e-02])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.21096134, 0.23986486, 0.32574001, 0.32266866, 0.31065615,
          0.3241124 , 0.35310394, 0.35864831, 0.36375321, 0.36375321,
          0.37025615, 0.36847673, 0.37042611, 0.37163999, 0.36513917,
          0.36513917, 0.3779922 , 0.382678  , 0.38139401, 0.37626593,
          0.3736887 , 0.37210262, 0.36870542, 0.37097358, 0.36761797,
          0.36761797, 0.36761797, 0.37374329, 0.37402192, 0.3644228 ,
          0.36110868, 0.35240402, 0.35476108, 0.05727926, 0.05579878,
          0.05411308, 0.05437005, 0.05454619, 0.05454619, 0.05771356,
          0.05261948, 0.05374231, 0.05413048, 0.0528412 , 0.05687848,
          0.05851687, 0.05938758, 0.05902015, 0.05283403, 0.05530666])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 5.62856901e-05,  5.62856901e-05, -1.10545548e-05,  5.78050642e-06,
           5.78050642e-06,  5.62856901e-05, -2.78896160e-05,  5.78050642e-06,
           5.78050642e-06,  5.78050642e-06,  5.78050642e-06,  5.78050642e-06,
           5.78050642e-06,  5.78050642e-06,  5.78050642e-06,  5.78050642e-06,
           9.14899914e-03,  5.78050642e-06,  5.78050642e-06,  5.78050642e-06,
           5.78050642e-06,  5.78050642e-06,  5.78050642e-06,  5.78050642e-06,
           5.78050642e-06,  5.91649416e-05,  5.91649416e-05,  5.91649416e-05,
           9.09172108e-05, -2.78896160e-05, -2.78896160e-05,  5.78050642e-06,
           1.12722777e-04,  5.78050642e-06,  2.15600994e-04, -1.10545548e-05,
           5.78050642e-06, -1.10545548e-05,  5.78050642e-06, -1.10545548e-05,
          -1.10545548e-05,  5.78050642e-06, -1.10545548e-05,  5.78050642e-06,
           5.78050642e-06,  5.78050642e-06,  5.78050642e-06,  5.78050642e-06,
           5.78050642e-06,  5.78050642e-06])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.32833164, 0.37513786, 0.39153989, 0.4044586 , 0.4092951 ,
          0.43877898, 0.43368308, 0.45072236, 0.46518766, 0.48125481,
          0.47051306, 0.54781853, 0.55652394, 0.70032844, 0.70453535,
          0.70911226, 0.71335974, 0.73041869, 0.75713929, 0.75633872,
          0.77433687, 0.78263783, 0.78263783, 0.78263783, 0.77844736,
          0.77428298, 0.77428298, 0.77419581, 0.78357783, 0.78351861,
          0.78806932, 0.78806932, 0.7880296 , 0.78357783, 0.7880296 ,
          0.78357783, 0.78777179, 0.7880296 , 0.80077018, 0.80049433,
          0.79655656, 0.80927657, 0.80447074, 0.79627783, 0.80044048,
          0.81426407, 0.81836996, 0.81838227, 0.67688549])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([5.62856901e-05, 2.22409466e-01, 1.67498159e-01, 1.19327621e-01,
          5.90635727e-02, 3.94309217e-02, 3.29990258e-02, 2.62034796e-02,
          2.32303643e-01, 2.36072571e-01, 2.47589679e-01, 2.44676486e-01,
          2.39024607e-01, 1.64885160e-01, 1.90602325e-01, 1.83966683e-01,
          1.71108509e-01, 1.71108509e-01, 1.80700575e-01, 1.71108509e-01,
          1.64557990e-01, 1.62917037e-01, 1.50790576e-01, 1.25486615e-01,
          1.16485561e-01, 1.12574967e-01, 1.05353458e-01, 9.44704924e-02,
          8.14846415e-02, 7.92368655e-02, 7.85989857e-02, 7.93363696e-02,
          7.77940083e-02, 7.82053091e-02, 7.82053091e-02, 7.82053091e-02,
          7.82053091e-02, 7.82053091e-02, 7.82053091e-02, 7.82053091e-02,
          7.82053091e-02, 7.82053091e-02, 7.82053091e-02, 7.82053091e-02,
          7.82053091e-02, 7.82053091e-02, 7.82053091e-02, 7.57967368e-02,
          7.57967368e-02, 7.57967368e-02])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.31549691, 0.32821177, 0.36368273, 0.36375321, 0.37025615,
          0.37283446, 0.38185142, 0.37946557, 0.38365596, 0.37097358,
          0.37097358, 0.37575357, 0.37367724, 0.375543  , 0.37126127,
          0.36807393, 0.36491824, 0.06211333, 0.06019845, 0.06038906,
          0.06038906, 0.04971318, 0.0524276 , 0.0516385 , 0.05153565,
          0.05268195, 0.05700682, 0.05815768, 0.05835617, 0.05887187,
          0.06063594, 0.05327376, 0.05383242, 0.05446479, 0.05539932,
          0.05782335, 0.05703881, 0.05687081, 0.05687081, 0.05594013,
          0.05530666, 0.05530666, 0.05530666, 0.05530666, 0.05530666,
          0.05530666, 0.05530666, 0.05530666, 0.05530666])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 5.62856901e-05,  5.62856901e-05, -1.10545548e-05,  5.78050642e-06,
          -1.10545548e-05,  5.78050642e-06,  5.78050642e-06,  6.88053180e-03,
           5.78050642e-06,  5.78050642e-06,  5.78050642e-06,  5.78050642e-06,
           5.62856901e-05,  5.62856901e-05,  5.78050642e-06,  5.62856901e-05,
           5.62856901e-05,  5.62856901e-05,  5.78050642e-06, -2.78896160e-05,
          -2.78896160e-05, -1.10545548e-05, -2.78896160e-05,  5.78050642e-06,
           5.78050642e-06, -2.78896160e-05, -2.78896160e-05,  4.01573984e-05,
           4.01573984e-05,  5.67754173e-05, -2.78896160e-05, -2.78896160e-05,
           1.15426537e-05,  1.15426537e-05,  1.15426537e-05,  1.15426537e-05,
           1.15426537e-05,  1.15426537e-05,  1.15426537e-05,  1.15426537e-05,
           1.15426537e-05,  1.15426537e-05,  1.15426537e-05,  1.15426537e-05,
           1.15426537e-05,  1.15426537e-05,  1.15426537e-05, -1.10545548e-05,
          -1.10545548e-05, -1.10545548e-05])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 3.18099096e-01,  3.25253430e-01,  3.56767022e-01,  3.66249283e-01,
           3.80022224e-01,  3.98610528e-01,  4.09295098e-01,  3.99550925e-01,
           3.99550925e-01,  4.13958319e-01,  4.18746421e-01,  4.26825487e-01,
           4.32073790e-01,  4.57308032e-01,  4.47488964e-01,  4.47575044e-01,
           4.49276703e-01,  4.49736320e-01,  4.54792756e-01,  4.53973318e-01,
           4.57706724e-01,  7.59812649e-01,  7.59609864e-01,  7.63993145e-01,
           7.81304787e-01,  7.77171028e-01,  7.81600273e-01,  4.78434839e-01,
           4.78434839e-01,  4.79421124e-01,  4.77817204e-01,  4.76262143e-01,
           4.80376502e-01,  4.73186838e-01,  4.73465710e-01,  4.75846160e-01,
           4.73199248e-01,  5.67754173e-05,  5.67754173e-05,  5.67754173e-05,
           5.67754173e-05,  5.67754173e-05,  5.67754173e-05,  5.67754173e-05,
           5.67754173e-05,  1.15426537e-05, -1.10545548e-05, -1.10545548e-05,
          -1.10545548e-05])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00000000e+00,  5.78050642e-06, -5.32070026e-06,  1.15426537e-05,
           5.67754173e-05,  3.98837711e-05,  3.98837711e-05,  1.64756313e-04,
           5.67754173e-05,  5.67754173e-05,  5.41151529e-03,  3.91985115e-01,
           3.64869020e-01,  4.48287387e-01,  4.75324608e-01,  4.73857218e-01,
           4.70049825e-01,  4.62614755e-01,  8.06048125e-01,  7.61111116e-01,
           7.51542571e-01,  8.01820083e-01,  7.92305298e-01,  8.15185193e-01,
           7.95539466e-01,  7.26782381e-01,  8.08641254e-01,  7.52064639e-01,
           8.00988886e-01,  8.09153125e-01,  8.06055298e-01,  7.99506179e-01,
           8.24067668e-01,  8.10082647e-01,  8.10082647e-01,  7.91318424e-01,
           8.01010778e-01,  7.94980694e-01,  7.99506179e-01,  8.00173178e-01,
           8.08358727e-01,  8.00494335e-01,  7.91276796e-01,  7.99432213e-01,
           8.04012604e-01,  7.94819383e-01,  7.85661402e-01,  7.76905515e-01,
           7.77786033e-01,  7.12884288e-01])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.48008016, 0.48008016, 0.81919893, 0.81919893,
          0.81447084, 0.8233846 , 0.81468393, 0.8233846 , 0.8233846 ,
          0.8233846 , 0.8233846 , 0.8233846 , 0.8233846 , 0.8233846 ,
          0.8233846 , 0.8233846 , 0.8233846 , 0.8233846 , 0.8233846 ,
          0.8233846 , 0.8233846 , 0.8233846 , 0.82793512, 0.82328661,
          0.82328661, 0.82328661, 0.82328661, 0.82328661, 0.82328661,
          0.81866538, 0.82787565, 0.82787565, 0.82787565, 0.82787565,
          0.82787565, 0.83251266, 0.83251266, 0.83717678, 0.83280573,
          0.83300632, 0.83300632, 0.82437216, 0.81984779, 0.82914092,
          0.82462022, 0.81148942, 0.80745785, 0.80700639, 0.8024261 ])),
  'RSP K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.69527466, 0.69527466, 0.69527466, 0.69527466, 0.69090913,
          0.69090913, 0.69090913, 0.69090913, 0.69090913, 0.69090913,
          0.69854   , 0.69148833, 0.68713381, 0.68713381, 0.69460509,
          0.69460509, 0.69460509, 0.69460509, 0.67581855, 0.67581855,
          0.67581855, 0.67522566, 0.67522566, 0.6795988 , 0.6795988 ,
          0.67580113, 0.6795988 , 0.67949469, 0.67949469, 0.67949469,
          0.67949469, 0.67519981, 0.68331484, 0.68387871, 0.66673046,
          0.67104562, 0.66272252, 0.65850882, 0.68321354, 0.68321354,
          0.69953361, 0.71313972, 0.71617741, 0.73628244, 0.76561889,
          0.80063527, 0.81888715])),
  'FE K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.69908775, 0.69908775, 0.70304382, 0.70304382, 0.70304382,
          0.70773857, 0.70773857, 0.70773857, 0.70773857, 0.70773857,
          0.70773857, 0.70773857, 0.71224522, 0.7208631 , 0.7208631 ,
          0.72888284, 0.73352927, 0.73352927, 0.73352927, 0.74564274,
          0.75404683, 0.75802497, 0.76285347, 0.76695558, 0.76695558,
          0.76695558, 0.77116023, 0.7758293 , 0.7758293 , 0.77588986,
          0.78404392, 0.79726494, 0.79254811, 0.80618051, 0.81050016,
          0.81019875, 0.81462957, 0.81462957, 0.81903363, 0.82350319,
          0.81450017, 0.8238597 , 0.81484499, 0.82408845, 0.8286142 ,
          0.82845834, 0.81476766])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([5.78050642e-06, 7.07199323e-01, 6.99208063e-01, 6.95274656e-01,
          6.95274656e-01, 6.90798939e-01, 6.95160587e-01, 6.95160587e-01,
          6.95160587e-01, 6.95160587e-01, 6.86915703e-01, 6.86915703e-01,
          6.95274656e-01, 6.95274656e-01, 6.95274656e-01, 6.83061603e-01,
          6.86915703e-01, 6.95274656e-01, 6.95274656e-01, 6.95274656e-01,
          6.95274656e-01, 6.95274656e-01, 6.95274656e-01, 6.95274656e-01,
          6.95274656e-01, 6.91262409e-01, 6.91262409e-01, 6.83061603e-01,
          6.83061603e-01, 6.83061603e-01, 6.83061603e-01, 6.83061603e-01,
          6.83061603e-01, 6.83061603e-01, 6.83061603e-01, 6.83061603e-01,
          6.83061603e-01, 6.83061603e-01, 6.83061603e-01, 6.83061603e-01,
          6.83061603e-01, 6.83061603e-01, 6.83061603e-01, 6.83061603e-01,
          6.83061603e-01, 6.83061603e-01, 6.83061603e-01, 6.83061603e-01,
          6.83061603e-01, 6.83061603e-01, 6.83061603e-01])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96]),
   array([0.20633389, 0.21365427, 0.22017242, 0.22655853, 0.2271392 ,
          0.24801486, 0.31191845, 0.31737139, 0.31391341, 0.33814511,
          0.36621425, 0.3857201 , 0.37064947, 0.39134956, 0.37830669,
          0.42530108, 0.46788301, 0.43917672, 0.56240139, 0.56240139,
          0.55774834, 0.62369693, 0.61137972, 0.61072571, 0.63563275,
          0.65019497, 0.66205077, 0.66205077, 0.67759207, 0.69977666,
          0.71279841, 0.70913376, 0.72134339, 0.7170323 , 0.74846803,
          0.75254835, 0.76495158, 0.76960339, 0.76635506, 0.76474774,
          0.7743185 , 0.72541151, 0.48188687, 0.41168434, 0.35976371,
          0.03024609, 0.02883355, 0.02901997])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.22351155, 0.24579281, 0.26966175, 0.28373987, 0.44731709,
          0.48926724, 0.49011044, 0.49389614, 0.57924318, 0.60647414,
          0.60647414, 0.63763922, 0.64857233, 0.64857233, 0.66120031,
          0.66494115, 0.66876084, 0.69370082, 0.70155041, 0.70558604,
          0.73050137, 0.75226336, 0.75622778, 0.76450605, 0.77333814,
          0.77333814, 0.7824652 , 0.80640321, 0.80156948, 0.81090236,
          0.81517776, 0.82812084, 0.82352861])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.22351155, 0.24579281, 0.26966175, 0.28373987, 0.44731709,
          0.48926724, 0.49011044, 0.49389614, 0.57924318, 0.60647414,
          0.60647414, 0.63763922, 0.64857233, 0.64857233, 0.66120031,
          0.66494115, 0.66876084, 0.69370082, 0.70155041, 0.70558604,
          0.73050137, 0.75226336, 0.75622778, 0.76450605, 0.77333814,
          0.77333814, 0.7824652 , 0.80640321, 0.80156948, 0.81090236,
          0.81517776, 0.82812084, 0.82352861])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.21263636, 0.21910635, 0.23012841, 0.24579281, 0.26373489,
          0.26983878, 0.27357402, 0.27357402, 0.45220784, 0.46043656,
          0.48926724, 0.49011044, 0.49011044, 0.49814991, 0.59184197,
          0.60303826, 0.60280133, 0.60993863, 0.60993863, 0.64125509,
          0.64125509, 0.64857233, 0.64902469, 0.66094689, 0.65722239,
          0.66096351, 0.66858987, 0.70165136, 0.70155041, 0.71411158,
          0.70954876, 0.70964904, 0.74666945, 0.76035351, 0.76450605,
          0.77333814, 0.7777956 , 0.79148777, 0.80047226, 0.80977217,
          0.81465542, 0.81436033, 0.81868338, 0.81407156]))},
 'news_3cl_3': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-2.80571979e-05,  1.40486405e-01,  8.88360070e-02,  1.57939568e-01,
           1.68902790e-01,  1.28091875e-01,  1.31365040e-01,  1.36875554e-01,
           1.83668642e-02,  1.83282928e-02,  1.90500764e-02,  1.89941380e-02,
           1.63868251e-02,  5.66947542e-03,  5.73375264e-03,  5.35597575e-03,
           5.35597575e-03,  1.45728935e-02,  1.39563003e-02,  1.33675373e-02,
           1.30236915e-02,  1.27237221e-02,  1.31594717e-02,  1.21790271e-02,
           1.43094933e-02,  1.56089554e-02,  1.63962211e-02,  1.92839334e-01,
           1.83798650e-01,  1.78087619e-01,  1.73374139e-01,  1.74005252e-01,
           1.71152161e-01,  3.81605905e-01,  5.50108340e-02,  5.33169202e-02,
           5.50108340e-02,  5.16535830e-02,  5.00207249e-02,  5.00207249e-02,
           5.15233176e-02,  5.48882077e-02,  5.66165759e-02,  1.39486880e-01,
           1.03193870e-01,  9.12557631e-02,  9.01144725e-02,  7.44562077e-02,
           8.33533581e-02,  8.29017174e-02,  7.81512732e-02])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.24350167, 0.29518667, 0.30780525, 0.32978628, 0.37428323,
          0.39130352, 0.38786458, 0.40695515, 0.41054578, 0.41260533,
          0.4124055 , 0.42374502, 0.4219384 , 0.41863519, 0.41725302,
          0.41863519, 0.43709585, 0.43575105, 0.43638834, 0.45626311,
          0.43894481, 0.45371108, 0.45244942, 0.45244942, 0.45244942,
          0.45244942, 0.4534076 , 0.4653231 , 0.45680219, 0.45806199,
          0.05942801, 0.05802079, 0.05796471, 0.05631252, 0.05664347,
          0.05167995, 0.05109397, 0.05215052, 0.05169523, 0.05823347,
          0.05364718, 0.05477724, 0.05567172, 0.05717184, 0.06102935,
          0.06143601, 0.06427363, 0.06254564, 0.07372275, 0.07494742])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-2.80571979e-05, -2.80571979e-05,  5.73269512e-04,  5.69712482e-05,
           9.17068001e-03,  2.29598697e-05, -1.10515087e-05, -2.80571979e-05,
          -1.10515087e-05, -1.10515087e-05,  4.23588000e-04,  1.04092449e-02,
           9.43070360e-04,  2.29598697e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05, -1.10515087e-05,  5.69712482e-05, -1.10515087e-05,
          -1.10515087e-05,  5.69712482e-05,  2.29598697e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05, -1.10515087e-05,  5.69712482e-05,
           2.29598697e-05,  1.94087847e-04,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  1.14039135e-04,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.32931869, 0.39932913, 0.42334721, 0.43111403, 0.43611134,
          0.44825278, 0.45417181, 0.45944005, 0.46512944, 0.47037383,
          0.46840893, 0.4721523 , 0.47234396, 0.4744527 , 0.4799588 ,
          0.4799588 , 0.4799588 , 0.48085496, 0.467327  , 0.46911773,
          0.46911773, 0.46911773, 0.46911773, 0.47237796, 0.47419875,
          0.47237796, 0.47718798, 0.47718798, 0.47718798, 0.47775413,
          0.47718798, 0.47775413, 0.47775413, 0.48335571, 0.4787286 ,
          0.4787286 , 0.47596033, 0.48032452, 0.4786312 , 0.48662691,
          0.48645463, 0.49216003, 0.48645463, 0.48535057, 0.48652774,
          0.49565837, 0.48652774, 0.4683049 , 0.48561354])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-2.80571979e-05,  1.96299590e-01,  1.31420567e-01,  1.99313684e-02,
           5.66947542e-03,  5.35597575e-03,  1.20629994e-02,  2.02735804e-01,
           2.05558891e-01,  2.06695335e-01,  2.77600697e-02,  1.91983955e-01,
           1.85175212e-01,  1.67539621e-01,  1.89115178e-01,  1.05040434e-01,
           1.45888166e-01,  1.49935649e-01,  1.29640920e-01,  1.29640920e-01,
           1.25450873e-01,  1.19087304e-01,  1.11180648e-01,  9.69398327e-02,
           7.70035159e-02,  7.59288488e-02,  7.56995375e-02,  7.39482481e-02,
           8.07636000e-02,  7.96743563e-02,  8.01793847e-02,  8.22478903e-02,
           7.61342108e-02,  7.53535272e-02,  8.01202591e-02,  8.20399069e-02,
           8.29017174e-02,  8.09047446e-02,  8.09047446e-02,  8.09047446e-02,
           8.09047446e-02,  8.09047446e-02,  7.81512732e-02,  7.81512732e-02,
           7.81512732e-02,  7.81512732e-02,  7.81512732e-02,  8.01202591e-02,
           8.01202591e-02,  8.01202591e-02])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.30260748, 0.37575403, 0.39597504, 0.41042595, 0.4124055 ,
          0.43805976, 0.43805976, 0.43901836, 0.44592875, 0.44160147,
          0.44492843, 0.44466945, 0.4520855 , 0.46449532, 0.46348812,
          0.46195572, 0.05796471, 0.05820458, 0.05480131, 0.05596108,
          0.05842198, 0.0522352 , 0.05696753, 0.05657223, 0.05653283,
          0.05791146, 0.06093186, 0.06219688, 0.0648149 , 0.06564598,
          0.0713708 , 0.07288937, 0.07240223, 0.07387027, 0.0747521 ,
          0.07524113, 0.07331949, 0.07331949, 0.07524113, 0.07494742,
          0.07494742, 0.07494742, 0.07494742, 0.07494742, 0.07494742,
          0.07494742, 0.07494742, 0.07494742, 0.07494742])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-2.80571979e-05,  2.29598697e-05, -1.10515087e-05,  2.29598697e-05,
           2.62424530e-03, -2.80571979e-05, -1.10515087e-05, -1.10515087e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  2.29598697e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           2.29598697e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           4.23588000e-04,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05,  5.69712482e-05,  5.69712482e-05,
           5.69712482e-05,  5.69712482e-05])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([3.02607477e-01, 3.34244800e-01, 4.07733445e-01, 4.10878136e-01,
          4.21830610e-01, 4.23347206e-01, 4.32685991e-01, 4.35318587e-01,
          4.57737002e-01, 4.60951348e-01, 4.61924172e-01, 4.61924172e-01,
          4.61924172e-01, 4.57525755e-01, 4.58144128e-01, 4.59815332e-01,
          4.62312684e-01, 4.61618999e-01, 4.61618999e-01, 4.63416681e-01,
          4.63306418e-01, 4.68845687e-01, 4.72205914e-01, 4.77684205e-01,
          4.77684205e-01, 4.79673034e-01, 4.79673034e-01, 4.79762500e-01,
          4.82285471e-01, 4.91676147e-01, 4.91676147e-01, 4.86902281e-01,
          4.76028760e-01, 4.68228520e-01, 4.60379737e-01, 3.40854812e-03,
          3.81286854e-03, 3.81286854e-03, 1.39854132e-03, 3.07578842e-03,
          2.34318509e-03, 8.28191916e-04, 1.94087847e-04, 1.94087847e-04,
          1.94087847e-04, 1.94087847e-04, 1.94087847e-04, 1.14039135e-04,
          5.69712482e-05])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.00000000e+00, 5.69712482e-05, 1.94087847e-04, 1.14039135e-04,
          1.14039135e-04, 2.74406968e-04, 2.40895308e-04, 6.54638585e-04,
          8.28191916e-04, 6.54638585e-04, 4.67842110e-01, 4.72534266e-01,
          4.73817227e-01, 4.99569897e-01, 4.46512558e-01, 1.38699680e-03,
          1.20010262e-03, 4.28432144e-01, 4.31242321e-01, 1.81894699e-01,
          4.73429657e-01, 4.77997015e-01, 4.73429657e-01, 4.76634427e-01,
          4.85204022e-01, 8.26831119e-01, 5.04640710e-01, 7.37368117e-01,
          4.96625021e-01, 4.98861491e-01, 4.96028443e-01, 7.36199007e-01,
          4.97838246e-01, 7.51153616e-01, 7.67062873e-01, 8.13687500e-01,
          4.79978683e-01, 4.91053554e-01, 4.92234833e-01, 4.92234833e-01,
          4.84830047e-01, 4.86626914e-01, 4.92335668e-01, 4.86626914e-01,
          4.80445000e-01, 4.80657742e-01, 4.79385230e-01, 4.86192152e-01,
          7.43966575e-01, 7.24787509e-01])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.00082819, 0.00082819, 0.00099082, 0.48344579,
          0.47857378, 0.4781982 , 0.46509114, 0.68580889, 0.69682305,
          0.48839819, 0.70805608, 0.7113362 , 0.70766183, 0.71540673,
          0.7113362 , 0.7113362 , 0.72656812, 0.73423814, 0.72244378,
          0.7622495 , 0.7622495 , 0.7622495 , 0.7622495 , 0.7622495 ,
          0.7622495 , 0.77892934, 0.79136172, 0.79136172, 0.79592435,
          0.79592435, 0.79592435, 0.79592435, 0.79592435, 0.80886986,
          0.80449134, 0.80449134, 0.79592614, 0.80002441, 0.78707319,
          0.79135506, 0.78698303, 0.79560892, 0.79114275, 0.80000404,
          0.78243625, 0.78670433, 0.78225139, 0.77374624, 0.75755861])),
  'RSP K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98, 1.  ]),
   array([7.44835875e-01, 7.44835875e-01, 7.44835875e-01, 7.53945698e-01,
          7.49932899e-01, 7.49932899e-01, 7.40847977e-01, 7.40847977e-01,
          7.44835875e-01, 7.44835875e-01, 7.44482508e-01, 7.48852764e-01,
          7.44482508e-01, 7.44482508e-01, 7.44482508e-01, 7.45007535e-01,
          7.45007535e-01, 7.20629930e-01, 7.25142605e-01, 7.32935626e-01,
          7.20629930e-01, 7.29116861e-01, 7.29116861e-01, 7.29116861e-01,
          7.24591323e-01, 7.24591323e-01, 7.24591323e-01, 7.29116861e-01,
          7.33117036e-01, 7.41002695e-01, 7.44989596e-01, 7.44989596e-01,
          7.49248397e-01, 7.49248397e-01, 7.61915970e-01, 7.57340596e-01,
          7.61409653e-01, 7.65309743e-01, 7.74061608e-01, 7.91186790e-01,
          7.91413681e-01, 7.91413681e-01, 7.78057143e-01, 7.82504324e-01,
          7.94885507e-01, 8.08784239e-01, 7.95201635e-01, 1.82694021e-04])),
  'FE K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.74885276, 0.74885276, 0.75295329, 0.75295329, 0.75295329,
          0.75707996, 0.76117249, 0.76117249, 0.76117249, 0.76117249,
          0.76944411, 0.77362302, 0.77342799, 0.76949347, 0.79064845,
          0.7949649 , 0.79979368, 0.79979368, 0.79980517, 0.80428699,
          0.80415795, 0.80415795, 0.80415795, 0.80415795, 0.79490345,
          0.79490345, 0.79056034, 0.79056034, 0.79091385, 0.78651012,
          0.78651012, 0.78651012, 0.78651012, 0.78221943, 0.78221943,
          0.80395517, 0.80395517, 0.81293304, 0.80849681, 0.80411249,
          0.80860673, 0.80402127, 0.80402127, 0.7952503 , 0.79996527,
          0.81783659, 0.81354163])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([5.69712482e-05, 7.38259638e-01, 7.49932899e-01, 7.49932899e-01,
          7.49932899e-01, 7.45949179e-01, 7.49932899e-01, 7.49932899e-01,
          7.49932899e-01, 7.49932899e-01, 7.45879179e-01, 7.45879179e-01,
          7.49868655e-01, 7.49932899e-01, 7.49932899e-01, 7.49932899e-01,
          7.49932899e-01, 7.49932899e-01, 7.49932899e-01, 7.49932899e-01,
          7.49932899e-01, 7.49932899e-01, 7.49932899e-01, 7.49932899e-01,
          7.49932899e-01, 7.49868655e-01, 7.49868655e-01, 7.49868655e-01,
          7.49868655e-01, 7.49868655e-01, 7.49868655e-01, 7.49868655e-01,
          7.49868655e-01, 7.49868655e-01, 7.49868655e-01, 7.49868655e-01,
          7.49868655e-01, 7.49868655e-01, 7.49868655e-01, 7.49868655e-01,
          7.49868655e-01, 7.49868655e-01, 7.49868655e-01, 7.49868655e-01,
          7.49868655e-01, 7.49868655e-01, 7.49868655e-01, 7.49868655e-01,
          7.49868655e-01, 7.49868655e-01, 7.49868655e-01])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94]),
   array([0.17994928, 0.22270892, 0.24440854, 0.2853938 , 0.2874383 ,
          0.31090173, 0.31379446, 0.32090158, 0.33173923, 0.33889333,
          0.33889333, 0.3491754 , 0.35580903, 0.36598227, 0.37516094,
          0.37447029, 0.38802322, 0.398774  , 0.41295273, 0.41506963,
          0.43270267, 0.43381951, 0.44387576, 0.4624604 , 0.47261767,
          0.46860339, 0.48640384, 0.48871747, 0.51436957, 0.51833575,
          0.52417102, 0.52172027, 0.5526188 , 0.54695247, 0.55704168,
          0.59019284, 0.60227258, 0.61324477, 0.61294417, 0.52151038,
          0.52630853, 0.48275449, 0.47728023, 0.47461752, 0.46109132,
          0.40713592, 0.04769832])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.21873076, 0.28959238, 0.32418509, 0.35751741, 0.37743811,
          0.40342161, 0.40882183, 0.42901737, 0.45935209, 0.46406028,
          0.47080869, 0.47726856, 0.6713362 , 0.68846469, 0.67269156,
          0.67288193, 0.74219551, 0.75513541, 0.75959136, 0.75581601,
          0.76807043, 0.7682114 , 0.75581181, 0.79008527, 0.79469604,
          0.79933426, 0.79933426, 0.7949898 , 0.79100855, 0.80846306,
          0.79973536, 0.79535836, 0.76206189])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.21873076, 0.28959238, 0.32418509, 0.35751741, 0.37743811,
          0.40342161, 0.40882183, 0.42901737, 0.45935209, 0.46406028,
          0.47080869, 0.47726856, 0.6713362 , 0.68846469, 0.67269156,
          0.67288193, 0.74219551, 0.75513541, 0.75959136, 0.75581601,
          0.76807043, 0.7682114 , 0.75581181, 0.79008527, 0.79469604,
          0.79933426, 0.79933426, 0.7949898 , 0.79100855, 0.80846306,
          0.79973536, 0.79535836, 0.76206189])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.12793286, 0.22238105, 0.2488869 , 0.29400806, 0.31922496,
          0.33791225, 0.35318856, 0.36887183, 0.37289406, 0.37101586,
          0.40342161, 0.40513928, 0.41229608, 0.43222187, 0.43724958,
          0.45358524, 0.46148042, 0.4651598 , 0.45861406, 0.47726856,
          0.4810256 , 0.4810256 , 0.68840428, 0.68429886, 0.70888462,
          0.66522823, 0.68143524, 0.74273539, 0.7510945 , 0.74277033,
          0.73907845, 0.74695691, 0.77282161, 0.77282161, 0.76426124,
          0.79066689, 0.7949898 , 0.79066689, 0.80810349, 0.81722877,
          0.80838282, 0.81279251, 0.80846306, 0.79535836]))},
 'news_5cl_1': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-1.40522628e-05,  1.95772121e-01,  1.97335464e-01,  2.00110274e-01,
           2.12176319e-01,  2.13440367e-01,  2.08847051e-01,  2.04666990e-01,
           2.05383506e-01,  1.99014124e-01,  2.25720061e-01,  2.43059180e-01,
           2.50700669e-01,  2.48615441e-01,  2.49778054e-01,  2.48508048e-01,
           1.48707665e-01,  1.45931293e-01,  1.45423726e-01,  1.39472489e-01,
           1.36192257e-01,  1.31980968e-01,  1.44201978e-01,  1.44584410e-01,
           1.38723627e-01,  1.38891716e-01,  1.36617119e-01,  1.32973903e-01,
           9.09438088e-02,  8.71839361e-02,  8.76206903e-02,  8.58935937e-02,
           8.38808201e-02,  8.23447764e-02,  2.00695871e-01,  2.05540838e-01,
           2.07318924e-01,  2.10120448e-01,  2.08476494e-01,  1.37051534e-01,
           1.74175398e-01,  1.27301769e-01,  1.28298432e-01,  1.67248415e-01,
           1.63029687e-01,  1.18557006e-01,  1.09961226e-01,  7.01966412e-02,
           7.34694501e-02,  7.56545121e-02,  1.99633591e-01])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.13750056, 0.1393898 , 0.15149802, 0.15670227, 0.14490256,
          0.14765527, 0.15114533, 0.19987512, 0.19874984, 0.20487394,
          0.20253853, 0.20561729, 0.2120254 , 0.21666393, 0.2184202 ,
          0.22405407, 0.22235895, 0.21434499, 0.21550947, 0.22367165,
          0.21814154, 0.16971367, 0.16352286, 0.16191187, 0.19539523,
          0.18685256, 0.17991891, 0.16848797, 0.15611478, 0.15672829,
          0.15588282, 0.15194842, 0.15301554, 0.15132158, 0.15317077,
          0.15296009, 0.07038934, 0.06676969, 0.06645578, 0.06550438,
          0.0618877 , 0.0622008 , 0.06115826, 0.06123064, 0.0604123 ,
          0.05901541, 0.06048575, 0.06257785, 0.06328951])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-1.40522628e-05,  1.62504281e-05,  3.14017735e-05,  1.62504281e-05,
           1.11999796e-05,  1.62504281e-05,  1.11999796e-05,  1.09908263e-06,
           1.11999796e-05,  1.09908263e-06,  1.09908263e-06,  1.62504281e-05,
           3.47220496e-05,  1.09908263e-06,  1.28680844e-03,  1.11999796e-05,
           1.09908263e-06,  1.11999796e-05,  1.09908263e-06,  1.09908263e-06,
           1.11999796e-05, -9.00181434e-06,  1.09908263e-06,  1.13108403e-05,
           2.44042986e-05,  2.02704185e-01,  4.17050944e-05,  4.20753000e-05,
           2.44042986e-05, -3.95136585e-06,  1.11999796e-05,  2.44042986e-05,
           3.14017735e-05,  1.09908263e-06,  4.49853265e-05,  2.12404470e-04,
           3.45204407e-05,  1.42881564e-05,  4.78001039e-03,  7.15793448e-05,
           5.69022215e-05,  3.45204407e-05,  1.11999796e-05,  1.11999796e-05,
           1.42881564e-05,  1.42820702e-04,  1.42881564e-05,  2.44042986e-05,
           2.44042986e-05,  7.80552997e-05])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.14681615, 0.22665669, 0.2443151 , 0.24904299, 0.25515519,
          0.44326226, 0.40313277, 0.45271438, 0.45362605, 0.46125804,
          0.46459288, 0.48039048, 0.4827556 , 0.48183551, 0.48102254,
          0.48102254, 0.484694  , 0.48486317, 0.48417125, 0.48998762,
          0.48690616, 0.49542801, 0.48340295, 0.49962822, 0.50023291,
          0.49723889, 0.48003395, 0.51145063, 0.49126261, 0.50899169,
          0.51307871, 0.50141369, 0.51474252, 0.51724559, 0.51124562,
          0.51304479, 0.51704071, 0.51288998, 0.33302513, 0.51587412,
          0.33442477, 0.51813234, 0.49372739, 0.50005925, 0.35151964,
          0.50670926, 0.11295708, 0.10709802, 0.07780839])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-1.40522628e-05,  2.14588536e-01,  2.21537305e-01,  2.44434440e-01,
           1.53670866e-01,  1.07192101e-01,  1.58766427e-01,  1.45729934e-01,
           1.14031835e-01,  2.26662075e-01,  2.12307887e-01,  1.97030227e-01,
           1.58906013e-01,  1.63703399e-01,  1.68587742e-01,  1.58719338e-01,
           1.39169589e-01,  1.29731668e-01,  1.19805285e-01,  1.03649389e-01,
           1.08534700e-01,  9.61202378e-02,  7.50456887e-02,  7.53292919e-02,
           7.53647193e-02,  7.49930545e-02,  7.48465033e-02,  7.48465033e-02,
           7.48465033e-02,  7.48465033e-02,  7.48465033e-02,  7.48465033e-02,
           7.48465033e-02,  7.48523850e-02,  7.48523850e-02,  7.48523850e-02,
           7.48523850e-02,  7.48523850e-02,  7.48523850e-02,  7.48523850e-02,
           7.48523850e-02,  7.48523850e-02,  7.48523850e-02,  7.48523850e-02,
           7.48523850e-02,  7.70749331e-02,  7.70749331e-02,  7.70749331e-02,
           7.70749331e-02,  6.66101847e-02])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.15221278, 0.16681203, 0.20487133, 0.21983541, 0.2261203 ,
          0.23036373, 0.2077243 , 0.17030676, 0.16630416, 0.18798289,
          0.15908607, 0.15013914, 0.15495162, 0.1536554 , 0.07230362,
          0.06311493, 0.06134633, 0.06125886, 0.0604288 , 0.06162153,
          0.06246426, 0.06098068, 0.06171001, 0.0632664 , 0.06306362,
          0.06442964, 0.06649336, 0.06523964, 0.06758844, 0.06766492,
          0.06780965, 0.06780965, 0.06780965, 0.06780965, 0.06780965,
          0.06780965, 0.06780965, 0.06780965, 0.06780965, 0.06780965,
          0.06780965, 0.06780965, 0.06780965, 0.06780965, 0.06780965,
          0.06780965, 0.06780965, 0.06780965, 0.06780965])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-1.40522628e-05, -9.00181434e-06,  1.11999796e-05,  1.09908263e-06,
           1.11999796e-05,  1.62504281e-05,  1.09908263e-06,  1.11999796e-05,
           1.11999796e-05,  1.11999796e-05,  3.14017735e-05,  1.09908263e-06,
           1.09908263e-06, -3.95136585e-06,  1.09908263e-06,  1.11999796e-05,
           1.09908263e-06,  9.23008530e-06,  2.44042986e-05,  7.77015384e-05,
           7.24148679e-05,  3.66393854e-05,  2.44042986e-05,  2.44042986e-05,
           2.14422584e-05,  4.17050944e-05,  4.17050944e-05,  4.17050944e-05,
           4.17050944e-05,  4.17050944e-05,  4.17050944e-05,  4.17050944e-05,
           4.17050944e-05,  4.17050944e-05,  4.17050944e-05,  4.17050944e-05,
           4.17050944e-05,  4.17050944e-05,  4.17050944e-05,  4.17050944e-05,
           4.17050944e-05,  2.44042986e-05,  2.44042986e-05,  2.44042986e-05,
           2.44042986e-05,  2.44042986e-05,  2.44042986e-05,  2.76073375e-05,
           2.76073375e-05,  6.24513133e-06])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([1.55483618e-01, 1.54959729e-01, 2.15519483e-01, 2.27075369e-01,
          2.44531580e-01, 2.51489009e-01, 2.54619116e-01, 2.63615982e-01,
          3.92136589e-01, 2.82272306e-01, 2.86945468e-01, 2.90425636e-01,
          2.86876925e-01, 2.97292150e-01, 2.98345992e-01, 3.05575807e-01,
          3.12234085e-01, 3.11662430e-01, 3.15321941e-01, 3.20835464e-01,
          3.17840527e-01, 3.09340943e-01, 3.05264093e-01, 3.06222050e-01,
          3.05191592e-01, 2.96229688e-01, 3.04521418e-01, 2.98659486e-01,
          2.99609668e-01, 2.94744134e-01, 1.05721452e-01, 9.34533160e-02,
          9.34644635e-02, 7.34856834e-02, 6.04540985e-02, 5.29741490e-05,
          1.97282143e-03, 1.30596297e-04, 1.30596297e-04, 1.30596297e-04,
          1.30596297e-04, 9.27389608e-05, 9.27389608e-05, 9.27389608e-05,
          9.27389608e-05, 2.76073375e-05, 2.76073375e-05, 2.76073375e-05,
          6.24513133e-06])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.00000000e+00, 3.15736764e-05, 1.63765494e-05, 1.01964544e-01,
          4.64976305e-02, 1.31397580e-04, 1.31397580e-04, 1.55633040e-04,
          1.16396308e-01, 4.27275469e-04, 9.91289589e-05, 2.35316908e-04,
          4.26262636e-01, 7.46745542e-02, 8.06374533e-02, 2.16625476e-01,
          2.36181050e-01, 2.76582279e-01, 2.49345148e-01, 2.58851625e-01,
          1.34643437e-01, 3.09136668e-01, 3.91788567e-01, 3.15469197e-01,
          4.19858195e-01, 3.00820198e-01, 2.70556945e-01, 4.22915837e-01,
          5.12652408e-01, 2.98058977e-01, 2.64102446e-01, 4.95128488e-01,
          4.35913703e-01, 4.85114762e-01, 4.82498820e-01, 5.23990031e-01,
          4.80035493e-01, 3.13916428e-01, 3.12247634e-01, 4.93378778e-01,
          4.78762996e-01, 4.95204545e-01, 3.20695639e-01, 5.12306410e-01,
          5.00293131e-01, 5.14765276e-01, 5.02264635e-01, 5.01003202e-01,
          4.84676272e-01, 4.79449721e-01])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.00000000e+00, 8.17369730e-05, 2.65472287e-01, 3.03270275e-01,
          2.98532514e-01, 2.99359017e-01, 3.03270275e-01, 2.98532514e-01,
          4.76613401e-01, 2.98532514e-01, 2.96233460e-01, 4.78077903e-01,
          2.99949902e-01, 2.99928585e-01, 4.68558328e-01, 4.65527192e-01,
          4.70847927e-01, 4.69236716e-01, 4.67134861e-01, 4.71540111e-01,
          4.71540111e-01, 4.72085988e-01, 4.72085988e-01, 4.64398353e-01,
          4.72085988e-01, 5.50446098e-01, 4.66792478e-01, 4.66051745e-01,
          4.71545760e-01, 4.95015090e-01, 5.51446747e-01, 5.58614145e-01,
          5.60605451e-01, 5.63096198e-01, 5.56450024e-01, 5.54253081e-01,
          5.62191998e-01, 5.76772363e-01, 5.82734688e-01, 5.82828102e-01,
          5.87036924e-01, 5.87749349e-01, 5.97586112e-01, 6.02807093e-01,
          6.23612020e-01, 6.52511252e-01, 6.71225604e-01, 6.83110015e-01,
          6.96296608e-01, 7.05990132e-01])),
  'RSP K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98, 1.  ]),
   array([7.01739907e-01, 6.99803716e-01, 7.00031051e-01, 7.01739907e-01,
          7.01708313e-01, 6.95362861e-01, 7.02043379e-01, 7.04052087e-01,
          7.04052087e-01, 7.01984402e-01, 6.99743887e-01, 7.01862449e-01,
          6.97686005e-01, 6.99578433e-01, 7.01639302e-01, 7.01808303e-01,
          6.97813496e-01, 6.99835314e-01, 6.97621971e-01, 7.01869248e-01,
          6.97761465e-01, 6.99770257e-01, 6.99770257e-01, 6.97598551e-01,
          6.95641660e-01, 6.97577374e-01, 6.95397034e-01, 6.93186814e-01,
          6.93349461e-01, 6.97260514e-01, 6.97397680e-01, 6.95473794e-01,
          7.05694180e-01, 7.03693084e-01, 7.03829835e-01, 6.99439783e-01,
          7.01416883e-01, 7.03385539e-01, 7.07273649e-01, 7.11510883e-01,
          7.20031265e-01, 7.28336770e-01, 7.32364581e-01, 7.12917513e-01,
          7.31831907e-01, 7.33941292e-01, 7.34749264e-01, 4.17050944e-05])),
  'FE K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.70818319, 0.71034251, 0.71038301, 0.71034251, 0.70783035,
          0.70783035, 0.70783035, 0.70998651, 0.71213172, 0.71210305,
          0.71383425, 0.71215238, 0.71004304, 0.71826404, 0.70787502,
          0.71393314, 0.71283697, 0.72589405, 0.7146709 , 0.71879284,
          0.72762681, 0.72520976, 0.7318725 , 0.73417498, 0.74281775,
          0.73826004, 0.73628963, 0.73635029, 0.73635029, 0.73634247,
          0.74285236, 0.7449391 , 0.74278585, 0.74264673, 0.72700664,
          0.73143172, 0.73584525, 0.73113084, 0.73803397, 0.73337233,
          0.72551581, 0.72786354, 0.72969293, 0.72468428, 0.7136412 ,
          0.70369106, 0.68616834])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([3.15736764e-05, 7.06059139e-01, 7.03967326e-01, 7.01708313e-01,
          6.99730419e-01, 6.99736463e-01, 6.99736463e-01, 7.06017041e-01,
          6.97624865e-01, 7.06138355e-01, 6.97624865e-01, 6.97624865e-01,
          6.97624865e-01, 7.01938252e-01, 7.01779595e-01, 7.01779595e-01,
          7.01739907e-01, 7.01739907e-01, 7.01739907e-01, 7.01739907e-01,
          7.01739907e-01, 7.01739907e-01, 7.01739907e-01, 7.01739907e-01,
          7.01739907e-01, 7.01739907e-01, 7.01739907e-01, 7.01739907e-01,
          7.01739907e-01, 7.01739907e-01, 7.01739907e-01, 7.01739907e-01,
          7.01739907e-01, 7.01739907e-01, 7.01739907e-01, 7.01739907e-01,
          7.01739907e-01, 7.01739907e-01, 7.01739907e-01, 7.01739907e-01,
          7.01739907e-01, 7.01739907e-01, 7.01739907e-01, 7.01739907e-01,
          7.01739907e-01, 7.01739907e-01, 7.01739907e-01, 7.01739907e-01,
          7.01739907e-01, 7.01739907e-01, 7.01739907e-01])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94]),
   array([0.10517457, 0.11217865, 0.11937096, 0.13576974, 0.15994975,
          0.15923467, 0.17190691, 0.20309947, 0.21338397, 0.21104809,
          0.2213947 , 0.21627501, 0.24141419, 0.2410403 , 0.28659921,
          0.31000295, 0.29262391, 0.30954481, 0.31820467, 0.32477879,
          0.31183735, 0.32403189, 0.33742567, 0.3917432 , 0.40657187,
          0.40197284, 0.4076274 , 0.41067994, 0.43293649, 0.41296747,
          0.41412742, 0.42010894, 0.42222496, 0.42315642, 0.42353746,
          0.42932461, 0.43906697, 0.45152885, 0.44700924, 0.44298194,
          0.4524831 , 0.44890166, 0.27708922, 0.26433128, 0.23619747,
          0.03184431, 0.02861505])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.14125032, 0.15423921, 0.27907179, 0.23317051, 0.31532893,
          0.31926886, 0.33972941, 0.37817065, 0.40582867, 0.43521397,
          0.44264334, 0.44947074, 0.45409821, 0.45652942, 0.46128886,
          0.47327523, 0.48817682, 0.50368058, 0.53135695, 0.53546736,
          0.54176798, 0.5879698 , 0.58462327, 0.58419449, 0.59121386,
          0.60431291, 0.59214051, 0.59224933, 0.60293654, 0.59141802,
          0.57636276, 0.57828084, 0.48257101])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.14125032, 0.15423921, 0.27907179, 0.23317051, 0.31532893,
          0.31926886, 0.33972941, 0.37817065, 0.40582867, 0.43521397,
          0.44264334, 0.44947074, 0.45409821, 0.45652942, 0.46128886,
          0.47327523, 0.48817682, 0.50368058, 0.53135695, 0.53546736,
          0.54176798, 0.5879698 , 0.58462327, 0.58419449, 0.59121386,
          0.60431291, 0.59214051, 0.59224933, 0.60293654, 0.59141802,
          0.57636276, 0.57828084, 0.48257101])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.11695605, 0.13506578, 0.14730507, 0.15339977, 0.16592309,
          0.22893695, 0.27795384, 0.28417894, 0.31532893, 0.30880082,
          0.32460074, 0.33456585, 0.35676397, 0.37539609, 0.39785431,
          0.41218144, 0.43424846, 0.43352002, 0.44190623, 0.4483224 ,
          0.45000766, 0.46137089, 0.45787502, 0.46185082, 0.47162479,
          0.48401562, 0.49678899, 0.52587637, 0.53641958, 0.54112146,
          0.53894593, 0.58248164, 0.58205182, 0.58312129, 0.5788658 ,
          0.57815248, 0.58409732, 0.59411942, 0.60978494, 0.607172  ,
          0.60132912, 0.60234148, 0.58841686, 0.59101803]))},
 'news_5cl_2': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-4.99993423e-06,  8.43710524e-02,  7.52479248e-02,  7.21547019e-02,
           7.07402534e-02,  6.93600831e-02,  1.02485666e-01,  1.00932453e-01,
           9.67256430e-02,  9.83836418e-02,  9.61061313e-02,  9.62804956e-02,
           5.83558268e-02,  5.65466880e-02,  5.47800427e-02,  5.27525917e-02,
           5.16173920e-02,  4.94188865e-02,  4.57567365e-02,  4.43311687e-02,
           4.27058877e-02,  4.14693628e-02,  4.24379690e-02,  4.23806056e-02,
           4.24401699e-02,  5.57153359e-02,  5.50179518e-02,  9.25085237e-02,
           9.08392437e-02,  9.07490166e-02,  7.81440014e-02,  8.08874660e-02,
           7.33674211e-02,  2.69974414e-02,  6.05491093e-02,  1.07308867e-01,
           1.05564045e-01,  1.12253551e-01,  1.15525011e-01,  1.32265323e-01,
           1.25561850e-01,  1.08256089e-01,  9.71957789e-02,  7.92066181e-02,
           8.67482923e-02,  1.00986241e-01,  8.66192290e-02,  7.22843047e-02,
           6.56411578e-02,  6.05639672e-02,  6.27175543e-02])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.09671816, 0.09977788, 0.10070522, 0.10727733, 0.10723701,
          0.13923581, 0.14269778, 0.15523367, 0.14518472, 0.15408364,
          0.15598948, 0.16026402, 0.16091851, 0.17512753, 0.17924083,
          0.18221004, 0.18591604, 0.17814471, 0.18735839, 0.16907165,
          0.16992836, 0.16735625, 0.16299259, 0.16165002, 0.15996818,
          0.1573135 , 0.16078353, 0.15548514, 0.16028849, 0.1534267 ,
          0.15755724, 0.14992178, 0.1433761 , 0.14986678, 0.14597824,
          0.13951004, 0.13511198, 0.13705531, 0.05817212, 0.0581867 ,
          0.05448402, 0.05610743, 0.05521921, 0.05565438, 0.05746987,
          0.05752472, 0.05883381, 0.0594904 , 0.06408548, 0.06829928])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-4.99993423e-06, -4.99993423e-06,  1.01209567e-05,  2.02015506e-05,
           3.31617263e-04, -4.99993423e-06,  5.08065971e-06,  1.01209567e-05,
          -4.99993423e-06,  2.02015506e-05,  2.02015506e-05,  4.03627385e-08,
           2.02015506e-05,  4.03627385e-05,  2.02015506e-05,  4.03627385e-08,
           4.03627385e-08,  5.08065971e-06,  2.02015506e-05,  1.01209567e-05,
           1.01209567e-05,  2.02015506e-05,  2.02015506e-05,  2.02015506e-05,
          -4.99993423e-06,  5.08065971e-06,  2.02015506e-05,  4.03627385e-08,
           4.03627385e-08,  1.01209567e-05,  1.01209567e-05,  1.15047842e-06,
           2.02015506e-05,  5.08065971e-06,  5.08065971e-06,  2.02015506e-05,
           2.02015506e-05,  2.02015506e-05,  2.02015506e-05,  1.01209567e-05,
           2.02015506e-05,  2.02015506e-05,  2.02015506e-05,  6.68298142e-05,
           1.01209567e-05,  2.02015506e-05,  2.02015506e-05,  2.02015506e-05,
           2.02015506e-05,  1.01209567e-05])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.11758866, 0.15422441, 0.29953348, 0.31416382, 0.32140934,
          0.31709735, 0.32173106, 0.32630306, 0.32981655, 0.44210566,
          0.46855099, 0.46088658, 0.46182959, 0.47924888, 0.5017487 ,
          0.488103  , 0.50009612, 0.49741279, 0.4992237 , 0.49824553,
          0.49870976, 0.49879599, 0.49650157, 0.49805428, 0.51198188,
          0.50544883, 0.50585593, 0.52439979, 0.51978214, 0.5217546 ,
          0.53022413, 0.5343793 , 0.42417217, 0.42373916, 0.54163519,
          0.44323976, 0.41405185, 0.42459152, 0.43405657, 0.44034961,
          0.55079835, 0.55567049, 0.45341376, 0.4493473 , 0.4428847 ,
          0.40742875, 0.45319366, 0.29670893, 0.42995101])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-4.99993423e-06,  7.08052957e-02,  9.51102929e-02,  7.50788240e-02,
           4.86346584e-02,  4.11490031e-02,  7.95931063e-02,  1.51932128e-01,
           7.72085308e-02,  3.63503776e-02,  1.56055373e-01,  6.42768914e-02,
           1.35697415e-01,  1.19735520e-01,  1.59084824e-01,  1.21930751e-01,
           1.20053180e-01,  1.03929748e-01,  8.04494209e-02,  8.04334905e-02,
           7.87573843e-02,  7.43336953e-02,  7.30172376e-02,  7.13435538e-02,
           6.43156691e-02,  6.40013225e-02,  6.40703332e-02,  6.27557167e-02,
           6.18596767e-02,  6.19039283e-02,  6.17997065e-02,  6.29968519e-02,
           6.25550833e-02,  6.27175543e-02,  6.25550833e-02,  6.25550833e-02,
           6.27175543e-02,  6.27175543e-02,  6.27175543e-02,  6.27175543e-02,
           6.27175543e-02,  6.27175543e-02,  6.27175543e-02,  6.27175543e-02,
           6.27175543e-02,  5.97812905e-02,  5.97812905e-02,  5.97812905e-02,
           5.97812905e-02,  5.97812905e-02])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.10159981, 0.12578917, 0.1551635 , 0.16014433, 0.18103826,
          0.18719504, 0.18830816, 0.17037491, 0.1722289 , 0.16559713,
          0.15979826, 0.15898097, 0.15729551, 0.15231236, 0.14704692,
          0.13726231, 0.13536854, 0.12758256, 0.05761616, 0.05493955,
          0.05384241, 0.0548241 , 0.054858  , 0.05486466, 0.05722438,
          0.0581294 , 0.05712611, 0.05924627, 0.06136508, 0.06180872,
          0.06251497, 0.06456167, 0.06613627, 0.06635383, 0.06889355,
          0.06729781, 0.06816445, 0.06829928, 0.06829928, 0.06829928,
          0.06829928, 0.06829928, 0.06829928, 0.06829928, 0.06829928,
          0.06829928, 0.06829928, 0.06829928, 0.06829928])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-4.99993423e-06, -4.99993423e-06,  4.03627385e-08,  5.08065971e-06,
           5.08065971e-06, -4.99993423e-06,  2.02015506e-05,  4.03627385e-05,
           2.02015506e-05,  2.02015506e-05,  5.08065971e-06,  2.02015506e-05,
           4.03627385e-08,  4.03627385e-05,  2.02015506e-05,  5.08065971e-06,
           5.08065971e-06,  5.08065971e-06,  2.42945816e-05,  1.18556165e-04,
           2.02015506e-05,  4.03627385e-05, -3.91264026e-06,  4.03627385e-05,
           4.03627385e-05,  2.02015506e-05,  2.02015506e-05,  2.02015506e-05,
           2.02015506e-05,  1.01209567e-05,  2.02015506e-05,  2.02015506e-05,
           5.08065971e-06,  5.08065971e-06,  2.02015506e-05,  5.08065971e-06,
           5.08065971e-06,  5.08065971e-06,  5.08065971e-06,  5.08065971e-06,
           5.08065971e-06,  5.08065971e-06,  5.08065971e-06,  5.08065971e-06,
           1.92466850e-05,  1.92466850e-05,  1.72908611e-04,  1.62025466e-04,
           1.62025466e-04,  1.62025466e-04])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([1.02317059e-01, 1.42826634e-01, 1.84613258e-01, 2.08314883e-01,
          1.81021039e-01, 2.16482201e-01, 3.19907032e-01, 3.30959410e-01,
          3.26410728e-01, 3.30933875e-01, 3.33639961e-01, 3.48930045e-01,
          3.54353296e-01, 3.61042362e-01, 3.62362356e-01, 3.75824750e-01,
          3.88024881e-01, 3.87357396e-01, 3.87464277e-01, 4.01277319e-01,
          4.12575409e-01, 4.23070688e-01, 4.26420902e-01, 4.28303819e-01,
          4.39004050e-01, 4.51155860e-01, 4.52308984e-01, 4.50835523e-01,
          4.53897494e-01, 4.51463983e-01, 4.49072379e-01, 4.50921222e-01,
          3.04830350e-01, 3.02752860e-01, 3.03762732e-01, 3.03201787e-01,
          4.20336623e-04, 1.98300465e-04, 3.64967909e-05, 5.08065971e-06,
          5.82581592e-05, 2.13302792e-05, 4.36726993e-05, 4.36726993e-05,
          1.92466850e-05, 1.97349581e-04, 1.62025466e-04, 1.62025466e-04,
          1.62025466e-04])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.00000000e+00, 1.01209567e-05, 1.46462151e-01, 4.03627385e-08,
          5.37989675e-05, 9.25356210e-03, 1.10936899e-04, 1.26395895e-04,
          8.40440186e-05, 2.38316169e-04, 3.39432722e-04, 3.17071130e-01,
          3.05811053e-01, 1.95710415e-02, 8.22113481e-03, 3.03492016e-01,
          4.04259193e-01, 3.72405709e-01, 3.02349900e-01, 2.18311650e-01,
          3.15300799e-01, 3.87379594e-01, 3.42974894e-01, 4.47642420e-01,
          4.40044480e-01, 5.94993854e-01, 4.50580639e-01, 4.53658594e-01,
          3.01772427e-01, 4.45018517e-01, 4.49833800e-01, 4.42324859e-01,
          4.53264708e-01, 5.28746219e-01, 4.45930718e-01, 5.31089465e-01,
          5.34319293e-01, 4.48114061e-01, 4.54829790e-01, 4.45706701e-01,
          4.46916197e-01, 4.44967109e-01, 4.43577200e-01, 4.34457401e-01,
          4.32813688e-01, 5.05855745e-01, 4.38642491e-01, 5.25922674e-01,
          5.09908090e-01, 4.74265375e-01])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.30040136, 0.44365508, 0.44348944, 0.4431741 ,
          0.44336667, 0.43801947, 0.44252428, 0.44478865, 0.44478865,
          0.44760978, 0.44638675, 0.44824307, 0.44824307, 0.45095977,
          0.45095977, 0.45029642, 0.4505483 , 0.4505483 , 0.44906802,
          0.45200651, 0.44988852, 0.45389137, 0.448601  , 0.45303585,
          0.51628255, 0.45461986, 0.4549441 , 0.53342235, 0.53861255,
          0.54035604, 0.5493098 , 0.548472  , 0.50367261, 0.50849202,
          0.50772323, 0.5191709 , 0.51193766, 0.52824264, 0.52660635,
          0.53409262, 0.53384808, 0.53396199, 0.52969814, 0.56932932,
          0.5669023 , 0.57034089, 0.57206354, 0.57265053, 0.56459137])),
  'RSP K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98, 1.  ]),
   array([5.03183201e-01, 5.08146841e-01, 5.07077851e-01, 5.05772029e-01,
          5.05772029e-01, 5.07077851e-01, 5.13280061e-01, 5.07077851e-01,
          5.07077851e-01, 5.05495154e-01, 5.04674815e-01, 5.12921988e-01,
          5.05495154e-01, 5.09387662e-01, 5.02283379e-01, 5.05408087e-01,
          5.05193088e-01, 5.02867233e-01, 5.07912857e-01, 5.02350064e-01,
          5.02350064e-01, 4.98851053e-01, 5.12615684e-01, 5.04561607e-01,
          4.99857384e-01, 4.99718717e-01, 5.05400032e-01, 5.00345511e-01,
          4.99284692e-01, 4.98789331e-01, 4.90577740e-01, 4.92717435e-01,
          4.95018603e-01, 4.98808303e-01, 4.94679925e-01, 4.92433896e-01,
          4.96753087e-01, 4.98899239e-01, 5.00512444e-01, 4.96307401e-01,
          5.03775428e-01, 5.09760416e-01, 5.04341760e-01, 5.18202017e-01,
          5.23597531e-01, 5.19184521e-01, 5.32898498e-01, 1.01209567e-05])),
  'FE K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.50867208, 0.50708418, 0.50999079, 0.50719115, 0.51213055,
          0.55448528, 0.51738397, 0.51173735, 0.52005446, 0.52005446,
          0.5149153 , 0.51320417, 0.51511271, 0.51402485, 0.51402485,
          0.51892012, 0.51225642, 0.51794369, 0.52175777, 0.52067302,
          0.52285544, 0.51986347, 0.52239996, 0.5235804 , 0.52135962,
          0.5266175 , 0.60335249, 0.52793347, 0.53046638, 0.53126996,
          0.52540406, 0.52648962, 0.52349957, 0.53282525, 0.5303754 ,
          0.52302103, 0.52272833, 0.53502913, 0.53488962, 0.53487036,
          0.53877975, 0.53295756, 0.53278238, 0.53985134, 0.53522274,
          0.52667637, 0.519614  ])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([1.01209567e-05, 5.19796687e-01, 5.11817986e-01, 5.10386443e-01,
          5.10607673e-01, 5.10896039e-01, 5.10386443e-01, 5.10386443e-01,
          5.12161762e-01, 5.12161762e-01, 5.13624953e-01, 5.12161762e-01,
          5.12202572e-01, 5.12202572e-01, 5.12202572e-01, 5.12202572e-01,
          5.12202572e-01, 5.12202572e-01, 5.12202572e-01, 5.12202572e-01,
          5.12202572e-01, 5.12202572e-01, 5.12202572e-01, 5.12202572e-01,
          5.12202572e-01, 5.12202572e-01, 5.07967337e-01, 5.07967337e-01,
          5.07967337e-01, 5.07967337e-01, 5.10659501e-01, 5.10659501e-01,
          5.10659501e-01, 5.10659501e-01, 5.10659501e-01, 5.03183201e-01,
          5.03183201e-01, 5.03183201e-01, 5.03183201e-01, 5.03183201e-01,
          5.03183201e-01, 5.03183201e-01, 5.03183201e-01, 5.03183201e-01,
          5.03183201e-01, 5.03183201e-01, 5.03183201e-01, 5.03183201e-01,
          5.03183201e-01, 5.03183201e-01, 5.03183201e-01])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96]),
   array([0.07431793, 0.07678079, 0.10256119, 0.09709556, 0.11481263,
          0.10683955, 0.10872379, 0.12300981, 0.15276828, 0.15597596,
          0.17471274, 0.1739181 , 0.17473288, 0.17445731, 0.18621255,
          0.17902934, 0.18875977, 0.25845856, 0.26562364, 0.26711497,
          0.27227262, 0.28360156, 0.19798831, 0.28770199, 0.29226198,
          0.29044889, 0.29414008, 0.41444136, 0.40948393, 0.42746653,
          0.42916731, 0.44387718, 0.44621448, 0.43322846, 0.44391396,
          0.44886583, 0.44760674, 0.43910034, 0.45754638, 0.44645187,
          0.44747234, 0.36687274, 0.33430721, 0.23899718, 0.19490637,
          0.13574959, 0.01942809, 0.02029689])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.1191981 , 0.16439464, 0.17211124, 0.1909137 , 0.19312516,
          0.20627478, 0.21664962, 0.2350903 , 0.28138148, 0.29571998,
          0.35623529, 0.36778329, 0.38752694, 0.376562  , 0.41559107,
          0.42301323, 0.42735362, 0.43827607, 0.44679653, 0.47139714,
          0.47966826, 0.48256366, 0.49314311, 0.48939389, 0.50333411,
          0.50764663, 0.47964778, 0.4769107 , 0.51279429, 0.51363975,
          0.51796728, 0.54869755, 0.53889776])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.1191981 , 0.16439464, 0.17211124, 0.1909137 , 0.19312516,
          0.20627478, 0.21664962, 0.2350903 , 0.28138148, 0.29571998,
          0.35623529, 0.36778329, 0.38752694, 0.376562  , 0.41559107,
          0.42301323, 0.42735362, 0.43827607, 0.44679653, 0.47139714,
          0.47966826, 0.48256366, 0.49314311, 0.48939389, 0.50333411,
          0.50764663, 0.47964778, 0.4769107 , 0.51279429, 0.51363975,
          0.51796728, 0.54869755, 0.53889776])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.11585392, 0.1189812 , 0.12793187, 0.13279523, 0.16496305,
          0.18070716, 0.18618735, 0.18837764, 0.19262031, 0.20477132,
          0.2053596 , 0.20800725, 0.2095355 , 0.23019596, 0.23753968,
          0.28405932, 0.27285439, 0.28863638, 0.34809853, 0.36744928,
          0.37608087, 0.40863111, 0.38432965, 0.40859833, 0.411895  ,
          0.42463278, 0.43000596, 0.44130122, 0.45760654, 0.46267262,
          0.47398985, 0.47840361, 0.48482676, 0.49431934, 0.49059491,
          0.49529567, 0.48433424, 0.47408128, 0.47958842, 0.48161598,
          0.51296128, 0.54981497, 0.55314276, 0.49220926]))},
 'news_5cl_3': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-1.29873126e-05,  1.10913408e-01,  1.24901395e-01,  1.23558501e-01,
           1.22947232e-01,  1.24498883e-01,  1.22881219e-01,  1.25360877e-01,
           1.20057657e-01,  1.13403760e-01,  1.01754813e-01,  9.46139835e-02,
           8.58965521e-02,  7.76394861e-02,  7.28459130e-02,  6.95020837e-02,
           6.45317462e-02,  6.13795476e-02,  7.02905751e-02,  6.99502154e-02,
           6.92368026e-02,  3.81222351e-02,  3.82372259e-02,  3.69557383e-02,
           3.55914415e-02,  3.55914415e-02,  3.38795426e-02,  5.23342070e-02,
           5.13645332e-02,  5.04044384e-02,  4.94147776e-02,  4.42547420e-02,
           4.25697963e-02,  4.09776901e-02,  4.16943271e-02,  3.87798541e-02,
           3.71810245e-02,  3.28183283e-02,  3.29416429e-02,  3.04332033e-02,
           1.11159714e-01,  9.53947661e-02,  9.23304785e-02,  7.85137553e-02,
           3.92321336e-02,  6.67504061e-02,  5.92451143e-02,  3.57856072e-02,
           3.16594881e-02,  2.67866032e-02,  1.31466599e-01])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.09730648, 0.19923173, 0.22611732, 0.25734486, 0.27502268,
          0.28552885, 0.28392552, 0.29322524, 0.2911455 , 0.2968147 ,
          0.27110887, 0.29619719, 0.30157258, 0.28147152, 0.30203005,
          0.29989265, 0.30023426, 0.30060531, 0.31326903, 0.3048258 ,
          0.30604789, 0.29215315, 0.30420039, 0.28923992, 0.29213032,
          0.28866061, 0.28375036, 0.27598159, 0.26755817, 0.26320738,
          0.26279262, 0.26060426, 0.2442851 , 0.15524124, 0.1379595 ,
          0.13817613, 0.13968807, 0.13407176, 0.13231942, 0.02744395,
          0.0255622 , 0.02102793, 0.02204285, 0.02281547, 0.02083478,
          0.02105797, 0.02068969, 0.02172134, 0.02319405])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-1.29873126e-05,  2.19460981e-06, -2.86603100e-06,  2.24371730e-05,
           1.64099701e-02,  1.33358224e-03,  4.28201228e-06,  2.19460981e-06,
           1.73765322e-05,  1.73765322e-05, -2.86603100e-06,  2.19460981e-06,
           2.24371730e-05,  2.24371730e-05,  1.73765322e-05,  1.23158914e-05,
           2.19460981e-06, -7.86274143e-07,  1.19728316e-02,  2.19460981e-06,
           4.28201228e-06,  4.26797363e-05,  3.46917308e-05,  1.23158914e-05,
           4.26797363e-05,  1.73765322e-05,  7.30435811e-05,  1.73765322e-05,
           1.73765322e-05,  7.30435811e-05,  2.19460981e-06,  1.73765322e-05,
           1.73765322e-05,  1.73765322e-05,  5.52611979e-05,  4.26797363e-05,
           7.30435811e-05,  1.73765322e-05,  2.98431176e-05,  1.73765322e-05,
           2.19460981e-06,  1.94868715e-05,  2.19460981e-06, -2.86603100e-06,
           2.19460981e-06,  2.19460981e-06,  2.19460981e-06,  2.19460981e-06,
           2.19460981e-06,  2.19460981e-06])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.27226095, 0.31712331, 0.35986645, 0.36295496, 0.37284069,
          0.37892728, 0.38990015, 0.39101299, 0.39605399, 0.39483157,
          0.39905988, 0.4062014 , 0.41989131, 0.42393526, 0.44400703,
          0.4271604 , 0.42983682, 0.44055449, 0.44697038, 0.45433926,
          0.46400336, 0.4991254 , 0.50366156, 0.48257592, 0.49027799,
          0.49549409, 0.48799831, 0.50373506, 0.51176349, 0.50243058,
          0.50608553, 0.50852512, 0.52443198, 0.52443198, 0.52382182,
          0.52955061, 0.4966305 , 0.50692399, 0.5338345 , 0.54856816,
          0.50450686, 0.53473365, 0.46574898, 0.53449791, 0.45359676,
          0.45743206, 0.52892201, 0.45178245, 0.31486535])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-1.29873126e-05,  1.23646169e-01,  1.22625405e-01,  9.31342099e-02,
           6.94353170e-02,  6.95146486e-02,  1.14313410e-01,  5.88784648e-02,
           1.45256077e-01,  6.04440797e-02,  1.02902302e-01,  1.89446410e-01,
           1.52305534e-01,  6.00640172e-02,  2.45649286e-01,  1.03291312e-01,
           1.61374830e-01,  1.13406521e-01,  1.00899447e-01,  9.11920425e-02,
           8.68354418e-02,  7.87052169e-02,  6.49651616e-02,  4.77840803e-02,
           4.46219911e-02,  3.04192446e-02,  3.10279702e-02,  2.85793470e-02,
           3.01461088e-02,  2.56714171e-02,  2.55964143e-02,  2.51178622e-02,
           2.58892120e-02,  2.51078899e-02,  2.55793059e-02,  2.55504099e-02,
           2.51475748e-02,  2.51078899e-02,  2.55504099e-02,  2.55504099e-02,
           2.55504099e-02,  2.55504099e-02,  2.55504099e-02,  2.55504099e-02,
           2.55504099e-02,  2.55504099e-02,  2.48071173e-02,  2.48071173e-02,
           2.48071173e-02,  2.48071173e-02])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.2122488 , 0.28004679, 0.30881376, 0.3013781 , 0.30252487,
          0.30581265, 0.31050085, 0.30874754, 0.30295874, 0.30403813,
          0.29675586, 0.29821221, 0.28995238, 0.2697237 , 0.25696513,
          0.25035862, 0.14554477, 0.13734277, 0.13642381, 0.13417382,
          0.02650468, 0.02412494, 0.0241088 , 0.02360203, 0.02174283,
          0.02201901, 0.02134248, 0.02045589, 0.02118323, 0.02107952,
          0.02162085, 0.02423073, 0.02223927, 0.02262417, 0.02306181,
          0.02287224, 0.02345216, 0.02368988, 0.02400889, 0.02400062,
          0.02399928, 0.02399928, 0.02399928, 0.02399928, 0.02399928,
          0.02399928, 0.02399928, 0.02399928, 0.02399928])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-1.29873126e-05, -1.29873126e-05,  2.19460981e-06, -1.29873126e-05,
          -2.86603100e-06,  1.73765322e-05, -1.29873126e-05,  1.73765322e-05,
           3.80178955e-03,  4.26797363e-05,  1.73765322e-05,  4.26797363e-05,
           2.97331151e-05,  1.73765322e-05,  1.73765322e-05,  1.94868715e-05,
           7.03406325e-05,  1.73765322e-05,  2.24371730e-05,  1.73765322e-05,
           1.73765322e-05,  2.19460981e-06,  2.19460981e-06,  2.19460981e-06,
           2.19460981e-06,  2.19460981e-06,  2.19460981e-06,  2.19460981e-06,
           4.26797363e-05,  1.73765322e-05,  4.26797363e-05,  3.46917308e-05,
           1.73765322e-05,  1.44185851e-05,  3.46917308e-05,  3.46917308e-05,
           1.73765322e-05,  1.73765322e-05,  1.73765322e-05,  3.46917308e-05,
           3.46917308e-05,  3.46917308e-05,  3.46917308e-05,  3.46917308e-05,
           3.46917308e-05,  9.35029870e-06,  9.35029870e-06,  9.42935643e-06,
           9.42935643e-06,  2.19460981e-06])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([2.20362903e-01, 2.78228837e-01, 3.08049782e-01, 3.26064716e-01,
          3.51519636e-01, 3.60670467e-01, 3.68863218e-01, 3.76267743e-01,
          3.82141710e-01, 3.90042672e-01, 3.91175808e-01, 3.98959850e-01,
          3.95681950e-01, 4.08856385e-01, 4.19680258e-01, 4.25093597e-01,
          4.39448370e-01, 4.48732951e-01, 4.59702713e-01, 4.54765210e-01,
          4.64384765e-01, 4.63977276e-01, 4.66398257e-01, 4.71118246e-01,
          4.74147265e-01, 4.69288440e-01, 4.88064328e-01, 4.81818418e-01,
          4.83831969e-01, 4.76936481e-01, 4.74633076e-01, 4.76994590e-01,
          4.77017956e-01, 2.81246020e-01, 2.81730098e-01, 2.27513464e-01,
          2.22395343e-04, 1.55306047e-04, 3.46917308e-05, 3.46917308e-05,
          3.46917308e-05, 3.46917308e-05, 3.46917308e-05, 9.35029870e-06,
          9.35029870e-06, 9.35029870e-06, 9.42935643e-06, 9.42935643e-06,
          2.19460981e-06])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00000000e+00, -2.86603100e-06,  2.24371730e-05,  1.49412266e-04,
           4.42330458e-05,  9.35029870e-06,  2.34332944e-01, -5.85456056e-06,
           1.58443722e-01,  5.04581454e-04,  8.66153086e-04,  2.30621748e-01,
           2.27676327e-01,  9.17954554e-03,  3.51330954e-01,  1.96623500e-01,
           3.47662400e-01,  4.11956743e-01,  3.26473837e-01,  4.44779618e-01,
           4.62518811e-01,  4.33549664e-01,  5.18540395e-01,  4.79043391e-01,
           4.41636018e-01,  4.57678299e-01,  4.79283258e-01,  4.51241415e-01,
           4.57873567e-01,  4.65622121e-01,  4.34315914e-01,  4.66446661e-01,
           4.86236401e-01,  5.11202779e-01,  5.07054413e-01,  5.28352839e-01,
           4.94099603e-01,  4.87579009e-01,  5.12955897e-01,  4.86201819e-01,
           4.52699149e-01,  4.67714121e-01,  4.86643368e-01,  4.68723221e-01,
           4.95548913e-01,  4.73542332e-01,  4.93868648e-01,  4.77134940e-01,
           4.32176161e-01,  4.94317055e-01])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.46234653, 0.45968663, 0.45525676, 0.45525676,
          0.46491353, 0.46105421, 0.46487207, 0.44449341, 0.46655578,
          0.46348585, 0.45984459, 0.46734375, 0.45613848, 0.46630521,
          0.46630521, 0.45192387, 0.47205323, 0.46659471, 0.47205323,
          0.46508604, 0.46508604, 0.47154036, 0.47202714, 0.45927798,
          0.46046065, 0.45708256, 0.46608498, 0.47500126, 0.46163378,
          0.46968815, 0.45690213, 0.45028354, 0.4672055 , 0.46729299,
          0.47336625, 0.47162904, 0.47816662, 0.48275226, 0.48708593,
          0.48724195, 0.4938448 , 0.49667452, 0.58344999, 0.57782072,
          0.60037254, 0.59177856, 0.56905441, 0.59919449, 0.602068  ])),
  'RSP K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98, 1.  ]),
   array([4.84404186e-01, 4.84404186e-01, 4.84404186e-01, 4.42831852e-01,
          4.39004363e-01, 4.40256312e-01, 4.42601809e-01, 4.76407601e-01,
          4.71581559e-01, 4.38009531e-01, 4.62992439e-01, 4.62992439e-01,
          4.41968786e-01, 4.42528676e-01, 4.42528676e-01, 4.65917595e-01,
          4.64953648e-01, 4.53074293e-01, 4.55139010e-01, 4.54941758e-01,
          4.67688610e-01, 4.66078970e-01, 4.82686909e-01, 4.85555986e-01,
          4.50111460e-01, 4.73786291e-01, 4.44566405e-01, 4.77179592e-01,
          4.59330092e-01, 4.40609617e-01, 4.70408977e-01, 4.73231575e-01,
          4.67831157e-01, 4.66545171e-01, 4.68184111e-01, 4.90147225e-01,
          4.67791202e-01, 4.57065456e-01, 4.51856858e-01, 4.65049900e-01,
          4.59015073e-01, 4.63850567e-01, 4.82198740e-01, 4.81235862e-01,
          4.81980076e-01, 4.95810643e-01, 4.98267190e-01, 4.33695298e-05])),
  'FE K': (array([0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24, 0.26,
          0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48,
          0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68, 0.7 ,
          0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 , 0.92,
          0.94, 0.96, 0.98]),
   array([0.44687663, 0.44988978, 0.4630862 , 0.48370446, 0.49467256,
          0.49693003, 0.48660573, 0.48541939, 0.49200989, 0.49042317,
          0.47641105, 0.47543407, 0.46541838, 0.50991953, 0.5135925 ,
          0.49575558, 0.51159974, 0.51323916, 0.51564338, 0.50415663,
          0.50572663, 0.51245396, 0.50848313, 0.49991396, 0.50314495,
          0.50744296, 0.50460563, 0.50150349, 0.49938068, 0.50728334,
          0.5211181 , 0.52568868, 0.5470711 , 0.52195638, 0.54319937,
          0.50862327, 0.51066795, 0.53241583, 0.5578904 , 0.54508245,
          0.5454698 , 0.56597541, 0.51665458, 0.58277455, 0.58214382,
          0.58882735, 0.62017549])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([2.19460981e-06, 4.59838029e-01, 4.66065275e-01, 4.39138698e-01,
          4.70132135e-01, 4.86880947e-01, 4.86113518e-01, 4.78054856e-01,
          4.78054856e-01, 4.60989841e-01, 4.84919824e-01, 4.78784482e-01,
          4.78784482e-01, 4.84919824e-01, 4.84919824e-01, 4.84919824e-01,
          4.84919824e-01, 4.65734923e-01, 4.65734923e-01, 4.65734923e-01,
          4.65734923e-01, 4.65734923e-01, 4.65734923e-01, 4.65734923e-01,
          4.65734923e-01, 4.65734923e-01, 4.65734923e-01, 4.65734923e-01,
          4.65734923e-01, 4.65734923e-01, 4.65734923e-01, 4.65734923e-01,
          4.65734923e-01, 4.65734923e-01, 4.65734923e-01, 4.65734923e-01,
          4.65734923e-01, 4.65734923e-01, 4.65734923e-01, 4.65734923e-01,
          4.65734923e-01, 4.65734923e-01, 4.65734923e-01, 4.65734923e-01,
          4.65734923e-01, 4.65734923e-01, 4.67282062e-01, 4.67282062e-01,
          4.67282062e-01, 4.67282062e-01, 4.67282062e-01])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96]),
   array([0.11389491, 0.11171854, 0.14023261, 0.18994746, 0.18363144,
          0.18941489, 0.22189767, 0.24144403, 0.24498087, 0.2548832 ,
          0.25412975, 0.2803986 , 0.28539987, 0.2884471 , 0.2881939 ,
          0.29598551, 0.29673082, 0.30539279, 0.30744042, 0.30760971,
          0.30954435, 0.31667213, 0.31914548, 0.31995278, 0.3267784 ,
          0.33215977, 0.33405913, 0.3407537 , 0.4137131 , 0.41637199,
          0.43021843, 0.42581227, 0.43079146, 0.43251403, 0.43355308,
          0.44333643, 0.43931572, 0.46202839, 0.46433205, 0.42263974,
          0.42116385, 0.41174805, 0.36340084, 0.3411906 , 0.29743871,
          0.13601687, 0.01503909, 0.01324233])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.12291847, 0.16700091, 0.2378646 , 0.25962165, 0.26198312,
          0.2759754 , 0.29149942, 0.30095362, 0.32030745, 0.3306442 ,
          0.32928152, 0.33223002, 0.3582047 , 0.36370123, 0.37644689,
          0.38213153, 0.38689398, 0.38884759, 0.3996887 , 0.39115509,
          0.41151675, 0.41341455, 0.42214757, 0.5260877 , 0.53285582,
          0.54260792, 0.53650484, 0.54804243, 0.55296738, 0.54389163,
          0.56274078, 0.49310079, 0.48344827])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.12291847, 0.16700091, 0.2378646 , 0.25962165, 0.26198312,
          0.2759754 , 0.29149942, 0.30095362, 0.32030745, 0.3306442 ,
          0.32928152, 0.33223002, 0.3582047 , 0.36370123, 0.37644689,
          0.38213153, 0.38689398, 0.38884759, 0.3996887 , 0.39115509,
          0.41151675, 0.41341455, 0.42214757, 0.5260877 , 0.53285582,
          0.54260792, 0.53650484, 0.54804243, 0.55296738, 0.54389163,
          0.56274078, 0.49310079, 0.48344827])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88]),
   array([0.08422202, 0.12382262, 0.1800875 , 0.1704442 , 0.22083945,
          0.21811702, 0.24572152, 0.26159264, 0.27289626, 0.26757617,
          0.2829341 , 0.27962339, 0.28469623, 0.30154787, 0.32025229,
          0.31195891, 0.32264262, 0.33730447, 0.32877174, 0.33862995,
          0.35346099, 0.3663422 , 0.36737469, 0.37423205, 0.38022302,
          0.38250716, 0.38795727, 0.39253354, 0.39019027, 0.39746582,
          0.40730491, 0.51186893, 0.41813452, 0.5232389 , 0.52294177,
          0.5286592 , 0.53729972, 0.54065534, 0.54435377, 0.54912933,
          0.57565696, 0.55686624, 0.47923312, 0.48574085]))},
 'polbooks': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-0.00504586,  0.11852593,  0.11852593,  0.11852593,  0.11852593,
           0.11852593,  0.11852593,  0.11852593,  0.62698221,  0.62698221,
           0.62698221,  0.62698221,  0.62698221,  0.62698221,  0.19481761,
           0.25218886,  0.27322911,  0.27322911,  0.27217452,  0.27217452,
           0.27217452,  0.27217452,  0.27217452,  0.29353977,  0.29353977,
           0.29353977,  0.29274053,  0.29274053,  0.31518876,  0.2381039 ,
           0.2381039 ,  0.03376906,  0.03376906,  0.04447436,  0.23683946,
           0.05048291,  0.05660961,  0.09242842,  0.27726648,  0.08526931,
           0.25251901,  0.13506531,  0.16428039,  0.16428039,  0.16428039,
           0.18034268,  0.16428039,  0.16428039,  0.16428039,  0.37221097,
           0.48796978])),
  'Walk H': (array([0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22, 0.24,
          0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46,
          0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66, 0.68,
          0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88, 0.9 ,
          0.92, 0.94, 0.96, 0.98]),
   array([0.61058463, 0.61058463, 0.61058463, 0.61058463, 0.61058463,
          0.61058463, 0.61058463, 0.61058463, 0.61058463, 0.61058463,
          0.63744772, 0.63744772, 0.63744772, 0.63744772, 0.63744772,
          0.63744772, 0.63744772, 0.63744772, 0.59995344, 0.59995344,
          0.59995344, 0.59995344, 0.61454992, 0.61454992, 0.61454992,
          0.61454992, 0.61454992, 0.61454992, 0.60535122, 0.54872658,
          0.55001129, 0.5454059 , 0.55213726, 0.5454059 , 0.56390594,
          0.5454059 , 0.55656765, 0.55981466, 0.55416731, 0.55261524,
          0.55261524, 0.55757637, 0.56110083, 0.54745631, 0.55387385,
          0.55387385, 0.59858088, 0.59858088])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00504586, -0.00504586,  0.13207959,  0.09791852, -0.00504586,
           0.09791852,  0.0356063 ,  0.02210781,  0.00229214,  0.47648846,
           0.04005666,  0.50131681,  0.49540919,  0.53456103,  0.51828957,
           0.55791313,  0.06048716,  0.5888828 ,  0.58880064,  0.56507687,
           0.5854209 ,  0.59339237,  0.59339237,  0.46389443,  0.48377203,
           0.47531133,  0.67432925,  0.67432925,  0.62410707,  0.66372367,
           0.68934782,  0.68934782,  0.68934782,  0.68934782,  0.72674094,
           0.72674094,  0.72674094,  0.72674094,  0.68934782,  0.68934782,
           0.68934782,  0.70781942,  0.67451016,  0.70781942,  0.68934782,
           0.67451016,  0.67451016,  0.70781942,  0.67451016,  0.70781942])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.61058463, 0.61058463, 0.61058463, 0.5883714 , 0.63356208,
          0.61058463, 0.5883714 , 0.5883714 , 0.61058463, 0.61058463,
          0.61058463, 0.61058463, 0.61373334, 0.62297773, 0.62297773,
          0.62297773, 0.62297773, 0.63780222, 0.63780222, 0.67486032,
          0.67486032, 0.67486032, 0.67486032, 0.67486032, 0.67486032,
          0.67486032, 0.67486032, 0.67486032, 0.67486032, 0.67486032,
          0.67486032, 0.67486032, 0.67486032, 0.67486032, 0.67486032,
          0.67486032, 0.67486032, 0.67486032, 0.67486032, 0.67486032,
          0.67486032, 0.67486032, 0.67486032, 0.67486032, 0.67486032,
          0.69344599, 0.68180557, 0.68180557, 0.68180557])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00504586,  0.11852593,  0.11852593,  0.62698221,  0.16139545,
           0.21297899,  0.3588909 ,  0.02819724,  0.31635937,  0.06530436,
           0.11836011,  0.22338833,  0.20171988,  0.19786322,  0.18034268,
           0.18034268,  0.18034268,  0.18034268,  0.18034268,  0.18034268,
           0.18034268,  0.18034268,  0.18034268,  0.18034268,  0.18034268,
           0.16428039,  0.16428039,  0.16428039,  0.16428039,  0.16428039,
           0.16428039,  0.16428039,  0.16428039,  0.16428039,  0.16428039,
           0.16428039,  0.16428039,  0.16428039,  0.16428039,  0.16428039,
           0.16428039,  0.16428039,  0.33990505,  0.3156664 ,  0.29242014,
           0.28842112,  0.17866602,  0.17596017,  0.17596017,  0.17596017])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.61058463, 0.61058463, 0.61058463, 0.61058463, 0.61058463,
          0.61058463, 0.61058463, 0.61058463, 0.61058463, 0.61058463,
          0.61058463, 0.61058463, 0.61444919, 0.5883714 , 0.61058463,
          0.58615415, 0.54774315, 0.54236441, 0.54236441, 0.55001129,
          0.55213726, 0.55213726, 0.55213726, 0.55656765, 0.55416731,
          0.55191219, 0.55191219, 0.55191219, 0.53321434, 0.53712117,
          0.53712117, 0.53712117, 0.55387385, 0.55387385, 0.59858088,
          0.59858088, 0.59858088, 0.59858088, 0.59858088, 0.59858088,
          0.59858088, 0.45150638, 0.43813311, 0.43813311, 0.41049526,
          0.12030235, 0.12030235, 0.12266946, 0.13945638])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00504586, -0.00504586, -0.00504586, -0.00504586,  0.13207959,
           0.53215958,  0.45130074,  0.49306185,  0.04446988,  0.37549269,
           0.61896016,  0.61896016,  0.38279964,  0.59339237,  0.52715086,
           0.47234974,  0.52715086,  0.60834751,  0.67451016,  0.67451016,
           0.72665608,  0.66372367,  0.72674094,  0.66372367,  0.71166033,
           0.71166033,  0.71166033,  0.71166033,  0.71166033,  0.71166033,
           0.71166033,  0.72722358,  0.72630829,  0.72722358,  0.72630829,
           0.72630829,  0.72630829,  0.72617988,  0.70781942,  0.70781942,
           0.70781942,  0.68180557,  0.68180557,  0.69344599,  0.69344599,
           0.66893165,  0.66893165,  0.66893165,  0.66893165,  0.66893165])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.61058463, 0.61058463, 0.61058463, 0.61058463, 0.61058463,
          0.61058463, 0.61058463, 0.61058463, 0.61058463, 0.61058463,
          0.5883714 , 0.5883714 , 0.5883714 , 0.5883714 , 0.5883714 ,
          0.5883714 , 0.53422255, 0.61058463, 0.61058463, 0.61058463,
          0.61058463, 0.61058463, 0.61058463, 0.61058463, 0.59694188,
          0.61058463, 0.62401055, 0.62401055, 0.62401055, 0.62401055,
          0.64273227, 0.64273227, 0.63780222, 0.63780222, 0.63780222,
          0.63780222, 0.65342239, 0.65342239, 0.65342239, 0.65342239,
          0.66893165, 0.65342239, 0.66893165, 0.67324247, 0.66893165,
          0.66893165, 0.66893165, 0.66893165, 0.66893165])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.70781942, 0.70781942, 0.70781942, 0.70781942,
          0.68934782, 0.68934782, 0.68934782, 0.68180557, 0.68755686,
          0.68755686, 0.66859174, 0.68180557, 0.68180557, 0.68180557,
          0.68180557, 0.68180557, 0.68180557, 0.68180557, 0.68180557,
          0.68180557, 0.68180557, 0.68180557, 0.68180557, 0.68180557,
          0.68180557, 0.68180557, 0.68180557, 0.68180557, 0.68180557,
          0.68180557, 0.68180557, 0.68180557, 0.68180557, 0.68180557,
          0.66281484, 0.66281484, 0.66281484, 0.66281484, 0.66281484,
          0.66281484, 0.68180557, 0.68180557, 0.68180557, 0.68180557,
          0.71461306, 0.71461306, 0.73314689, 0.73314689, 0.73314689])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.68180557, 0.68180557, 0.68180557, 0.68180557,
          0.68180557, 0.68180557, 0.68180557, 0.68180557, 0.68180557,
          0.68180557, 0.68180557, 0.68180557, 0.68180557, 0.69344599,
          0.69344599, 0.69344599, 0.69344599, 0.69344599, 0.69344599,
          0.69344599, 0.69344599, 0.69344599, 0.69344599, 0.69344599,
          0.69344599, 0.69344599, 0.69344599, 0.69344599, 0.69344599,
          0.69344599, 0.69344599, 0.69344599, 0.69344599, 0.69344599,
          0.69344599, 0.69344599, 0.68180557, 0.68180557, 0.68180557,
          0.68180557, 0.68180557, 0.69784033, 0.71461306, 0.71461306,
          0.7222709 , 0.7222709 , 0.73825727, 0.73825727, 0.75492719])),
  'RSP K': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.61360686, 0.61360686, 0.61360686, 0.61360686, 0.61360686,
          0.61360686, 0.61360686, 0.62744025, 0.61360686, 0.61360686,
          0.61360686, 0.61360686, 0.61360686, 0.61360686, 0.61360686,
          0.61360686, 0.61360686, 0.61360686, 0.61058463, 0.61058463,
          0.61058463, 0.61058463, 0.59298072, 0.59960103, 0.59960103,
          0.59960103, 0.59960103, 0.59960103, 0.58944511, 0.58944511,
          0.59960103, 0.59960103, 0.55515871, 0.59960103, 0.55515871,
          0.55515871, 0.55515871, 0.55515871, 0.568152  , 0.568152  ,
          0.568152  , 0.568152  , 0.62767172, 0.62767172, 0.6687841 ,
          0.63988321, 0.68700544, 0.67486032, 0.67486032, 0.49678237])),
  'FE K': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.62744025, 0.61360686, 0.61360686, 0.61360686, 0.61360686,
          0.61360686, 0.61360686, 0.60722697, 0.60722697, 0.59298072,
          0.59298072, 0.62375509, 0.62375509, 0.62375509, 0.62375509,
          0.63505996, 0.63505996, 0.63440055, 0.63440055, 0.66497675,
          0.66497675, 0.64011746, 0.68700544, 0.68700544, 0.68700544,
          0.68700544, 0.65700127, 0.65700127, 0.65700127, 0.65700127,
          0.65700127, 0.65700127, 0.65700127, 0.65700127, 0.68700544,
          0.68700544, 0.67486032, 0.67486032, 0.67486032, 0.67486032,
          0.67486032, 0.67486032, 0.67486032, 0.67486032, 0.67486032,
          0.67486032, 0.67486032, 0.67486032, 0.69344599])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.70781942, 0.68180557, 0.66281484, 0.67486032, 0.67486032,
          0.67486032, 0.67486032, 0.67486032, 0.67486032, 0.67486032,
          0.67486032, 0.67486032, 0.67486032, 0.67486032, 0.65841087,
          0.65841087, 0.65841087, 0.65841087, 0.62933531, 0.62744025,
          0.62744025, 0.62744025, 0.62744025, 0.62744025, 0.62744025,
          0.62744025, 0.62744025, 0.62744025, 0.62744025, 0.62744025,
          0.62744025, 0.62744025, 0.62744025, 0.62744025, 0.62744025,
          0.62744025, 0.62744025, 0.62744025, 0.62744025, 0.62744025,
          0.62744025, 0.62744025, 0.62744025, 0.62744025, 0.62744025,
          0.61360686, 0.61360686, 0.61360686, 0.61360686, 0.61360686,
          0.61360686])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94]),
   array([0.61058463, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.60824915, 0.58337652,
          0.60895911, 0.58337652])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.61058463, 0.61058463, 0.61058463, 0.61058463, 0.56693733,
          0.56693733, 0.56693733, 0.56693733, 0.56693733, 0.56693733,
          0.56693733, 0.54629669, 0.54629669, 0.54629669, 0.54629669,
          0.54629669, 0.54629669, 0.54629669, 0.54629669, 0.54629669,
          0.53412808, 0.57313764, 0.57313764, 0.63308303, 0.63308303,
          0.63308303, 0.62876763, 0.64863079, 0.64863079, 0.68700544,
          0.68700544, 0.67486032, 0.67486032])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.61058463, 0.61058463, 0.61058463, 0.61058463, 0.56693733,
          0.56693733, 0.56693733, 0.56693733, 0.56693733, 0.56693733,
          0.56693733, 0.54629669, 0.54629669, 0.54629669, 0.54629669,
          0.54629669, 0.54629669, 0.54629669, 0.54629669, 0.54629669,
          0.53412808, 0.57313764, 0.57313764, 0.63308303, 0.63308303,
          0.63308303, 0.62876763, 0.64863079, 0.64863079, 0.68700544,
          0.68700544, 0.67486032, 0.67486032])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84]),
   array([0.61058463, 0.61058463, 0.53422255, 0.61058463, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.5883714 , 0.5883714 , 0.5883714 , 0.5883714 ,
          0.61058463, 0.61058463, 0.61058463, 0.5883714 , 0.5883714 ,
          0.5883714 , 0.5883714 , 0.5883714 , 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.53422255, 0.53422255, 0.53422255,
          0.53422255, 0.53422255, 0.55478086, 0.55478086, 0.55478086,
          0.55478086, 0.55478086, 0.55478086, 0.55478086, 0.55478086,
          0.56350253, 0.56350253]))},
 'webkb_cornell': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-0.00242311,  0.0153258 ,  0.0153258 ,  0.0153258 ,  0.0153258 ,
           0.0153258 ,  0.0153258 ,  0.0153258 ,  0.0153258 ,  0.0153258 ,
           0.0153258 ,  0.00813199,  0.00813199,  0.05295227,  0.05295227,
           0.05295227,  0.05295227,  0.05295227,  0.05295227,  0.04512009,
           0.05023099,  0.05023099,  0.04551937,  0.04551937,  0.04551937,
           0.04551937,  0.04551937,  0.04551937,  0.04596736,  0.05039965,
           0.04397422,  0.04397422,  0.04974984,  0.04974984,  0.04974984,
           0.04974984,  0.07777537,  0.0530873 ,  0.0530873 ,  0.02787882,
           0.07424314,  0.08517969,  0.05199599,  0.06027209,  0.07028473,
           0.07354107,  0.07854555,  0.09038832,  0.06057485,  0.0438722 ,
           0.04519445])),
  'Walk H': (array([], dtype=float64), array([], dtype=float64)),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-2.42310989e-03,  1.71674101e-05,  1.71674101e-05,  1.71674101e-05,
          -1.33845705e-02, -1.33845705e-02, -2.68556085e-02, -2.68556085e-02,
          -2.68556085e-02,  3.18743409e-03,  3.18743409e-03,  2.45123622e-02,
          -3.04251274e-03, -3.04251274e-03,  4.97299259e-02,  4.97299259e-02,
           2.11707109e-02,  2.11707109e-02,  2.11707109e-02,  2.11707109e-02,
           2.11707109e-02,  1.86970178e-03, -2.40397434e-03, -6.49681610e-03,
           2.16944637e-03, -1.85647576e-02, -1.70844210e-02, -1.70844210e-02,
          -1.70844210e-02, -2.10022138e-02, -6.19100523e-03,  6.58957748e-03,
          -2.63883849e-03, -6.60134366e-03, -9.99919337e-03,  2.22671111e-02,
           2.22671111e-02,  2.22671111e-02,  2.02843442e-02,  1.11488586e-02,
           1.90981964e-02,  2.37306123e-02,  9.30670795e-03,  1.11488586e-02,
           9.30670795e-03,  9.30670795e-03,  2.02843442e-02,  2.02843442e-02,
           1.58612486e-02,  2.02843442e-02])),
  'logFor H': (array([], dtype=float64), array([], dtype=float64)),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00242311,  0.0153258 ,  0.0153258 ,  0.00710593, -0.00705582,
           0.05295227,  0.05295227, -0.00705582,  0.04958107,  0.04580394,
           0.04986583,  0.05610059,  0.0496587 ,  0.0674797 ,  0.07702907,
           0.07120621,  0.07211757,  0.0723677 ,  0.08333656,  0.08333656,
           0.08245483,  0.09151639,  0.09197597,  0.07519221,  0.10247455,
           0.0537077 ,  0.05075637,  0.05090266,  0.04735852,  0.04735852,
           0.04519445,  0.04519445,  0.04658365,  0.04519445,  0.04519445,
           0.04519445,  0.04519445,  0.04519445,  0.04519445,  0.04519445,
           0.04519445,  0.04519445,  0.04519445,  0.04519445,  0.04519445,
           0.04519445,  0.04541262,  0.04541262,  0.04541262,  0.04541262])),
  'logComm H': (array([], dtype=float64), array([], dtype=float64)),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-2.42310989e-03,  1.71674101e-05,  1.71674101e-05, -1.33845705e-02,
          -1.33845705e-02, -6.39835516e-04, -6.39835516e-04, -6.39835516e-04,
          -3.04023763e-03, -3.04023763e-03,  1.45576349e-03,  5.23833696e-03,
           5.23833696e-03, -6.94965944e-04, -6.94965944e-04, -6.94965944e-04,
          -6.08521631e-04, -6.08521631e-04, -5.17873608e-03, -5.17873608e-03,
           4.49920815e-02,  4.49920815e-02,  4.01153515e-02,  2.22671111e-02,
           2.02843442e-02,  2.02843442e-02,  2.02843442e-02,  2.22671111e-02,
           2.22671111e-02,  2.22671111e-02,  2.22671111e-02,  2.22671111e-02,
           1.90981964e-02,  1.90981964e-02,  1.90981964e-02,  2.37306123e-02,
           1.72893610e-02,  1.72893610e-02,  1.30336415e-02,  6.59997524e-03,
           2.20482503e-02,  2.02843442e-02,  2.02843442e-02,  2.02843442e-02,
           2.59276039e-02,  2.69952477e-02,  1.58612486e-02,  1.40310022e-02,
           1.40310022e-02,  1.40310022e-02])),
  'logHeat H': (array([], dtype=float64), array([], dtype=float64)),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.        ,  0.01490479,  0.01490479,  0.01490479, -0.02560605,
           0.02207765,  0.03362347,  0.04872022,  0.05160885,  0.00098908,
           0.02444657,  0.03032723,  0.00392319,  0.04637995, -0.01371432,
          -0.01128125, -0.02329171, -0.0070515 ,  0.00139701, -0.01212475,
          -0.00989043,  0.02571648, -0.00989043, -0.00989043,  0.01323518,
           0.00918389,  0.02005184,  0.00983627,  0.00901908,  0.00901908,
           0.00983627,  0.00355109, -0.0100678 ,  0.00301747,  0.00301747,
           0.00301747,  0.00301747,  0.00301747,  0.00616205,  0.00635512,
           0.00355109,  0.00616205,  0.00355109,  0.0022071 ,  0.0022071 ,
           0.0022071 ,  0.0022071 ,  0.00713794,  0.00941448,  0.01772447])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.        , -0.02502667, -0.02502667,  0.05148347,  0.00064563,
           0.04755324, -0.02966803,  0.0511337 , -0.00690972,  0.03925024,
           0.04287031, -0.00690972,  0.01792074,  0.01792074,  0.01792074,
           0.01985055,  0.01985055,  0.01985055,  0.01985055,  0.02005184,
           0.01209938,  0.01679273,  0.02005184,  0.00412992,  0.00412992,
          -0.01199558,  0.00901908,  0.00901908,  0.00036315,  0.00901908,
           0.00301747,  0.00301747,  0.00301747,  0.00301747,  0.00355109,
           0.00271905,  0.00355109,  0.00271905,  0.00355109,  0.00355109,
           0.00355109,  0.0022071 ,  0.0022071 ,  0.02562725,  0.02562725,
           0.02562725,  0.02757297,  0.02757297,  0.02265674,  0.0299746 ])),
  'RSP K': (array([], dtype=float64), array([], dtype=float64)),
  'FE K': (array([], dtype=float64), array([], dtype=float64)),
  'SP-CT H': (array([], dtype=float64), array([], dtype=float64)),
  'NormalizedHeat R': (array([], dtype=float64), array([], dtype=float64)),
  'PersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'ModifiedPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'HeatPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64))},
 'webkb_texas': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([ 0.00091887,  0.06959979,  0.06959979,  0.09317805,  0.09317805,
           0.09317805,  0.09317805,  0.14297685,  0.09317805,  0.09317805,
           0.09317805,  0.09317805,  0.16206167,  0.16206167,  0.16206167,
           0.17113772,  0.17113772,  0.17113772,  0.17113772,  0.17113772,
           0.19091153,  0.19150594, -0.03237325,  0.19844455,  0.19844455,
           0.14534992,  0.20371991,  0.20371991,  0.21355083,  0.21355083,
           0.21355083,  0.21355083,  0.18829925,  0.16831329,  0.18072815,
           0.16831329,  0.18973779,  0.19462101,  0.18609709,  0.20370307,
           0.20751411,  0.15176651,  0.15764751,  0.21524629,  0.19515894,
           0.13440553,  0.15942718,  0.13605906,  0.17427431,  0.17467769,
           0.03016479])),
  'Walk H': (array([], dtype=float64), array([], dtype=float64)),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00091887, -0.01171244, -0.01171244, -0.02397225, -0.02397225,
           0.02469533,  0.02469533,  0.02469533,  0.02469533,  0.02469533,
           0.02469533,  0.0335955 , -0.00182675,  0.00129038,  0.06971016,
           0.06971016,  0.06971016, -0.01594672,  0.11594879,  0.11419096,
           0.11001673, -0.01013352, -0.01013352,  0.02404694, -0.00975056,
           0.02715715,  0.02715715,  0.03693245,  0.03693245,  0.0403413 ,
           0.01765722,  0.01765722, -0.00356284,  0.02395521,  0.02395521,
          -0.00384508,  0.01864833,  0.01864833,  0.02118917, -0.02584367,
           0.02226377,  0.00622181,  0.00622181,  0.00622181,  0.00972422,
           0.0068397 ,  0.0117222 , -0.01295808, -0.01295808, -0.01295808])),
  'logFor H': (array([], dtype=float64), array([], dtype=float64)),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00091887,  0.09317805,  0.09317805,  0.16206167,  0.16206167,
          -0.03237325,  0.12337964,  0.12337964,  0.16950253,  0.17033099,
           0.19320889,  0.19028374,  0.17214374,  0.19219485,  0.22589274,
           0.22746948,  0.20549534,  0.16397303,  0.20362854,  0.18688636,
           0.16852511,  0.14296271,  0.17130631,  0.17130631,  0.17467769,
           0.17467769,  0.17467769,  0.17467769,  0.17467769,  0.17467769,
           0.17467769,  0.17467769,  0.17467769,  0.17467769,  0.17467769,
           0.17467769,  0.17467769,  0.17467769,  0.17467769,  0.17467769,
           0.17467769,  0.17467769,  0.17467769,  0.17467769,  0.17467769,
           0.17467769,  0.17467769,  0.17467769,  0.17467769,  0.17467769])),
  'logComm H': (array([], dtype=float64), array([], dtype=float64)),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00091887, -0.01171244, -0.01171244, -0.02397225,  0.00091887,
           0.02469533,  0.02469533,  0.02469533,  0.02469533,  0.00129038,
           0.00129038,  0.00129038,  0.02064133,  0.02595628, -0.00772635,
          -0.00772635, -0.00772635, -0.00772635, -0.00772635, -0.00975056,
          -0.00975056, -0.00975056, -0.00365442, -0.00365442, -0.00365442,
          -0.00365442, -0.00365442,  0.04001115,  0.01238217, -0.0067812 ,
           0.00177295,  0.00287213,  0.07057853,  0.17270927,  0.03097737,
          -0.00053076,  0.00622181,  0.01507765,  0.0045838 ,  0.00369692,
          -0.004685  ,  0.00177295, -0.01295808, -0.01295808, -0.01295808,
          -0.01295808, -0.01295808, -0.01295808, -0.01295808, -0.01295808])),
  'logHeat H': (array([], dtype=float64), array([], dtype=float64)),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.        , -0.0109916 , -0.0109916 , -0.0109916 , -0.0109916 ,
          -0.0109916 , -0.01267072, -0.01267072,  0.00183638,  0.01005937,
           0.00451921,  0.02617539,  0.00451921,  0.01760796,  0.10917952,
          -0.00289975, -0.01884644, -0.00289975, -0.00289975, -0.00289975,
          -0.00289975,  0.01146598,  0.00635145,  0.00635145,  0.08772393,
           0.00635145, -0.00985999,  0.08710978,  0.08710978,  0.01715606,
           0.01715606,  0.02287787,  0.02287787,  0.02287787,  0.08710978,
           0.08042368,  0.08042368,  0.08042368,  0.08653494,  0.08653494,
           0.08653494,  0.09985535,  0.09376061,  0.09985535,  0.14550958,
           0.13981263,  0.14550958,  0.15556865,  0.14051665,  0.12966007])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00000000e+00,  5.35195727e-03,  1.70963916e-02,  8.15757629e-05,
           8.15757629e-05,  8.15757629e-05,  8.15757629e-05,  5.19385321e-03,
          -1.61070426e-02, -1.61070426e-02, -1.78837685e-02, -2.37227889e-04,
          -2.37227889e-04,  1.14659792e-02,  1.14659792e-02,  1.14659792e-02,
           1.14659792e-02,  1.14659792e-02,  1.14659792e-02,  8.73649794e-02,
           8.73649794e-02,  1.01292962e-01,  1.01292962e-01,  9.49959115e-02,
           9.49959115e-02,  1.76363250e-02,  1.76363250e-02,  2.28778715e-02,
           2.28778715e-02,  2.28778715e-02,  8.71097797e-02,  8.04236769e-02,
           1.70615367e-02,  9.30415731e-02,  9.30415731e-02,  1.11905138e-01,
           1.27072109e-01,  9.26519072e-02,  8.66537432e-02,  9.98553512e-02,
           1.07191689e-01,  1.28309492e-01,  1.34054178e-01,  1.39586325e-01,
           1.23614113e-01,  1.23614113e-01,  1.41911890e-01,  1.30736419e-01,
           1.25476729e-01,  1.25524804e-01])),
  'RSP K': (array([], dtype=float64), array([], dtype=float64)),
  'FE K': (array([], dtype=float64), array([], dtype=float64)),
  'SP-CT H': (array([], dtype=float64), array([], dtype=float64)),
  'NormalizedHeat R': (array([], dtype=float64), array([], dtype=float64)),
  'PersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'ModifiedPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'HeatPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64))},
 'webkb_washington': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-0.00128351, -0.01211373,  0.04161003, -0.01211373,  0.00955561,
           0.00955561,  0.0183142 ,  0.0183142 ,  0.0183142 ,  0.0183142 ,
           0.0183142 ,  0.0183142 ,  0.0183142 ,  0.0183142 ,  0.0183142 ,
           0.0183142 ,  0.0183142 ,  0.0183142 ,  0.0183142 ,  0.0183142 ,
           0.28234877,  0.2589989 ,  0.2589989 ,  0.2589989 ,  0.2589989 ,
           0.2589989 ,  0.2589989 ,  0.28234877,  0.18146361,  0.18379899,
           0.29586205,  0.32211763,  0.29167619,  0.29167619,  0.29167619,
           0.03716316,  0.28376935,  0.27753649,  0.27307905,  0.24891524,
           0.25769612,  0.27781324,  0.28317934,  0.2932691 ,  0.28619538,
           0.28257911,  0.23830153,  0.33835379,  0.30116648,  0.3098554 ,
           0.33843393])),
  'Walk H': (array([], dtype=float64), array([], dtype=float64)),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00128351,  0.02161742, -0.00963453,  0.00239276,  0.00239276,
           0.0124401 , -0.01015046,  0.00239276,  0.01856621, -0.01015046,
           0.01280842,  0.00278711,  0.00974334,  0.02744967,  0.00807182,
          -0.01006222, -0.0103015 , -0.0231836 , -0.02358692,  0.10842886,
          -0.00854654,  0.00417307,  0.01189294, -0.00031194, -0.00031194,
          -0.00440453,  0.00212942,  0.00162621, -0.01111919,  0.01069831,
           0.00829839,  0.00540437,  0.00575812,  0.0017426 ,  0.0017426 ,
           0.00892529,  0.01584402,  0.01216583,  0.00928118,  0.01584402,
           0.00685419,  0.00082477,  0.00939539, -0.00560331,  0.00459882,
          -0.0008288 , -0.01015046, -0.00560331, -0.0008288 ,  0.00871016])),
  'logFor H': (array([], dtype=float64), array([], dtype=float64)),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00128351,  0.0183142 ,  0.0183142 ,  0.0183142 ,  0.0183142 ,
           0.28234877,  0.28234877,  0.26383724,  0.2686663 ,  0.26945656,
           0.27866417,  0.25041736,  0.26363378,  0.27600875,  0.26999295,
           0.27718678,  0.28598334,  0.29775065,  0.30807409,  0.23405128,
           0.20295468,  0.32582068,  0.33180026,  0.33843393,  0.32957684,
           0.33843393,  0.32957684,  0.32957684,  0.33843393,  0.33843393,
           0.33843393,  0.33843393,  0.33843393,  0.33843393,  0.33843393,
           0.33843393,  0.33843393,  0.33843393,  0.33843393,  0.33843393,
           0.33843393,  0.33843393,  0.33843393,  0.33843393,  0.33843393,
           0.33843393,  0.31712192,  0.31712192,  0.31712192,  0.31712192])),
  'logComm H': (array([], dtype=float64), array([], dtype=float64)),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00128351,  0.02161742, -0.00963453,  0.00239276,  0.00239276,
          -0.01015046,  0.01337474, -0.01015046,  0.01453366, -0.0008288 ,
           0.00974334,  0.03705997, -0.00270001,  0.01632878, -0.00096682,
           0.01084693,  0.00417307, -0.00128351, -0.00759452, -0.00817169,
           0.00575812,  0.00409399,  0.01069147,  0.01603358,  0.00739834,
           0.01272396,  0.01274476,  0.01157962,  0.01194621, -0.00195521,
           0.02075921,  0.01100335,  0.018746  ,  0.02091521, -0.00560331,
           0.00437042, -0.00560331,  0.00685419,  0.01373514, -0.00042615,
          -0.0008288 ,  0.00383204, -0.00042615,  0.00459882, -0.00042615,
          -0.00042615, -0.00042615, -0.00042615, -0.00042615,  0.0040594 ])),
  'logHeat H': (array([], dtype=float64), array([], dtype=float64)),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.03930287, 0.03930287, 0.04596878, 0.04596878,
          0.03692454, 0.04439555, 0.03692454, 0.04497925, 0.04439555,
          0.03958192, 0.04439555, 0.03684411, 0.03609671, 0.04241297,
          0.03564247, 0.02816171, 0.03892228, 0.03895803, 0.03399481,
          0.03982277, 0.05038598, 0.03796078, 0.04269732, 0.027121  ,
          0.03796078, 0.03796078, 0.06894227, 0.04572683, 0.02263101,
          0.02263101, 0.03023694, 0.03023694, 0.01236991, 0.01236991,
          0.01236991, 0.01935606, 0.00435662, 0.01935606, 0.00699136,
          0.0216745 , 0.00699136, 0.02924532, 0.05961634, 0.0211341 ,
          0.05201244, 0.08332852, 0.06785157, 0.07673892, 0.07673641])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.04860245, 0.04384665, 0.04384665, 0.04384665,
          0.04384665, 0.04384665, 0.04384665, 0.04384665, 0.03489469,
          0.03958192, 0.04497925, 0.04497925, 0.04497925, 0.03895803,
          0.03895803, 0.03945302, 0.03393833, 0.03449425, 0.03982277,
          0.02809818, 0.04269732, 0.03796078, 0.04093895, 0.04572683,
          0.04093895, 0.04093895, 0.04093895, 0.04093895, 0.04093895,
          0.03629026, 0.01819068, 0.01236991, 0.01819068, 0.01236991,
          0.01596988, 0.01348573, 0.0216745 , 0.01635147, 0.0119372 ,
          0.01235984, 0.01988549, 0.01317692, 0.01717309, 0.02274904,
          0.05135248, 0.04837761, 0.05842341, 0.07853107, 0.04889115])),
  'RSP K': (array([], dtype=float64), array([], dtype=float64)),
  'FE K': (array([], dtype=float64), array([], dtype=float64)),
  'SP-CT H': (array([], dtype=float64), array([], dtype=float64)),
  'NormalizedHeat R': (array([], dtype=float64), array([], dtype=float64)),
  'PersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'ModifiedPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'HeatPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64))},
 'webkb_wisconsin': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([ 7.02328504e-04,  5.68771567e-02,  5.68771567e-02,  7.77584184e-05,
           3.01269529e-02,  3.01269529e-02,  3.01269529e-02,  3.01269529e-02,
           3.01269529e-02,  3.01269529e-02,  3.01269529e-02,  3.01269529e-02,
           3.01269529e-02,  6.36375082e-02,  6.36375082e-02,  6.36375082e-02,
           6.92096138e-02,  6.42868946e-02,  7.64244121e-02,  7.64244121e-02,
           7.64244121e-02,  7.64244121e-02,  7.64244121e-02,  7.64244121e-02,
           7.64244121e-02,  7.64244121e-02,  7.64244121e-02,  7.64244121e-02,
           7.64244121e-02,  5.33447372e-02,  5.71627303e-02,  5.71627303e-02,
          -1.14022237e-02, -8.25049027e-03,  5.46408676e-02,  6.72256446e-02,
           6.72256446e-02,  6.71051565e-02,  6.80128094e-02,  5.86570013e-02,
           5.72328918e-02,  6.05138534e-02,  2.51341917e-02,  4.51119146e-02,
           6.18473808e-02,  6.18451229e-02,  8.48604032e-02,  9.17367684e-02,
           6.14890779e-02,  7.87813783e-02,  2.23957232e-02])),
  'Walk H': (array([], dtype=float64), array([], dtype=float64)),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 7.02328504e-04,  6.63193939e-03,  6.63193939e-03, -2.76785372e-02,
          -3.04322883e-03, -5.17138641e-03, -5.17138641e-03, -7.99557102e-03,
           2.81862708e-03,  1.06578090e-02,  4.19250693e-03, -1.23219959e-02,
          -1.23219959e-02, -1.23219959e-02,  6.28621630e-03,  2.47894831e-04,
           2.47894831e-04,  5.29003701e-03,  1.48101655e-02,  7.72229579e-03,
           7.72229579e-03,  7.72229579e-03, -8.66156483e-03,  6.44724007e-03,
          -7.99557102e-03,  3.19161455e-02,  2.63462841e-03,  2.63462841e-03,
           2.63462841e-03,  2.63462841e-03, -1.04241399e-02,  2.33530253e-02,
           2.33530253e-02,  2.33530253e-02,  2.33530253e-02,  2.33530253e-02,
          -4.12861405e-05,  3.57026633e-02,  1.13188021e-02, -2.68080780e-02,
          -7.17966025e-03,  1.76834001e-02,  1.03512671e-02,  4.22290266e-03,
           6.31941798e-03,  1.56790279e-02,  1.56790279e-02, -4.00195062e-04,
           4.29984633e-03,  7.13769887e-03])),
  'logFor H': (array([], dtype=float64), array([], dtype=float64)),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00070233,  0.03012695,  0.03012695,  0.05687716,  0.06363751,
           0.06655439,  0.0725349 ,  0.07034429, -0.00076522,  0.05134678,
           0.03791319,  0.02912854,  0.0470314 ,  0.06276849,  0.07627918,
           0.06846746,  0.06846746,  0.04615486,  0.0476434 ,  0.0648402 ,
           0.08697626,  0.07518427,  0.07518427,  0.09148341,  0.07472321,
           0.07663407,  0.07663407,  0.08129167,  0.08129167,  0.08959427,
           0.08129167,  0.08129167,  0.08129167,  0.08129167,  0.08129167,
           0.08129167,  0.08129167,  0.08129167,  0.08129167,  0.08129167,
           0.08129167,  0.08129167,  0.08129167,  0.08129167,  0.08129167,
           0.08129167,  0.08268586,  0.08268586,  0.08268586,  0.08268586])),
  'logComm H': (array([], dtype=float64), array([], dtype=float64)),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.00070233,  0.00663194,  0.02247971, -0.00304323,  0.00010804,
           0.0197722 , -0.00799557, -0.012322  , -0.012322  ,  0.01874324,
           0.00671906, -0.012322  ,  0.01874324,  0.01816267,  0.01816267,
           0.00337184, -0.00083951,  0.01131517,  0.01582209,  0.01745702,
           0.00961972,  0.0112316 ,  0.00925599,  0.00925599,  0.00761206,
           0.01633095,  0.02752124,  0.01679113,  0.04146609,  0.02502575,
           0.02432967,  0.03963002,  0.00809224,  0.00019249,  0.01268892,
           0.01035127,  0.01035127,  0.01035127,  0.01567903,  0.01567903,
           0.01035127,  0.01035127,  0.01567903,  0.00429985,  0.01997125,
           0.01997125,  0.00429985,  0.01567903,  0.01997125,  0.01567903])),
  'logHeat H': (array([], dtype=float64), array([], dtype=float64)),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.        ,  0.04058637,  0.04058637,  0.02333275,  0.02531918,
           0.01350076,  0.01671546, -0.01276149,  0.00587834, -0.00303955,
           0.04871141, -0.01911924, -0.01497842,  0.00336491,  0.00946909,
           0.04622432,  0.06134235,  0.06156961,  0.0127522 ,  0.0127522 ,
           0.00704035,  0.0127522 ,  0.00886386,  0.00201344,  0.00320155,
           0.00320155, -0.00122635, -0.00122635, -0.00122635,  0.01506096,
           0.01506096,  0.01699065,  0.00324525,  0.01832927,  0.01864833,
           0.02276915,  0.02409801,  0.01902734,  0.01933191,  0.01062531,
           0.01062531,  0.01062531,  0.00924912,  0.0124814 ,  0.01219181,
           0.01250234,  0.01079516,  0.01252527,  0.01151666,  0.01235783])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.        ,  0.03255695,  0.01514933,  0.01514933,  0.04544787,
           0.03298552,  0.01064143, -0.02489666,  0.03560297, -0.00597052,
           0.04135831,  0.05158162,  0.04784936,  0.04784936,  0.04425001,
           0.00320155,  0.00320155,  0.00454044,  0.00247863,  0.00107027,
           0.00320155,  0.00320155,  0.00107027,  0.01843589,  0.02405229,
           0.01493888,  0.01795928,  0.01465635,  0.02671793,  0.01694933,
           0.01465635,  0.01465635,  0.01465635,  0.01062531,  0.01062531,
           0.01062531,  0.01193351,  0.01193351,  0.0124814 ,  0.0124814 ,
           0.01063895,  0.00994973,  0.01180367,  0.01079516,  0.01252527,
           0.01252527,  0.02021494,  0.02323707,  0.02235516,  0.03252189])),
  'RSP K': (array([], dtype=float64), array([], dtype=float64)),
  'FE K': (array([], dtype=float64), array([], dtype=float64)),
  'SP-CT H': (array([], dtype=float64), array([], dtype=float64)),
  'NormalizedHeat R': (array([], dtype=float64), array([], dtype=float64)),
  'PersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'ModifiedPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64)),
  'HeatPersonalizedPageRank R': (array([], dtype=float64),
   array([], dtype=float64))},
 'zachary': {'pWalk H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([-0.00655603,  0.88230245,  0.88230245,  0.88230245,  0.88230245,
           0.88230245,  0.88230245,  0.88230245,  0.88230245,  0.88230245,
           0.88230245,  0.88230245,  0.88230245,  0.88230245,  0.88230245,
           0.88230245,  0.88230245,  0.88230245,  0.88230245,  0.88230245,
           0.88230245,  0.88230245,  0.88230245,  0.77184688,  0.77184688,
           0.77184688,  0.77184688,  0.77184688,  0.77184688,  0.77184688,
           0.77184688,  0.77184688,  0.15346311,  0.15346311,  0.15346311,
           0.15346311,  0.15346311,  0.03185982,  0.15346311,  0.00562139,
           0.00562139,  0.03185982,  0.03185982,  0.00863375,  0.00863375,
           0.00863375,  0.00863375,  0.00863375,  0.00863375,  0.00863375,
           0.00863375])),
  'Walk H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([ 0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.3297491 ,  0.3297491 ,  0.3297491 ,  0.3297491 ,
           0.3297491 ,  0.3297491 ,  0.3297491 ,  0.19995817, -0.02747253,
          -0.02747253, -0.02747253, -0.02747253, -0.02747253, -0.02747253])),
  'For H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00655603,  0.66868606,  0.66868606,  0.66868606,  0.77167277,
           0.77167277,  0.77167277,  0.66868606,  0.88230245,  0.88230245,
           0.88230245,  0.88230245,  0.88230245,  0.88230245,  0.10954057,
           0.10954057,  0.10954057,  0.10954057,  0.10954057,  0.10954057,
           0.10954057,  0.10954057,  0.10954057,  0.77184688,  0.77184688,
           0.66868606,  0.40337284,  0.88225754,  0.40337284,  0.10954057,
           0.10954057,  0.15346311,  0.15346311,  0.15346311,  0.15346311,
           0.15346311,  0.15346311,  0.88230245,  0.88230245,  0.88230245,
           0.88230245,  0.88230245,  0.88230245,  0.88230245,  0.88230245,
           0.88230245,  0.88230245,  0.88230245,  0.88230245,  0.88230245])),
  'logFor H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245])),
  'Comm H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00655603,  0.88230245,  0.88230245,  0.88230245,  0.88230245,
           0.88230245,  0.88230245,  0.88230245,  0.77184688,  0.77184688,
           0.77184688,  0.77184688,  0.77184688,  0.88230245,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  0.03185982,  0.03185982,  0.03185982,  0.03185982,
           0.00863375,  0.00863375,  0.00863375,  0.00863375,  0.00863375,
           0.00863375,  0.00863375,  0.00863375,  0.00863375,  0.00863375,
           0.00863375,  0.00863375,  0.00863375,  0.00863375,  0.00863375,
           0.00863375,  0.00863375,  0.00863375,  0.00863375,  0.00863375,
           0.00863375,  0.00863375,  0.00863375,  0.00863375,  0.00863375])),
  'logComm H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([ 0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.66868606,  0.66868606,  0.66868606,  0.40337284,  0.3297491 ,
           0.3297491 ,  0.3297491 ,  0.3297491 ,  0.3297491 ,  0.19995817,
          -0.02747253, -0.02747253, -0.02747253, -0.02747253, -0.02747253,
          -0.02747253, -0.02747253, -0.02747253, -0.02747253, -0.02747253,
          -0.02747253, -0.02747253, -0.02747253, -0.02747253, -0.02747253,
          -0.02747253, -0.02747253, -0.02747253, -0.02747253])),
  'Heat H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([-0.00655603,  0.66868606,  0.66868606,  0.77167277,  0.77167277,
           0.77167277,  0.77167277,  0.77167277,  0.77167277,  0.66868606,
           0.66868606,  0.66868606,  0.88230245,  0.88230245,  0.88230245,
           0.88230245,  0.48343526,  0.48343526,  0.40337284,  0.88230245,
           0.88230245,  0.88225754,  0.88230245,  0.10954057,  0.15346311,
           0.15346311,  0.15346311,  0.10954057,  0.10954057,  0.10954057,
           0.88230245,  0.88230245,  0.15346311,  0.88230245,  0.88230245,
           0.88230245,  0.66868606,  0.66868606,  0.88230245,  0.88230245,
           0.40337284,  0.66868606,  0.66868606,  0.66868606,  0.66868606,
           0.40337284,  0.40337284,  0.40337284,  0.40337284,  0.40337284])),
  'logHeat H': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.40337284, 0.40337284])),
  'SCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 1.        , 1.        , 1.        ])),
  'SCCT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.        , 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 1.        , 1.        ,
          1.        , 1.        , 1.        , 1.        , 1.        ])),
  'RSP K': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245])),
  'FE K': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94, 0.96, 0.98]),
   array([0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245])),
  'SP-CT H': (array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
          0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
          0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
          0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
          0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  ]),
   array([0.88230245, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606])),
  'NormalizedHeat R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86, 0.88,
          0.9 , 0.92, 0.94]),
   array([0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245])),
  'PersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245])),
  'ModifiedPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66]),
   array([0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245])),
  'HeatPersonalizedPageRank R': (array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
          0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44,
          0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64, 0.66,
          0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86]),
   array([0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.66868606, 0.66868606, 0.66868606, 0.66868606,
          0.66868606, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245, 0.88230245, 0.88230245,
          0.88230245, 0.88230245, 0.88230245]))}}

In [20]:
for d_name, d_values in results.items():
    for m_name, m_values in d_values.items():
        print("{}\t{}\t{}".format(d_name, m_name, np.max(m_values[1]) if len(m_values[1]) > 0 else '-'))

cora	pWalk H	0.07740777
cora	Walk H	-
cora	For H	0.114810621
cora	logFor H	-
cora	Comm H	0.07892732
cora	logComm H	-
cora	Heat H	0.00745664435
cora	logHeat H	-
cora	SCT H	0.40148547
cora	SCCT H	0.000777293437
cora	RSP K	-
cora	FE K	-
cora	SP-CT H	-
cora	NormalizedHeat R	-
cora	PersonalizedPageRank R	-
cora	ModifiedPersonalizedPageRank R	-
cora	HeatPersonalizedPageRank R	-
football	pWalk H	0.90633416
football	Walk H	0.90441989
football	For H	0.89665001
football	logFor H	0.89665001
football	Comm H	0.90937709
football	logComm H	0.90475255
football	Heat H	0.90589882
football	logHeat H	0.89494799
football	SCT H	0.89665001
football	SCCT H	0.9004381
football	RSP K	0.89854076
football	FE K	0.90234207
football	SP-CT H	0.89854076
football	NormalizedHeat R	0.89665001
football	PersonalizedPageRank R	0.89836389
football	ModifiedPersonalizedPageRank R	0.89836389
football	HeatPersonalizedPageRank R	0.89889265
news_2cl_1	pWalk H	0.13486726
news_2cl_1	Walk H	0.03368033
news_2cl_1	For H	5.01278134e-05
n